In [1]:
import sys
import os
sys.path.append('..')

from incf.incf import MainArea
import param

import io
import panel as pn
import numpy as np
import pandas as pd

pn.extension('tabulator', 'ace', 'jsoneditor', 'ipywidgets', sizing_mode='stretch_width', notifications=True)

NotificationArea(sizing_mode='stretch_width')

In [2]:
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

In [3]:
path = '../output'
sys.path.append(path)

In [5]:
test = MainArea()
pn.serve(test.view()) 


Launching server at http://localhost:65236


OrderedDict([('01', {'ses-preop': OrderedDict([('areas.txt', {'name': 'areas', 'fname': 'areas.txt', 'sid': '01', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\areas.txt', 'ext': 'txt'}), ('average_orientations.txt', {'name': 'average_orientations', 'fname': 'average_orientations.txt', 'sid': '01', 'desc': 'default', 'sep': '\\s', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\average_orientations.txt', 'ext': 'txt'}), ('centres.txt', {'name': 'centres', 'fname': 'centres.txt', 'sid': '01', 'desc': 'default', 'sep': '\\s', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\centres.txt', 'ext': 'txt'}), ('cortical.txt', {'name': 'cortical', 'fname': 'cortical.txt', 'sid': '01', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\cortical.txt', 'ext': 'txt'}), ('distances.txt', {'name': 'distanc

OrderedDict([('01', {'ses-preop': OrderedDict([('areas.txt', {'name': 'areas', 'fname': 'areas.txt', 'sid': '01', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\areas.txt', 'ext': 'txt'}), ('average_orientations.txt', {'name': 'average_orientations', 'fname': 'average_orientations.txt', 'sid': '01', 'desc': 'default', 'sep': '\\s', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\average_orientations.txt', 'ext': 'txt'}), ('centres.txt', {'name': 'centres', 'fname': 'centres.txt', 'sid': '01', 'desc': 'default', 'sep': '\\s', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\centres.txt', 'ext': 'txt'}), ('cortical.txt', {'name': 'cortical', 'fname': 'cortical.txt', 'sid': '01', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\cortical.txt', 'ext': 'txt'}), ('distances.txt', {'name': 'distanc

OrderedDict([('01', {'ses-preop': OrderedDict([('areas.txt', {'name': 'areas', 'fname': 'areas.txt', 'sid': '01', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\areas.txt', 'ext': 'txt'}), ('average_orientations.txt', {'name': 'average_orientations', 'fname': 'average_orientations.txt', 'sid': '01', 'desc': 'default', 'sep': '\\s', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\average_orientations.txt', 'ext': 'txt'}), ('centres.txt', {'name': 'centres', 'fname': 'centres.txt', 'sid': '01', 'desc': 'default', 'sep': '\\s', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\centres.txt', 'ext': 'txt'}), ('cortical.txt', {'name': 'cortical', 'fname': 'cortical.txt', 'sid': '01', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\cortical.txt', 'ext': 'txt'}), ('distances.txt', {'name': 'distanc

In [5]:
test = MainArea()
pn.serve(test.view()) 


Launching server at http://localhost:56759


to_extract: ['weights.txt', 'centres.txt', 'distances.txt', 'average_orientations.txt', 'areas.txt', 'cortical.txt', 'hemisphere.txt']
contents: ['areas.txt', 'average_orientations.txt', 'centres.txt', 'cortical.txt', 'FC.mat', 'hemisphere.txt', 'HRF.csv', 'HRF.mat', 'ROIts.dat', 'SC.zip', 'SCthrAn.mat', 'weights.txt']
to_extract: ['weights.txt', 'centres.txt', 'distances.txt', 'average_orientations.txt', 'areas.txt', 'cortical.txt', 'hemisphere.txt']
contents: ['areas.txt', 'average_orientations.txt', 'centres.txt', 'cortical.txt', 'FC.mat', 'hemisphere.txt', 'HRF.csv', 'HRF.mat', 'ROI.dat', 'SC.zip', 'SCthrAn.mat', 'weights.txt']
to_extract: ['weights.txt', 'centres.txt', 'distances.txt', 'average_orientations.txt', 'areas.txt', 'cortical.txt', 'hemisphere.txt']
contents: ['areas.txt', 'average_orientations.txt', 'centres.txt', 'cortical.txt', 'FC.mat', 'hemisphere.txt', 'HRF.csv', 'HRF.mat', 'ROIts.dat', 'SC.zip', 'SCthrAn.mat', 'weights.txt']
to_extract: ['weights.txt', 'centres.tx

In [6]:
s = pn.widgets.StaticText(value='|___ output/<br>&emsp;&emsp;|___code/<br>&emsp;&emsp;|___coord/<br>&emsp;&emsp;|___eq/<br>&emsp;&emsp;|___param/<br>&emsp;&emsp;|___sub-01/<br>&emsp;&emsp;&emsp;&emsp;|___ses-preop/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___net/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-01_desc-default_weights.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-01_desc-default_weights.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___ts/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-01_desc-default_FC.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-01_desc-default_FC.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-01_desc-default_HRF.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-01_desc-default_HRF.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-01_desc-default_SCthrAn.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-01_desc-default_SCthrAn.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___spatial/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___coord/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.json<br>&emsp;&emsp;|___sub-02/<br>&emsp;&emsp;&emsp;&emsp;|___ses-preop/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___net/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-02_desc-default_weights.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-02_desc-default_weights.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___ts/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-02_desc-default_FC.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-02_desc-default_FC.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-02_desc-default_HRF.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-02_desc-default_HRF.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-02_desc-default_SCthrAn.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-02_desc-default_SCthrAn.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___spatial/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___coord/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.json<br>&emsp;&emsp;&emsp;&emsp;|___ses-postop/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___net/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-02_desc-default_weights.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-02_desc-default_weights.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___ts/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-02_desc-default_FC.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-02_desc-default_FC.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-02_desc-default_HRF.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-02_desc-default_HRF.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-02_desc-default_SCthrAn.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-02_desc-default_SCthrAn.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___spatial/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___coord/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.json<br>&emsp;&emsp;|___sub-03/<br>&emsp;&emsp;&emsp;&emsp;|___ses-preop/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___net/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-03_desc-default_weights.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-03_desc-default_weights.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___ts/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-03_desc-default_FC.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-03_desc-default_FC.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-03_desc-default_HRF.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-03_desc-default_HRF.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-03_desc-default_SCthrAn.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-03_desc-default_SCthrAn.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___spatial/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___coord/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.json<br>&emsp;&emsp;&emsp;&emsp;|___ses-postop/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___net/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-03_desc-default_weights.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-03_desc-default_weights.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___ts/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-03_desc-default_FC.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-03_desc-default_FC.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-03_desc-default_HRF.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-03_desc-default_HRF.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-03_desc-default_SCthrAn.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-03_desc-default_SCthrAn.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___spatial/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___coord/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.json<br>&emsp;&emsp;|___sub-04/<br>&emsp;&emsp;&emsp;&emsp;|___ses-preop/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___net/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-04_desc-default_weights.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-04_desc-default_weights.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___ts/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-04_desc-default_FC.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-04_desc-default_FC.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-04_desc-default_HRF.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-04_desc-default_HRF.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-04_desc-default_SCthrAn.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-04_desc-default_SCthrAn.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___spatial/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___coord/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.json<br>&emsp;&emsp;&emsp;&emsp;|___ses-postop/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___net/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-04_desc-default_weights.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-04_desc-default_weights.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___ts/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-04_desc-default_FC.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-04_desc-default_FC.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-04_desc-default_HRF.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-04_desc-default_HRF.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-04_desc-default_SCthrAn.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-04_desc-default_SCthrAn.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___spatial/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___coord/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___desc-default_times.json<br>|___CHANGES.txt<br>|___dataset_description.json<br>|___participants.tsv<br>|___README.txt<br>')
pn.serve(s)

Launching server at http://localhost:56667


OrderedDict([('01', {'ses-preop': OrderedDict([('areas.txt', {'name': 'areas', 'fname': 'areas.txt', 'sid': '01', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\areas.txt', 'ext': 'txt'}), ('average_orientations.txt', {'name': 'average_orientations', 'fname': 'average_orientations.txt', 'sid': '01', 'desc': 'default', 'sep': '\\s', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\average_orientations.txt', 'ext': 'txt'}), ('centres.txt', {'name': 'centres', 'fname': 'centres.txt', 'sid': '01', 'desc': 'default', 'sep': '\\s', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\centres.txt', 'ext': 'txt'}), ('cortical.txt', {'name': 'cortical', 'fname': 'cortical.txt', 'sid': '01', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\cortical.txt', 'ext': 'txt'}), ('FC.mat', {'name': 'FC', 'fname': 'FC.mat', 'sid': '01', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\FC.mat', 'ext': 'mat'}), ('hemisphere.txt', {'name': 'hemisphere', 'fname': 'hemisphere.txt', 'sid': '01', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\hemisphere.txt', 'ext': 'txt'}), ('HRF.csv', {'name': 'HRF', 'fname': 'HRF.csv', 'sid': '01', 'desc': 'default', 'sep': ',', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\HRF.csv', 'ext': 'csv'}), ('HRF.mat', {'name': 'HRF', 'fname': 'HRF.mat', 'sid': '01', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\HRF.mat', 'ext': 'mat'}), ('ROIts.dat', {'name': 'ROIts', 'fname': 'ROIts.dat', 'sid': '01', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\ROIts.dat', 'ext': 'dat'}), ('SCthrAn.mat', {'name': 'SCthrAn', 'fname': 'SCthrAn.mat', 'sid': '01', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\SCthrAn.mat', 'ext': 'mat'}), ('weights.txt', {'name': 'weights', 'fname': 'weights.txt', 'sid': '01', 'desc': 'default', 'sep': '\\s', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\weights.txt', 'ext': 'txt'})])}), ('02', {'ses-preop': OrderedDict([('areas.txt', {'name': 'areas', 'fname': 'areas.txt', 'sid': '02', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON02\\ses-preop\\areas.txt', 'ext': 'txt'}), ('average_orientations.txt', {'name': 'average_orientations', 'fname': 'average_orientations.txt', 'sid': '02', 'desc': 'default', 'sep': '\\s', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON02\\ses-preop\\average_orientations.txt', 'ext': 'txt'}), ('centres.txt', {'name': 'centres', 'fname': 'centres.txt', 'sid': '02', 'desc': 'default', 'sep': '\\s', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON02\\ses-preop\\centres.txt', 'ext': 'txt'}), ('cortical.txt', {'name': 'cortical', 'fname': 'cortical.txt', 'sid': '02', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON02\\ses-preop\\cortical.txt', 'ext': 'txt'}), ('FC.mat', {'name': 'FC', 'fname': 'FC.mat', 'sid': '02', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON02\\ses-preop\\FC.mat', 'ext': 'mat'}), ('hemisphere.txt', {'name': 'hemisphere', 'fname': 'hemisphere.txt', 'sid': '02', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON02\\ses-preop\\hemisphere.txt', 'ext': 'txt'}), ('HRF.csv', {'name': 'HRF', 'fname': 'HRF.csv', 'sid': '02', 'desc': 'default', 'sep': ',', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON02\\ses-preop\\HRF.csv', 'ext': 'csv'}), ('HRF.mat', {'name': 'HRF', 'fname': 'HRF.mat', 'sid': '02', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON02\\ses-preop\\HRF.mat', 'ext': 'mat'}), ('ROIts.dat', {'name': 'ROIts', 'fname': 'ROIts.dat', 'sid': '02', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON02\\ses-preop\\ROIts.dat', 'ext': 'dat'}), ('SCthrAn.mat', {'name': 'SCthrAn', 'fname': 'SCthrAn.mat', 'sid': '02', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON02\\ses-preop\\SCthrAn.mat', 'ext': 'mat'}), ('weights.txt', {'name': 'weights', 'fname': 'weights.txt', 'sid': '02', 'desc': 'default', 'sep': '\\s', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON02\\ses-preop\\weights.txt', 'ext': 'txt'})]), 'ses-postop': OrderedDict([('areas.txt', {'name': 'areas', 'fname': 'areas.txt', 'sid': '02', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON02\\ses-postop\\areas.txt', 'ext': 'txt'}), ('average_orientations.txt', {'name': 'average_orientations', 'fname': 'average_orientations.txt', 'sid': '02', 'desc': 'default', 'sep': '\\s', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON02\\ses-postop\\average_orientations.txt', 'ext': 'txt'}), ('centres.txt', {'name': 'centres', 'fname': 'centres.txt', 'sid': '02', 'desc': 'default', 'sep': '\\s', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON02\\ses-postop\\centres.txt', 'ext': 'txt'}), ('cortical.txt', {'name': 'cortical', 'fname': 'cortical.txt', 'sid': '02', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON02\\ses-postop\\cortical.txt', 'ext': 'txt'}), ('FC.mat', {'name': 'FC', 'fname': 'FC.mat', 'sid': '02', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON02\\ses-postop\\FC.mat', 'ext': 'mat'}), ('hemisphere.txt', {'name': 'hemisphere', 'fname': 'hemisphere.txt', 'sid': '02', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON02\\ses-postop\\hemisphere.txt', 'ext': 'txt'}), ('HRF.csv', {'name': 'HRF', 'fname': 'HRF.csv', 'sid': '02', 'desc': 'default', 'sep': ',', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON02\\ses-postop\\HRF.csv', 'ext': 'csv'}), ('HRF.mat', {'name': 'HRF', 'fname': 'HRF.mat', 'sid': '02', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON02\\ses-postop\\HRF.mat', 'ext': 'mat'}), ('ROI.dat', {'name': 'ROI', 'fname': 'ROI.dat', 'sid': '02', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON02\\ses-postop\\ROI.dat', 'ext': 'dat'}), ('SCthrAn.mat', {'name': 'SCthrAn', 'fname': 'SCthrAn.mat', 'sid': '02', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON02\\ses-postop\\SCthrAn.mat', 'ext': 'mat'}), ('weights.txt', {'name': 'weights', 'fname': 'weights.txt', 'sid': '02', 'desc': 'default', 'sep': '\\s', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON02\\ses-postop\\weights.txt', 'ext': 'txt'})])}), ('03', {'ses-preop': OrderedDict([('areas.txt', {'name': 'areas', 'fname': 'areas.txt', 'sid': '03', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON03\\ses-preop\\areas.txt', 'ext': 'txt'}), ('average_orientations.txt', {'name': 'average_orientations', 'fname': 'average_orientations.txt', 'sid': '03', 'desc': 'default', 'sep': '\\s', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON03\\ses-preop\\average_orientations.txt', 'ext': 'txt'}), ('centres.txt', {'name': 'centres', 'fname': 'centres.txt', 'sid': '03', 'desc': 'default', 'sep': '\\s', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON03\\ses-preop\\centres.txt', 'ext': 'txt'}), ('cortical.txt', {'name': 'cortical', 'fname': 'cortical.txt', 'sid': '03', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON03\\ses-preop\\cortical.txt', 'ext': 'txt'}), ('FC.mat', {'name': 'FC', 'fname': 'FC.mat', 'sid': '03', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON03\\ses-preop\\FC.mat', 'ext': 'mat'}), ('hemisphere.txt', {'name': 'hemisphere', 'fname': 'hemisphere.txt', 'sid': '03', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON03\\ses-preop\\hemisphere.txt', 'ext': 'txt'}), ('HRF.csv', {'name': 'HRF', 'fname': 'HRF.csv', 'sid': '03', 'desc': 'default', 'sep': ',', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON03\\ses-preop\\HRF.csv', 'ext': 'csv'}), ('HRF.mat', {'name': 'HRF', 'fname': 'HRF.mat', 'sid': '03', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON03\\ses-preop\\HRF.mat', 'ext': 'mat'}), ('ROIts.dat', {'name': 'ROIts', 'fname': 'ROIts.dat', 'sid': '03', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON03\\ses-preop\\ROIts.dat', 'ext': 'dat'}), ('SCthrAn.mat', {'name': 'SCthrAn', 'fname': 'SCthrAn.mat', 'sid': '03', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON03\\ses-preop\\SCthrAn.mat', 'ext': 'mat'}), ('weights.txt', {'name': 'weights', 'fname': 'weights.txt', 'sid': '03', 'desc': 'default', 'sep': '\\s', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON03\\ses-preop\\weights.txt', 'ext': 'txt'})]), 'ses-postop': OrderedDict([('areas.txt', {'name': 'areas', 'fname': 'areas.txt', 'sid': '03', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON03\\ses-postop\\areas.txt', 'ext': 'txt'}), ('average_orientations.txt', {'name': 'average_orientations', 'fname': 'average_orientations.txt', 'sid': '03', 'desc': 'default', 'sep': '\\s', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON03\\ses-postop\\average_orientations.txt', 'ext': 'txt'}), ('centres.txt', {'name': 'centres', 'fname': 'centres.txt', 'sid': '03', 'desc': 'default', 'sep': '\\s', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON03\\ses-postop\\centres.txt', 'ext': 'txt'}), ('cortical.txt', {'name': 'cortical', 'fname': 'cortical.txt', 'sid': '03', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON03\\ses-postop\\cortical.txt', 'ext': 'txt'}), ('FC.mat', {'name': 'FC', 'fname': 'FC.mat', 'sid': '03', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON03\\ses-postop\\FC.mat', 'ext': 'mat'}), ('hemisphere.txt', {'name': 'hemisphere', 'fname': 'hemisphere.txt', 'sid': '03', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON03\\ses-postop\\hemisphere.txt', 'ext': 'txt'}), ('HRF.csv', {'name': 'HRF', 'fname': 'HRF.csv', 'sid': '03', 'desc': 'default', 'sep': ',', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON03\\ses-postop\\HRF.csv', 'ext': 'csv'}), ('HRF.mat', {'name': 'HRF', 'fname': 'HRF.mat', 'sid': '03', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON03\\ses-postop\\HRF.mat', 'ext': 'mat'}), ('ROIts.dat', {'name': 'ROIts', 'fname': 'ROIts.dat', 'sid': '03', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON03\\ses-postop\\ROIts.dat', 'ext': 'dat'}), ('SCthrAn.mat', {'name': 'SCthrAn', 'fname': 'SCthrAn.mat', 'sid': '03', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON03\\ses-postop\\SCthrAn.mat', 'ext': 'mat'}), ('weights.txt', {'name': 'weights', 'fname': 'weights.txt', 'sid': '03', 'desc': 'default', 'sep': '\\s', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON03\\ses-postop\\weights.txt', 'ext': 'txt'})])}), ('04', {'ses-preop': OrderedDict([('areas.txt', {'name': 'areas', 'fname': 'areas.txt', 'sid': '04', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON04\\ses-preop\\areas.txt', 'ext': 'txt'}), ('average_orientations.txt', {'name': 'average_orientations', 'fname': 'average_orientations.txt', 'sid': '04', 'desc': 'default', 'sep': '\\s', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON04\\ses-preop\\average_orientations.txt', 'ext': 'txt'}), ('centres.txt', {'name': 'centres', 'fname': 'centres.txt', 'sid': '04', 'desc': 'default', 'sep': '\\s', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON04\\ses-preop\\centres.txt', 'ext': 'txt'}), ('cortical.txt', {'name': 'cortical', 'fname': 'cortical.txt', 'sid': '04', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON04\\ses-preop\\cortical.txt', 'ext': 'txt'}), ('FC.mat', {'name': 'FC', 'fname': 'FC.mat', 'sid': '04', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON04\\ses-preop\\FC.mat', 'ext': 'mat'}), ('hemisphere.txt', {'name': 'hemisphere', 'fname': 'hemisphere.txt', 'sid': '04', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON04\\ses-preop\\hemisphere.txt', 'ext': 'txt'}), ('HRF.csv', {'name': 'HRF', 'fname': 'HRF.csv', 'sid': '04', 'desc': 'default', 'sep': ',', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON04\\ses-preop\\HRF.csv', 'ext': 'csv'}), ('HRF.mat', {'name': 'HRF', 'fname': 'HRF.mat', 'sid': '04', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON04\\ses-preop\\HRF.mat', 'ext': 'mat'}), ('ROIts.dat', {'name': 'ROIts', 'fname': 'ROIts.dat', 'sid': '04', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON04\\ses-preop\\ROIts.dat', 'ext': 'dat'}), ('SCthrAn.mat', {'name': 'SCthrAn', 'fname': 'SCthrAn.mat', 'sid': '04', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON04\\ses-preop\\SCthrAn.mat', 'ext': 'mat'}), ('weights.txt', {'name': 'weights', 'fname': 'weights.txt', 'sid': '04', 'desc': 'default', 'sep': '\\s', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON04\\ses-preop\\weights.txt', 'ext': 'txt'})]), 'ses-postop': OrderedDict([('areas.txt', {'name': 'areas', 'fname': 'areas.txt', 'sid': '04', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON04\\ses-postop\\areas.txt', 'ext': 'txt'}), ('average_orientations.txt', {'name': 'average_orientations', 'fname': 'average_orientations.txt', 'sid': '04', 'desc': 'default', 'sep': '\\s', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON04\\ses-postop\\average_orientations.txt', 'ext': 'txt'}), ('centres.txt', {'name': 'centres', 'fname': 'centres.txt', 'sid': '04', 'desc': 'default', 'sep': '\\s', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON04\\ses-postop\\centres.txt', 'ext': 'txt'}), ('cortical.txt', {'name': 'cortical', 'fname': 'cortical.txt', 'sid': '04', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON04\\ses-postop\\cortical.txt', 'ext': 'txt'}), ('FC.mat', {'name': 'FC', 'fname': 'FC.mat', 'sid': '04', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON04\\ses-postop\\FC.mat', 'ext': 'mat'}), ('hemisphere.txt', {'name': 'hemisphere', 'fname': 'hemisphere.txt', 'sid': '04', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON04\\ses-postop\\hemisphere.txt', 'ext': 'txt'}), ('HRF.csv', {'name': 'HRF', 'fname': 'HRF.csv', 'sid': '04', 'desc': 'default', 'sep': ',', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON04\\ses-postop\\HRF.csv', 'ext': 'csv'}), ('HRF.mat', {'name': 'HRF', 'fname': 'HRF.mat', 'sid': '04', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON04\\ses-postop\\HRF.mat', 'ext': 'mat'}), ('ROIts.dat', {'name': 'ROIts', 'fname': 'ROIts.dat', 'sid': '04', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON04\\ses-postop\\ROIts.dat', 'ext': 'dat'}), ('SCthrAn.mat', {'name': 'SCthrAn', 'fname': 'SCthrAn.mat', 'sid': '04', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON04\\ses-postop\\SCthrAn.mat', 'ext': 'mat'}), ('weights.txt', {'name': 'weights', 'fname': 'weights.txt', 'sid': '04', 'desc': 'default', 'sep': '\\s', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON04\\ses-postop\\weights.txt', 'ext': 'txt'})])})])

In [8]:
import pandas as pd

cortical = pd.read_csv('C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\cortical.txt')

In [9]:
hemisphere = pd.read_csv('C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\hemisphere.txt')

In [14]:
import csv 

with open('C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\cortical.txt') as f:
    sniffer = csv.Sniffer()
    print(sniffer.sniff(f.read(50)).delimiter)


1


In [15]:
cortical

,1
0,1
1,1
2,1
3,1
4,1
...,...
62,1
63,1
64,1
65,1


In [5]:
import os
import csv
import re

import pandas as pd
import panel as pn

from incf.convert import convert
import os
from collections import OrderedDict
import incf.preprocess.preprocess as prep
import incf.convert.convert as conv


class Files:
    def __init__(self, path, files):
        self.path = path
        self.selected_files = files
        self.subs = OrderedDict()

        # decide whether input files are for one or more patients
        self.content = conv.get_content(path, files)
        self.basename = set(conv.get_content(path, files, basename=True))
        self.single = len(self.content) == len(self.basename)


        # set multi-subject input to true
        conv.MULTI_INPUT = False if self.single else True

        # traverse files and create subjects
        self.traverse_files()

    def traverse_files(self):
        # traverse multi-folder input
        if conv.MULTI_INPUT:
            files = self.selected_files
            changed_path = False

            # Step 1: change directory to the folders if only one folder is listed
            if len(self.selected_files) == 1:
                changed_path = True
                files = os.listdir(os.path.join(self.path, self.selected_files[0]))

            # Step 1: traverse over the provided input
            for file in files:
                # Step 2: create individual ID; if it already has BIDS format, leave it as is but remove 'sub-'
                sid = prep.create_uuid()

                # Step 3: create a dictionary to store values
                if sid not in self.subs.keys():
                    self.subs[sid] = {}

                # Step 4: change path to the files
                if changed_path:
                    path = os.path.join(self.path, self.selected_files[0], file)
                else:
                    path = os.path.join(self.path, file)

                # Step 5: get all content
                all_files = os.listdir(path)

                # Step 6: traverse ses-preop if present
                if 'ses-preop' in all_files:
                    if 'ses-preop' not in self.subs[sid]:
                        self.subs[sid]['ses-preop'] = OrderedDict()
                        
                    self.subs[sid]['ses-preop'].update(prepare_subs(conv.get_content(path, 'ses-preop'), sid))

                # Step 7: traverse ses-postop if present
                if 'ses-postop' in all_files:
                    if 'ses-postop' not in self.subs[sid]:
                        self.subs[sid]['ses-postop'] = OrderedDict()
                        
                    self.subs[sid]['ses-postop'].update(prepare_subs(conv.get_content(path, 'ses-postop'), sid))


                # Step 8: if there are no `ses-preop` and `ses-postop`, traverse the folders as usual
                if 'ses-preop' not in all_files and 'ses-postop' not in all_files:
                    if changed_path:
                        path = path.replace(file, '')

                    if not os.path.exists(os.path.join(path, file)):
                        self.subs[sid] = prepare_subs(conv.get_content(self.path, file), sid)
                    else:
                        self.subs[sid] = prepare_subs(conv.get_content(path.replace(file, ''), file), sid)

        # traverse over single-subject and multi-subject in one folder structure
        else:
            # Step 1: check if the structure contains multi-subject
            match = find_matches(self.basename)

            if len(match) > 0:
                for k, v in get_unique_subs(match, self.content).items():
                    sid = prep.create_uuid()

                    if sid not in self.subs.keys():
                        self.subs[sid] = {}

                    if len(self.selected_files) == 1:
                        self.subs[sid].update(
                            prepare_subs([os.path.join(self.path, self.selected_files[0], x) for x in v], sid))
                    else:
                        self.subs[sid].update(prepare_subs([os.path.join(self.path, x) for x in v], sid))
            else:
                sid = prep.create_uuid()

                if len(self.selected_files) == 1 and os.path.isdir(os.path.join(self.path, self.selected_files[0])):
                    self.subs[sid] = prepare_subs(conv.get_content(self.path, self.selected_files), sid)
                else:
                    paths = [os.path.join(self.path, file) for file in self.selected_files]
                    self.subs[sid] = prepare_subs(paths, sid)


def find_matches(paths):
    unique_ids = []

    for path in paths:
        match = re.findall('^[A-Za-z]{2,}_[0-9]{2,}', path)
        if len(match) > 0:
            unique_ids.append(match[0])

    return list(set(unique_ids))


def get_unique_subs(match, contents):
    subs = OrderedDict()

    for idx in range(len(match)):
        subs[match[idx]] = [x for x in contents if match[idx] in x]

    return subs


def prepare_subs(file_paths, sid):
    subs = {}

    for file_path in file_paths:
        name = get_filename(file_path)
        desc = convert.DESC + 'h5' if file_path.endswith('h5') else convert.DESC

        subs[name] = {
            'name': name.split('.')[0],
            'fname': name,
            'sid': sid,
            'desc': desc,
            'sep': find_separator(file_path),
            'path': file_path,
            'ext': get_file_ext(file_path),
        }

        if subs[name]['name'] in ['tract_lengths', 'tract_lengths_preop', 'tract_lengths_postop']:
            subs[name]['name'] = 'distances'

        if 'tract_lengths' in file_path and not os.path.exists(file_path.replace('tract_lengths', 'distances')):
            os.replace(file_path, file_path.replace('tract_lengths', 'distances'))

        if subs[name]['name'] in ['centres', 'centers', 'centres_preop', 'centres_postop']:
            conv.CENTERS = True
    return subs


def get_filename(path):
    return os.path.basename(path)


def get_file_ext(path):
    return path.split('.')[-1]


def find_separator(path):
    """
    Find the separator/delimiter used in the file to ensure no exception
    is raised while reading files.
    :param path:
    :return:
    """
    if path.split('.')[-1] not in ['txt', 'csv']:
        return

    try:
        f = pd.read_csv(path)
    except pd.errors.EmptyDataError:
        pn.state.notifications.error(f'File {os.path.basename(path)} is empty! Creating an empty file...')
        return ''

    sniffer = csv.Sniffer()

    with open(path) as fp:
        try:
            delimiter = sniffer.sniff(fp.read(5000)).delimiter
        except Exception:
            delimiter = sniffer.sniff(fp.read(100)).delimiter
        except Exception:
            delimiter = sniffer.sniff(fp.read(500)).delimiter

    delimiter = '\s' if delimiter == ' ' else delimiter
    return delimiter


In [6]:
path = 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor'

subs = Files(path, os.listdir(path))
subs.subs

OrderedDict([('05',
              {'ses-preop': OrderedDict([('centres.txt',
                             {'name': 'centres',
                              'fname': 'centres.txt',
                              'sid': '05',
                              'desc': 'default',
                              'sep': '\\s',
                              'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\centres.txt',
                              'ext': 'txt'}),
                            ('distances.txt',
                             {'name': 'distances',
                              'fname': 'distances.txt',
                              'sid': '05',
                              'desc': 'default',
                              'sep': '',
                              'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\distances.txt',
                              'ext': 'txt'}),
                            ('FC.mat',
             

In [15]:
import os
import h5py as h
from pathlib import Path
from incf.convert import convert
from collections import OrderedDict
from incf.preprocess import simulations_h5 as sim

default_format, coord_format = 'sub-{}_desc-{}_{}.{}', 'desc-{}_{}.{}'


class FolderStructure:
    def __init__(self, path, subs):
        self.path = path
        self.subs = subs

        self.components = OrderedDict({
            'code': [],
            'coord': [],
            'eq': [],
            'param': [],
            'subjects': {},
            'files': ['CHANGES.txt', 'dataset_description.json', 'participants.tsv', 'README.txt']
        })

        self.layout = []
        self.default_format = 'sub-{}_desc-{}_{}.{}'
        self.coord_format = 'desc-{}_{}.{}'
        self.populate()

    def common_structure(self, v, name=None):
        name = v['name'] if name is None else name
        return [self.default_format.format(v['sid'], v['desc'], name, 'tsv'),
                self.default_format.format(v['sid'], v['desc'], name, 'json')]

    def coord_structure(self, v):
        return [self.coord_format.format(v['desc'], 'nodes', 'tsv'),
                self.coord_format.format(v['desc'], 'nodes', 'json'),
                self.coord_format.format(v['desc'], 'labels', 'tsv'),
                self.coord_format.format(v['desc'], 'labels', 'json')]

    def iterate(self, k, v, ses=None, sid=None):
        sid = v['sid'] if sid is None else sid

        if sid not in self.components['subjects'] and ses is None:
            self.components['subjects'][sid] = {'net': [], 'ts': [], 'spatial': []}

        # save weights, distances, and centres
        if ses is None:
            if k in ['weights.txt', 'distances.txt', 'tract_lengths.txt']:
                self.save_wd(v, sid)
            elif k in ['centers.txt']:
                self.save_centres(v)
            elif k.endswith('.mat'):
                self.save_mat(v, sid)
            elif k.endswith('.h5'):
                self.save_h5(v, sid)
        else:
            for k2, v2 in v.items():
                if k2 in ['weights.txt', 'distances.txt', 'tract_lengths.txt']:
                    self.save_wd(v2, sid, ses=ses)
                elif k2 in ['centers.txt']:
                    self.save_centres(v2, sid, ses=ses)
                elif k2.endswith('.mat'):
                    self.save_mat(v2, sid, ses=ses)
                elif k2.endswith('.h5'):
                    self.save_h5(v2, sid, ses=ses)

    def save_wd(self, v, sid, ses=None):
        if ses is None:
            self.components['subjects'][sid]['net'] += self.common_structure(v)
        else:
            self.components['subjects'][sid][ses]['net'] += self.common_structure(v)

    def save_centres(self, v, sid, ses=None):
        if ses is None:
            self.components['coord'] += self.coord_structure(v)
        else:
            del self.components['coord']
            self.components['subjects'][sid][ses]['coord'] += self.coord_structure(v)

    def save_mat(self, v, sid, ses=None):
        if ses is None:
            self.components['subjects'][sid]['ts'] += self.common_structure(v)
            self.components['subjects'][sid]['coord'] += [self.coord_format.format(v['desc'], 'times', 'tsv'),
                                                          self.coord_format.format(v['desc'], 'times', 'json')]
        else:
            self.components['subjects'][sid][ses]['ts'] += self.common_structure(v)
            self.components['subjects'][sid][ses]['coord'] += [self.coord_format.format(v['desc'], 'times', 'tsv'),
                                                               self.coord_format.format(v['desc'], 'times', 'json')]

    def save_h5(self, v, sid, ses=None):
        file = h.File(v['path'])
        keys = file.keys()
        name = v['fname'].split('_')[0].lower()

        if ses is None:
            sid = v['sid']
            if sid not in self.components['subjects']:
                self.components['subjects'][sid] = {'net': [], 'ts': [], 'spatial': []}

            if sim.check_params(file):
                self.components['subjects'][sid]['net'] += self.common_structure(v, 'weights')
                self.components['subjects'][sid]['net'] += self.common_structure(v, 'distances')
                self.components['coord'] += self.coord_structure(v)
            else:
                if len(list(keys)) > 0:
                    self.components['param'] += [self.coord_format.format(v['desc'], name, 'xml'),
                                                 self.coord_format.format(v['desc'], name, 'json')]

    def populate(self):
        ses_exists = False
        for k, v in self.subs.items():
            if 'sid' in v:
                # traverse single-instance files
                self.iterate(k, v)
            else:
                for k2, v2 in v.items():
                    if k not in self.components['subjects'].keys() and k2 in ['ses-preop', 'ses-postop']:
                        self.components['subjects'][k] = OrderedDict(
                            {'ses-preop': {'net': [], 'ts': [], 'spatial': [], 'coord': []},
                             'ses-postop': {'net': [], 'ts': [], 'spatial': [], 'coord': []}})
                    
                    if k2 == 'ses-preop':
                        ses_exists = True
                        self.iterate(k2, v2, sid=k, ses='ses-preop')
                    if k2 == 'ses-postop':
                        ses_exists = True
                        self.iterate(k2, v2, sid=k, ses='ses-postop')
                    if k2 not in ['ses-preop', 'ses-postop']:
                        self.iterate(k2, v2)
                        self.components['coord'] = list(set(self.components['coord']))
        
        if ses_exists:
            self.create_ses_layout()
        else:
            self.create_layout()

    def join(self, files, form='files'):
        subfile = '&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___{}<br>'
        file = '&emsp;&emsp;&emsp;&emsp;|___{}<br>'
        joiner = lambda x: ''.join(x)

        if form == 'files':
            return joiner([file.format(f) for f in files])
        return joiner([subfile.format(f) for f in files])

    def create_layout(self):
        fold = '&emsp;&emsp;|___{}/<br>'
        subfold = '&emsp;&emsp;&emsp;&emsp;|___{}/<br>'
        main_files = '|___{}<br>'

        self.layout.append('|___ output/<br>')

        for k, v in self.components.items():
            if len(v) == 0:
                self.layout.append(fold.format(k))
            else:
                if isinstance(v, list) and k != 'files':
                    self.layout += [fold.format(k), self.join(v)]
                elif isinstance(v, dict):
                    for k2, v2 in v.items():
                        self.layout += [fold.format(f'sub-{k2}'), subfold.format('net'),
                                        self.join(v2['net'], 'subfile'), subfold.format('ts'),
                                        self.join(v2['ts'], 'subfile'), subfold.format('spatial'),
                                        self.join(v2['spatial'], 'subfile')]

        self.layout += [main_files.format(x) for x in self.components['files']]
        self.layout = ''.join(self.layout)
        
    
    def create_ses_layout(self):
        fold = '&emsp;&emsp;|___{}/<br>'
        subfold = '&emsp;&emsp;&emsp;&emsp;|___{}/<br>'
        subfile = '&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___{}/<br>'
        main_files = '|___{}<br>'
        
        self.layout.append('|___ output/<br>')
        
        for k, v in self.components.items():
            if isinstance(v, list) and k != 'files':
                self.layout += [fold.format(k), self.join(v)]
            elif isinstance(v, dict):
                for k2, v2 in v.items(): 
                    layout = []
                    self.layout += [fold.format(f'sub-{k2}')]

                    for k3 in v2.keys():
                        if len(v2[k3]['net']) == 0:
                            continue
                        layout += [subfold.format(k3), 
                                  '&emsp;&emsp;' + subfold.format('net'), self.join(v2[k3]['net']), 
                                  '&emsp;&emsp;' + subfold.format('ts'), self.join(v2[k3]['ts'], 'subfile'),
                                  '&emsp;&emsp;' + subfold.format('spatial'), self.join(v2[k3]['spatial'], 'subfile'), 
                                  '&emsp;&emsp;' + subfold.format('coord'), self.join(v2[k3]['coord'], 'subfile')]
                        self.layout += layout
        
        self.layout += [main_files.format(x) for x in self.components['files']]
        self.layout = ''.join(self.layout)

def common_structure(v, name=None):
    name = v['name'] if name is None else name
    return [default_format.format(v['sid'], v['desc'], name, 'tsv'),
            default_format.format(v['sid'], v['desc'], name, 'json')]


def coord_structure(v):
    return [coord_format.format(v['desc'], 'nodes', 'tsv'),
            coord_format.format(v['desc'], 'nodes', 'json'),
            coord_format.format(v['desc'], 'labels', 'tsv'),
            coord_format.format(v['desc'], 'labels', 'json')]


def create_layout(subs=None, output='../output'):
    """
    Create folder structure according to BEP034 and passed `subs` parameter.
    :param output:
    :param subs:
    :return:
    """

    output = output.replace('.', '').replace('/', '')
    return FolderStructure(output, subs).layout


def create_sub_folders(path):
    sub = os.path.join(path, f'sub-{convert.SID}')
    net = os.path.join(sub, 'net')
    ts = os.path.join(sub, 'ts')
    spatial = os.path.join(sub, 'spatial')

    for file in [sub, net, ts, spatial]:
        if not os.path.exists(file):
            os.mkdir(file)


def check_folders(path):
    eq = os.path.join(path, 'eq')
    code = os.path.join(path, 'code')
    coord = os.path.join(path, 'coord')
    param = os.path.join(path, 'param')

    for p in [path, eq, code, coord, param]:
        if not os.path.exists(p):
            print(f'Creating folder `{os.path.basename(p)}`...')
            os.mkdir(p)

    read = os.path.join(path, 'README.txt')
    part = os.path.join(path, 'participants.tsv')
    desc = os.path.join(path, 'dataset_description.json')
    chgs = os.path.join(path, 'CHANGES.txt')

    for p in [read, part, desc, chgs]:
        if not os.path.exists(p):
            print(f'Creating file `{os.path.basename(p)}`...')
            Path(p).touch()


In [16]:
FolderStructure(path, subs.subs).layout

'|___ output/<br>&emsp;&emsp;|___code/<br>&emsp;&emsp;|___coord/<br>&emsp;&emsp;|___eq/<br>&emsp;&emsp;|___param/<br>&emsp;&emsp;|___sub-05/<br>&emsp;&emsp;&emsp;&emsp;|___ses-preop/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___net/<br>&emsp;&emsp;&emsp;&emsp;|___sub-05_desc-default_distances.tsv<br>&emsp;&emsp;&emsp;&emsp;|___sub-05_desc-default_distances.json<br>&emsp;&emsp;&emsp;&emsp;|___sub-05_desc-default_weights.tsv<br>&emsp;&emsp;&emsp;&emsp;|___sub-05_desc-default_weights.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___ts/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-05_desc-default_FC.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-05_desc-default_FC.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-05_desc-default_HRF.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-05_desc-default_HRF.json<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-05_desc-default_SCthrAn.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-05_desc-default_SCthrAn.json<br>&emsp;&emsp;&emsp;&emsp;&e

In [8]:
import scipy.io as sc

sc.loadmat('C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\FC_preop.mat')

{'__header__': b'MATLAB 5.0 MAT-file, written by Octave 4.0.2, 2017-02-21 10:35:45 UTC',
 '__version__': '1.0',
 '__globals__': [],
 'CON01T1_ROIts': array([[  923.70188, 10967.75413, 10075.33814, ...,  9686.10222,
         10282.49023, 10792.33884],
        [  916.27902, 10939.30918,  9958.87689, ...,  9665.7039 ,
         10121.32577, 10689.26117],
        [  906.29391, 10909.99712,  9918.38669, ...,  9593.44135,
         10135.87439, 10713.51698],
        ...,
        [  911.26295, 10948.92151,  9881.90784, ...,  9584.80664,
         10276.75591, 10826.52015],
        [  911.89234, 10926.43679,  9956.72355, ...,  9550.86363,
         10231.36131, 10768.50695],
        [  914.11457, 10917.22143,  9884.80337, ...,  9557.05501,
         10139.23922, 10715.65225]]),
 'CON01T1_ROIts_DK68': array([[12974.08918, 11915.09521, 12078.56191, ...,  9686.10222,
         10282.49023, 10792.33884],
        [12959.45346, 11819.84904, 12055.11605, ...,  9665.7039 ,
         10121.32577, 10689.26117]

In [ ]:

sc.loadmat('C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\FC_preop.mat')

In [ ]:
# editable json
        "ModelEq": "",
        "ModelParam": "",
        "SourceCode": "",
        "SourceCodeVersion": "",
        "SoftwareVersion": "",
        "SoftwareName": "",
        "SoftwareRepository": "",
        "Network": [],
        "Description": ""

In [122]:
from collections import OrderedDict
import json

JE_FIELDS = ['Units', 'Description', 'CoordsRows', 'CoordsColumns', 'ModelEq', 'ModelParam','SourceCode', 
             'SourceCodeVersion', 'SoftwareVersion', 'SoftwareName', 'SoftwareRepository', 'Network']
UNITS = ['s', 'm', 'ms', 'degrees', 'radians']


class ViewResults(param.Parameterized):
    options = ['JSON files', 'TSV files']
    file_selection = pn.widgets.RadioButtonGroup(options=options, button_type='primary', value=[])
    select_options = pn.widgets.Select()
    je_btn = param.Action(lambda self: self._update_je(), label='Update JSON file')
    
    def __init__(self):
        super().__init__()
        self.path = convert.OUTPUT
        self.layout = None
        self.widget = pn.WidgetBox('### Select File', self.select_options)
        self.je_widget = pn.WidgetBox()

    @pn.depends('file_selection.value', watch=True)
    def _change_filetype(self):
        if self.file_selection.value == 'JSON files':
            self.select_options.options = get_files(path=self.path)
        elif self.file_selection.value == 'TSV files':
            self.select_options.options = get_files(path=self.path, ftype='.tsv')

    @pn.depends('select_options.value', watch=True)
    def _change_file(self):
        if len(self.widget) > 2:
            self.widget.pop(-1)

        if self.file_selection.value == 'JSON files':
            try:
                file = json.load(open(self.select_options.value))
            except Exception:
                print(f'File `{self.select_options.value}` is empty!')
            else:
                if len(self.je_widget) > 0:
                    self.je_widget = pn.WidgetBox()
                
                je = pn.widgets.JSONEditor(value=file, height=350, mode='view')
                je_widget = get_settings(OrderedDict(je.value))                          
                self.je_widget.append(pn.Row(je, pn.Column(je_widget, 
                                                    pn.Param(self, parameters=['je_btn'], show_name=False, 
                                               widgets={'je_btn': {'button_type': 'primary'}}))))
                self.widget.append(self.je_widget)

        elif self.file_selection.value == 'TSV files':
            try:
                file = pd.read_csv(self.select_options.value, sep='\t', header=None, index_col=None)
            except Exception:
                print(f'File `{self.select_options.value}` is empty!')
            else:
                self.widget.append(pn.widgets.Tabulator(file))
        
    def _update_je(self, event=None):        
        txt_inputs = self.je_widget[0][1][0]
        json_editor = self.je_widget[0][0]
        
        for idx, inputs in enumerate(txt_inputs):
            value = inputs.value
            name = inputs.name.split(' ')[-1].replace(':', '')
            self.je_widget[0][0].value[name] = value
            self.widget.pop(-1)
            self.widget.append(self.je_widget)
            
            # save output
            with open(self.select_options.value, 'w') as f:
                json.dump(OrderedDict(self.je_widget[0][0].value), f)
    
    def view(self):
        return pn.Column(self.file_selection, self.widget)


def get_settings(json_editor):
    widget = pn.WidgetBox()
    
    for k, v in json_editor.items():
        name = f'Specify {k}:'
        
        if k == 'Units':
            widget.append(pn.widgets.Select(name=name, options=UNITS, value=''))
            continue
        
        elif k not in ['NumberOfColumns', 'NumberOfRows', 'Units'] and len(v) == 0:
            widget.append(pn.widgets.TextInput(name=name))
    
    # append button 
    return widget

    
def get_files(path='../output', ftype='.json'):
    f = []

    for root, dirs, files in os.walk(path, topdown=False):
        for name in files:
            if ftype in name:
                f.append(os.path.join(root, name))
    return f

In [123]:
import json
import os

import pandas as pd
import panel as pn
import param

import incf.preprocess.preprocess as prep
from incf.convert import convert

vr = ViewResults()

pn.serve(vr.view())

Launching server at http://localhost:51838


File `../output\coord\desc-default_labels.json` is empty!








[]


In [7]:
import os
import csv
import re

import pandas as pd
import panel as pn

from incf.convert import convert
import os
from collections import OrderedDict
import incf.preprocess.preprocess as prep
import incf.convert.convert as conv


class Files:
    def __init__(self, path, files):
        self.path = path
        self.selected_files = files
        self.subs = OrderedDict()

        # decide whether input files are for one or more patients
        self.content = conv.get_content(path, files)
        self.basename = set(conv.get_content(path, files, basename=True))
        self.single = len(self.content) == len(self.basename)
        
        print('self.content', self.content)
        print('self.basename', self.basename)

        # set multi-subject input to true
        conv.MULTI_INPUT = False if self.single else True
        
        # traverse files and create subjects 
        self.traverse_files()
        
    def traverse_files(self):
        # traverse multi-folder input
        if conv.MULTI_INPUT:
            files = self.selected_files
            changed_path = False
            
            # Step 1: change directory to the folders if only one folder is listed
            if len(self.selected_files) == 1:
                changed_path = True
                files = os.listdir(os.path.join(self.path, self.selected_files[0]))
            
            # Step 1: traverse over the provided input
            for file in files:
                # Step 2: create individual ID; if it already has BIDS format, leave it as is but remove 'sub-'
                sid = prep.create_uuid()
                
                # Step 3: create a dictionary to store values 
                if sid not in self.subs.keys():
                    self.subs[sid] = {}
                
                # Step 4: change path to the files
                if changed_path:
                    path = os.path.join(self.path, self.selected_files[0], file)
                else:
                    path = os.path.join(self.path, file)
                    
                # Step 5: get all content 
                all_files = os.listdir(path)
                
                # Step 6: traverse ses-preop if present
                if 'ses-preop' in all_files:
                    self.subs[sid].update(prepare_subs(conv.get_content(path, 'ses-preop'), sid))
                # Step 7: traverse ses-postop if present
                if 'ses-postop' in all_files:
                    self.subs[sid].update(prepare_subs(conv.get_content(path, 'ses-postop'), sid))
                # Step 8: if there are no `ses-preop` and `ses-postop`, traverse the folders as usual
                if 'ses-preop' not in all_files and 'ses-postop' not in all_files:
                    if changed_path:
                        path = path.replace(file, '')
                        
                    if not os.path.exists(os.path.join(path, file)):
                        self.subs[sid] = prepare_subs(conv.get_content(self.path, file), sid)
                    else:
                        self.subs[sid] = prepare_subs(conv.get_content(path.replace(file, ''), file), sid)
                    
        # traverse over single-subject and multi-subject in one folder structure        
        else:
            # Step 1: check if the structure contains multi-subject 
            match = find_matches(self.basename)
            
            if len(match) > 0: 
                for k, v in get_unique_subs(match, self.content).items():
                    sid = prep.create_uuid()

                    if sid not in self.subs.keys():
                        self.subs[sid] = {}
                    
                    if len(self.selected_files) == 1:
                        self.subs[sid].update(prepare_subs([os.path.join(self.path, self.selected_files[0], x) for x in v], sid))
                    else:
                        self.subs[sid].update(prepare_subs([os.path.join(self.path, x) for x in v], sid))
            else:
                sid = prep.create_uuid()
                print('self.path:', self.path)
                print('self.selected_files:', self.selected_files)
                
                if len(self.selected_files) == 1 and os.path.isdir(os.path.join(self.path, self.selected_files[0])):
                    self.subs[sid] = prepare_subs(conv.get_content(self.path, self.selected_files), sid)
                else:
                    paths = [os.path.join(self.path, file) for file in self.selected_files]
                    self.subs[sid] = prepare_subs(paths, sid)

        
def find_matches(paths):
    unique_ids = []
    
    for path in paths:
        match = re.findall('^[A-Za-z]{2,}_[0-9]{2,}', path)
        if len(match) > 0:
            unique_ids.append(match[0])

    return list(set(unique_ids))


def get_unique_subs(match, contents):
    subs = OrderedDict()
    
    for idx in range(len(match)):
        subs[match[idx]] = [x for x in contents if match[idx] in x]
    
    return subs


def prepare_subs(file_paths, sid):
    subs = {}

    accepted = ['tract_lengths.txt', 'weights.txt', 'centres.txt',
                'tract_lengths_preop.txt', 'weights_preop.txt', 'centres_preop.txt',
                'tract_lengths_postop.txt', 'weights_postop.txt', 'centres_postop.txt',
                'distances.txt', 'distances_preop.txt', 'distances_postop.txt']

    for file_path in file_paths:
        name = get_filename(file_path)
        desc = convert.DESC + 'h5' if file_path.endswith('h5') else convert.DESC

        subs[name] = {
            'name': name.split('.')[0],
            'fname': name,
            'sid': sid,
            'desc': desc,
            'sep': find_separator(file_path),
            'path': file_path,
            'ext': get_file_ext(file_path),
        }

        if subs[name]['name'] in ['tract_lengths', 'tract_lengths_preop', 'tract_lengths_postop']:
            subs[name]['name'] = 'distances'
            
        if 'tract_lengths' in file_path and not os.path.exists(file_path.replace('tract_lengths', 'distances')):
            os.replace(file_path, file_path.replace('tract_lengths', 'distances'))
            
        if subs[name]['name'] in ['centres', 'centers', 'centres_preop', 'centres_postop']:
            conv.CENTERS = True
    return subs
        


def get_filename(path):
    return os.path.basename(path)


def get_file_ext(path):
    return path.split('.')[-1]


def find_separator(path):
    """
    Find the separator/delimiter used in the file to ensure no exception
    is raised while reading files.
    :param path:
    :return:
    """
    if path.split('.')[-1] not in ['txt', 'csv']:
        return

    try:
        f = pd.read_csv(path)
    except pd.errors.EmptyDataError:
        pn.state.notifications.error(f'File {os.path.basename(path)} is empty! Creating an empty file...')
        return ''

    sniffer = csv.Sniffer()

    with open(path) as fp:
        try:
            delimiter = sniffer.sniff(fp.read(5000)).delimiter
        except Exception:
            delimiter = sniffer.sniff(fp.read(100)).delimiter
        except Exception:
            delimiter = sniffer.sniff(fp.read(500)).delimiter

    delimiter = '\s' if delimiter == ' ' else delimiter
    return delimiter


# files = Files('C:\\Users\\dinar\\Desktop\\gsoc_data', ['brain_tumor'])
# f = Files('C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor', os.listdir('C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor'))
# f.subs

In [9]:
path = 'C:\\Users\\dinar\\Desktop\\gsoc_data'

f = Files(path, ['Micha-TVB-paper'])
f.subs

self.content ['C:\\Users\\dinar\\Desktop\\gsoc_data\\Micha-TVB-paper\\AA_20120815_fMRI_new.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\Micha-TVB-paper\\AA_20120815_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\Micha-TVB-paper\\AC_20120917_fMRI_new.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\Micha-TVB-paper\\AC_20120917_SC_new.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\Micha-TVB-paper\\AC_20151014_1500_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\Micha-TVB-paper\\AJ_20140516_1600_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\Micha-TVB-paper\\AR_20120813_fMRI_new.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\Micha-TVB-paper\\AR_20120813_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\Micha-TVB-paper\\AY_20111004_fMRI_new.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\Micha-TVB-paper\\AY_20111004_SC_new.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\Micha-TVB-paper\\BQ_20120904_fMRI_new.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\Micha-TVB-paper\\BQ_20120904_SC_new.mat', 'C:\\Users

OrderedDict([(59,
              {'DA_20120813_fMRI_new.mat': {'name': 'DA_20120813_fMRI_new',
                'fname': 'DA_20120813_fMRI_new.mat',
                'sid': 59,
                'desc': 'default',
                'sep': None,
                'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\Micha-TVB-paper\\DA_20120813_fMRI_new.mat',
                'ext': 'mat'},
               'DA_20120813_SC.mat': {'name': 'DA_20120813_SC',
                'fname': 'DA_20120813_SC.mat',
                'sid': 59,
                'desc': 'default',
                'sep': None,
                'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\Micha-TVB-paper\\DA_20120813_SC.mat',
                'ext': 'mat'}}),
             (60,
              {'JA_20150803_1000_SC.mat': {'name': 'JA_20150803_1000_SC',
                'fname': 'JA_20150803_1000_SC.mat',
                'sid': 60,
                'desc': 'default',
                'sep': None,
                'path': 'C:\\Users\\dinar\\Desktop\\g

In [17]:
path = 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3'

files = Files(path, os.listdir(path))
files.subs

self.content ['AA_20120815_aparc_to_DWI_mrtrixlabels.csv', 'AA_20120815_SC.mat', 'AA_20120815_SCdist.csv', 'AC_20120917_aparc_to_DWI_mrtrixlabels.csv', 'AC_20120917_SC.mat', 'AC_20120917_SCdist.csv', 'AD_20120813_aparc_to_DWI_mrtrixlabels.csv', 'AD_20120813_SC.mat', 'AD_20120813_SCdist.csv', 'AR_20120813_aparc_to_DWI_mrtrixlabels.csv', 'AR_20120813_SC.mat', 'AR_20120813_SCdist.csv', 'AY_20111004_aparc_to_DWI_mrtrixlabels.csv', 'AY_20111004_SC.mat', 'AY_20111004_SCdist.csv', 'AY_20111004_SCstrength_sift2.csv', 'BQ_20120904_aparc_to_DWI_mrtrixlabels.csv', 'BQ_20120904_SC.mat', 'BQ_20120904_SCdist.csv', 'CN_20120927_aparc_to_DWI_mrtrixlabels.csv', 'CN_20120927_SC.mat', 'CN_20120927_SCdist.csv', 'DG_20120903_aparc_to_DWI_mrtrixlabels.csv', 'DG_20120903_SC.mat', 'DG_20120903_SCdist.csv', 'DH_20120806_aparc_to_DWI_mrtrixlabels.csv', 'DH_20120806_SC.mat', 'DH_20120806_SCdist.csv', 'DJ_20120810_aparc_to_DWI_mrtrixlabels.csv', 'DJ_20120810_SC.mat', 'DJ_20120810_SCdist.csv', 'FE_20111010_aparc_t

OrderedDict([(152,
              {'DH_20120806_aparc_to_DWI_mrtrixlabels.csv': {'name': 'DH_20120806_aparc_to_DWI_mrtrixlabels',
                'fname': 'DH_20120806_aparc_to_DWI_mrtrixlabels.csv',
                'sid': 152,
                'desc': 'default',
                'sep': '\\s',
                'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\DH_20120806_aparc_to_DWI_mrtrixlabels.csv',
                'ext': 'csv'},
               'DH_20120806_SC.mat': {'name': 'DH_20120806_SC',
                'fname': 'DH_20120806_SC.mat',
                'sid': 152,
                'desc': 'default',
                'sep': None,
                'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\DH_20120806_SC.mat',
                'ext': 'mat'},
               'DH_20120806_SCdist.csv': {'name': 'DH_20120806_SCdist',
                'fname': 'DH_20120806_SCdist.csv',
                'sid': 152,
                'desc': 'default',
                

In [18]:
path = 'C:\\Users\\dinar\\Desktop\\gsoc_data'

files = Files(path, ['New_Connectomes_MRTrix3'])
files.subs

self.content ['C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AA_20120815_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AA_20120815_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AA_20120815_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AC_20120917_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AC_20120917_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AC_20120917_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AD_20120813_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AD_20120813_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AD_20120813_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AR_20120813_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc

OrderedDict([(202,
              {'DH_20120806_aparc_to_DWI_mrtrixlabels.csv': {'name': 'DH_20120806_aparc_to_DWI_mrtrixlabels',
                'fname': 'DH_20120806_aparc_to_DWI_mrtrixlabels.csv',
                'sid': 202,
                'desc': 'default',
                'sep': '\\s',
                'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\DH_20120806_aparc_to_DWI_mrtrixlabels.csv',
                'ext': 'csv'},
               'DH_20120806_SC.mat': {'name': 'DH_20120806_SC',
                'fname': 'DH_20120806_SC.mat',
                'sid': 202,
                'desc': 'default',
                'sep': None,
                'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\DH_20120806_SC.mat',
                'ext': 'mat'},
               'DH_20120806_SCdist.csv': {'name': 'DH_20120806_SCdist',
                'fname': 'DH_20120806_SCdist.csv',
                'sid': 202,
                'desc': 'default',
                

In [33]:
path = 'C:\\Users\\dinar\\Desktop\\gsoc_data\\txt_files\\2'

files = Files(path, os.listdir(path))
files.subs

self.content ['centres.txt', 'distances.txt', 'weights.txt']
self.basename {'weights.txt', 'centres.txt', 'distances.txt'}
self.path: C:\Users\dinar\Desktop\gsoc_data\txt_files\2
self.selected_files: ['centres.txt', 'tract_lengths.txt', 'weights.txt']


OrderedDict([(257,
              {'centres.txt': {'name': 'centres',
                'fname': 'centres.txt',
                'sid': 257,
                'desc': 'default',
                'sep': '\t',
                'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\txt_files\\2\\centres.txt',
                'ext': 'txt'},
               'tract_lengths.txt': {'name': 'distances',
                'fname': 'tract_lengths.txt',
                'sid': 257,
                'desc': 'default',
                'sep': '\\s',
                'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\txt_files\\2\\tract_lengths.txt',
                'ext': 'txt'},
               'weights.txt': {'name': 'weights',
                'fname': 'weights.txt',
                'sid': 257,
                'desc': 'default',
                'sep': '\\s',
                'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\txt_files\\2\\weights.txt',
                'ext': 'txt'}})])

In [34]:
path = 'C:\\Users\\dinar\\Desktop\\gsoc_data\\txt_files'

files = Files(path, ['2'])
files.subs

self.content ['C:\\Users\\dinar\\Desktop\\gsoc_data\\txt_files\\2\\centres.txt', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\txt_files\\2\\distances.txt', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\txt_files\\2\\weights.txt']
self.basename {'weights.txt', 'centres.txt', 'distances.txt'}
self.path: C:\Users\dinar\Desktop\gsoc_data\txt_files
self.selected_files: ['2']


OrderedDict([(258,
              {'centres.txt': {'name': 'centres',
                'fname': 'centres.txt',
                'sid': 258,
                'desc': 'default',
                'sep': '\t',
                'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\txt_files\\2\\centres.txt',
                'ext': 'txt'},
               'distances.txt': {'name': 'distances',
                'fname': 'distances.txt',
                'sid': 258,
                'desc': 'default',
                'sep': '\\s',
                'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\txt_files\\2\\distances.txt',
                'ext': 'txt'},
               'weights.txt': {'name': 'weights',
                'fname': 'weights.txt',
                'sid': 258,
                'desc': 'default',
                'sep': '\\s',
                'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\txt_files\\2\\weights.txt',
                'ext': 'txt'}})])

In [14]:
path = 'C:\\Users\\dinar\\Desktop\\gsoc_data\\txt_files\\2'

# C:\Users\dinar\Desktop\gsoc_data\txt_files\2

files = Files(path, os.listdir(path)).subs

Not a match


In [15]:
find_matches(['C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AA_20120815_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AA_20120815_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AA_20120815_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AC_20120917_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AC_20120917_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AC_20120917_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AD_20120813_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AD_20120813_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AD_20120813_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AR_20120813_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AR_20120813_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AR_20120813_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AY_20111004_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AY_20111004_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AY_20111004_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AY_20111004_SCstrength_sift2.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\BQ_20120904_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\BQ_20120904_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\BQ_20120904_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\CN_20120927_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\CN_20120927_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\CN_20120927_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\convert_connectomes_2_SC.m', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\DG_20120903_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\DG_20120903_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\DG_20120903_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\DH_20120806_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\DH_20120806_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\DH_20120806_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\DJ_20120810_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\DJ_20120810_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\DJ_20120810_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\FE_20111010_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\FE_20111010_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\FE_20111010_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\FI_20120727_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\FI_20120727_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\FI_20120727_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\FJ_20120808_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\FJ_20120808_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\FJ_20120808_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\FR_20120903_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\FR_20120903_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\FR_20120903_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\GC_20120803_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\GC_20120803_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\GC_20120803_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\HA_20120813_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\HA_20120813_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\HA_20120813_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\IC_20120810_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\IC_20120810_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\IC_20120810_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\IQ_20120904_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\IQ_20120904_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\IQ_20120904_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\IS_20120809_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\IS_20120809_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\IS_20120809_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\JH_20120925_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\JH_20120925_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\JH_20120925_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\JH_20121009_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\JH_20121009_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\JH_20121009_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\JL_20120927_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\JL_20120927_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\JL_20120927_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\JS_20120910_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\JS_20120910_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\JS_20120910_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\JZ_20120824_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\JZ_20120824_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\JZ_20120824_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\KI_20121009_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\KI_20121009_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\KI_20121009_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\NI_20120831_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\NI_20120831_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\NI_20120831_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\NN_20120824_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\NN_20120824_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\NN_20120824_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\NN_20120831_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\NN_20120831_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\NN_20120831_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\OG_20120917_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\OG_20120917_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\OG_20120917_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\OK_20121011_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\OK_20121011_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\OK_20121011_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\OQ_20120925_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\OQ_20120925_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\OQ_20120925_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\QL_20110925_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\QL_20110925_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\QL_20110925_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\QL_20120814_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\QL_20120814_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\QL_20120814_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\QR_20111010_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\QR_20111010_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\QR_20111010_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\RE_20120816_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\RE_20120816_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\RE_20120816_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\RF_20120809_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\RF_20120809_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\RF_20120809_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\RI_20110924_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\RI_20110924_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\RI_20110924_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\RI_20120815_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\RI_20120815_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\RI_20120815_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\RL_20110925_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\RL_20110925_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\RL_20110925_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\RQ_20120903_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\RQ_20120903_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\RQ_20120903_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\RQ_20120917_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\RQ_20120917_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\RQ_20120917_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\RS_20120723_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\RS_20120723_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\RS_20120723_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\RT_20110925_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\RT_20110925_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\RT_20110925_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\SE_20110924_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\SE_20110924_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\SE_20110924_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\UB_20120806_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\UB_20120806_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\UB_20120806_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\UE_20120803_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\UE_20120803_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\UE_20120803_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\UK_20110924_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\UK_20110924_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\UK_20110924_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\UK_20111004_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\UK_20111004_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\UK_20111004_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\XB_20120831_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\XB_20120831_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\XB_20120831_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\YE_20120910_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\YE_20120910_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\YE_20120910_SCdist.csv'])

[]

In [21]:
files = OrderedDict([('UK_20111004', ['UK_20111004_aparc_to_DWI_mrtrixlabels.csv', 'UK_20111004_SC.mat', 'UK_20111004_SCdist.csv']), ('JS_20120910', ['JS_20120910_aparc_to_DWI_mrtrixlabels.csv', 'JS_20120910_SC.mat', 'JS_20120910_SCdist.csv']), ('OK_20121011', ['OK_20121011_aparc_to_DWI_mrtrixlabels.csv', 'OK_20121011_SC.mat', 'OK_20121011_SCdist.csv']), ('RI_20110924', ['RI_20110924_aparc_to_DWI_mrtrixlabels.csv', 'RI_20110924_SC.mat', 'RI_20110924_SCdist.csv']), ('RQ_20120903', ['RQ_20120903_aparc_to_DWI_mrtrixlabels.csv', 'RQ_20120903_SC.mat', 'RQ_20120903_SCdist.csv']), ('AC_20120917', ['AC_20120917_aparc_to_DWI_mrtrixlabels.csv', 'AC_20120917_SC.mat', 'AC_20120917_SCdist.csv']), ('DG_20120903', ['DG_20120903_aparc_to_DWI_mrtrixlabels.csv', 'DG_20120903_SC.mat', 'DG_20120903_SCdist.csv']), ('NN_20120831', ['NN_20120831_aparc_to_DWI_mrtrixlabels.csv', 'NN_20120831_SC.mat', 'NN_20120831_SCdist.csv']), ('KI_20121009', ['KI_20121009_aparc_to_DWI_mrtrixlabels.csv', 'KI_20121009_SC.mat', 'KI_20121009_SCdist.csv']), ('RF_20120809', ['RF_20120809_aparc_to_DWI_mrtrixlabels.csv', 'RF_20120809_SC.mat', 'RF_20120809_SCdist.csv']), ('RI_20120815', ['RI_20120815_aparc_to_DWI_mrtrixlabels.csv', 'RI_20120815_SC.mat', 'RI_20120815_SCdist.csv']), ('UB_20120806', ['UB_20120806_aparc_to_DWI_mrtrixlabels.csv', 'UB_20120806_SC.mat', 'UB_20120806_SCdist.csv']), ('CN_20120927', ['CN_20120927_aparc_to_DWI_mrtrixlabels.csv', 'CN_20120927_SC.mat', 'CN_20120927_SCdist.csv']), ('GC_20120803', ['GC_20120803_aparc_to_DWI_mrtrixlabels.csv', 'GC_20120803_SC.mat', 'GC_20120803_SCdist.csv']), ('FE_20111010', ['FE_20111010_aparc_to_DWI_mrtrixlabels.csv', 'FE_20111010_SC.mat', 'FE_20111010_SCdist.csv']), ('RL_20110925', ['RL_20110925_aparc_to_DWI_mrtrixlabels.csv', 'RL_20110925_SC.mat', 'RL_20110925_SCdist.csv']), ('IS_20120809', ['IS_20120809_aparc_to_DWI_mrtrixlabels.csv', 'IS_20120809_SC.mat', 'IS_20120809_SCdist.csv']), ('QR_20111010', ['QR_20111010_aparc_to_DWI_mrtrixlabels.csv', 'QR_20111010_SC.mat', 'QR_20111010_SCdist.csv']), ('IQ_20120904', ['IQ_20120904_aparc_to_DWI_mrtrixlabels.csv', 'IQ_20120904_SC.mat', 'IQ_20120904_SCdist.csv']), ('DJ_20120810', ['DJ_20120810_aparc_to_DWI_mrtrixlabels.csv', 'DJ_20120810_SC.mat', 'DJ_20120810_SCdist.csv']), ('JL_20120927', ['JL_20120927_aparc_to_DWI_mrtrixlabels.csv', 'JL_20120927_SC.mat', 'JL_20120927_SCdist.csv']), ('JH_20121009', ['JH_20121009_aparc_to_DWI_mrtrixlabels.csv', 'JH_20121009_SC.mat', 'JH_20121009_SCdist.csv']), ('OQ_20120925', ['OQ_20120925_aparc_to_DWI_mrtrixlabels.csv', 'OQ_20120925_SC.mat', 'OQ_20120925_SCdist.csv']), ('NI_20120831', ['NI_20120831_aparc_to_DWI_mrtrixlabels.csv', 'NI_20120831_SC.mat', 'NI_20120831_SCdist.csv']), ('UE_20120803', ['UE_20120803_aparc_to_DWI_mrtrixlabels.csv', 'UE_20120803_SC.mat', 'UE_20120803_SCdist.csv']), ('BQ_20120904', ['BQ_20120904_aparc_to_DWI_mrtrixlabels.csv', 'BQ_20120904_SC.mat', 'BQ_20120904_SCdist.csv']), ('OG_20120917', ['OG_20120917_aparc_to_DWI_mrtrixlabels.csv', 'OG_20120917_SC.mat', 'OG_20120917_SCdist.csv']), ('FJ_20120808', ['FJ_20120808_aparc_to_DWI_mrtrixlabels.csv', 'FJ_20120808_SC.mat', 'FJ_20120808_SCdist.csv']), ('RE_20120816', ['RE_20120816_aparc_to_DWI_mrtrixlabels.csv', 'RE_20120816_SC.mat', 'RE_20120816_SCdist.csv']), ('QL_20120814', ['QL_20120814_aparc_to_DWI_mrtrixlabels.csv', 'QL_20120814_SC.mat', 'QL_20120814_SCdist.csv']), ('QL_20110925', ['QL_20110925_aparc_to_DWI_mrtrixlabels.csv', 'QL_20110925_SC.mat', 'QL_20110925_SCdist.csv']), ('AR_20120813', ['AR_20120813_aparc_to_DWI_mrtrixlabels.csv', 'AR_20120813_SC.mat', 'AR_20120813_SCdist.csv']), ('FI_20120727', ['FI_20120727_aparc_to_DWI_mrtrixlabels.csv', 'FI_20120727_SC.mat', 'FI_20120727_SCdist.csv']), ('IC_20120810', ['IC_20120810_aparc_to_DWI_mrtrixlabels.csv', 'IC_20120810_SC.mat', 'IC_20120810_SCdist.csv']), ('HA_20120813', ['HA_20120813_aparc_to_DWI_mrtrixlabels.csv', 'HA_20120813_SC.mat', 'HA_20120813_SCdist.csv']), ('JZ_20120824', ['JZ_20120824_aparc_to_DWI_mrtrixlabels.csv', 'JZ_20120824_SC.mat', 'JZ_20120824_SCdist.csv']), ('FR_20120903', ['FR_20120903_aparc_to_DWI_mrtrixlabels.csv', 'FR_20120903_SC.mat', 'FR_20120903_SCdist.csv']), ('UK_20110924', ['UK_20110924_aparc_to_DWI_mrtrixlabels.csv', 'UK_20110924_SC.mat', 'UK_20110924_SCdist.csv']), ('JH_20120925', ['JH_20120925_aparc_to_DWI_mrtrixlabels.csv', 'JH_20120925_SC.mat', 'JH_20120925_SCdist.csv']), ('NN_20120824', ['NN_20120824_aparc_to_DWI_mrtrixlabels.csv', 'NN_20120824_SC.mat', 'NN_20120824_SCdist.csv']), ('AA_20120815', ['AA_20120815_aparc_to_DWI_mrtrixlabels.csv', 'AA_20120815_SC.mat', 'AA_20120815_SCdist.csv']), ('DH_20120806', ['DH_20120806_aparc_to_DWI_mrtrixlabels.csv', 'DH_20120806_SC.mat', 'DH_20120806_SCdist.csv']), ('XB_20120831', ['XB_20120831_aparc_to_DWI_mrtrixlabels.csv', 'XB_20120831_SC.mat', 'XB_20120831_SCdist.csv']), ('AY_20111004', ['AY_20111004_aparc_to_DWI_mrtrixlabels.csv', 'AY_20111004_SC.mat', 'AY_20111004_SCdist.csv', 'AY_20111004_SCstrength_sift2.csv']), ('RQ_20120917', ['RQ_20120917_aparc_to_DWI_mrtrixlabels.csv', 'RQ_20120917_SC.mat', 'RQ_20120917_SCdist.csv']), ('AD_20120813', ['AD_20120813_aparc_to_DWI_mrtrixlabels.csv', 'AD_20120813_SC.mat', 'AD_20120813_SCdist.csv']), ('RS_20120723', ['RS_20120723_aparc_to_DWI_mrtrixlabels.csv', 'RS_20120723_SC.mat', 'RS_20120723_SCdist.csv']), ('RT_20110925', ['RT_20110925_aparc_to_DWI_mrtrixlabels.csv', 'RT_20110925_SC.mat', 'RT_20110925_SCdist.csv']), ('SE_20110924', ['SE_20110924_aparc_to_DWI_mrtrixlabels.csv', 'SE_20110924_SC.mat', 'SE_20110924_SCdist.csv']), ('YE_20120910', ['YE_20120910_aparc_to_DWI_mrtrixlabels.csv', 'YE_20120910_SC.mat', 'YE_20120910_SCdist.csv'])])
subs = OrderedDict()

for k, v in files.items():
    sid = prep.create_uuid()
    
    if sid not in subs.keys():
        subs[sid] = {}
    
    subs[sid].update(prepare_subs([os.path.join(path, x) for x in v], sid))
    
subs
    

OrderedDict([(57,
              {'UK_20111004_aparc_to_DWI_mrtrixlabels.csv': {'name': 'UK_20111004_aparc_to_DWI_mrtrixlabels',
                'fname': 'UK_20111004_aparc_to_DWI_mrtrixlabels.csv',
                'sid': 57,
                'desc': 'default',
                'sep': '\\s',
                'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\UK_20111004_aparc_to_DWI_mrtrixlabels.csv',
                'ext': 'csv'},
               'UK_20111004_SC.mat': {'name': 'UK_20111004_SC',
                'fname': 'UK_20111004_SC.mat',
                'sid': 57,
                'desc': 'default',
                'sep': None,
                'path': 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\UK_20111004_SC.mat',
                'ext': 'mat'},
               'UK_20111004_SCdist.csv': {'name': 'UK_20111004_SCdist',
                'fname': 'UK_20111004_SCdist.csv',
                'sid': 57,
                'desc': 'default',
                'sep

In [6]:
# f = Files('C:\\Users\\dinar\\Desktop\\gsoc_data\\timeseries', os.listdir('C:\\Users\\dinar\\Desktop\\gsoc_data\\timeseries'))
# f.subs

In [7]:
# f = Files('C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor', os.listdir('C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor'))
# f.subs

In [8]:
# f = Files('C:\\Users\\dinar\\Desktop\\gsoc_data', ['timeseries'])
# f.subs

In [42]:
conv.get_content('C:\\Users\\dinar\\Desktop\\gsoc_data\\timeseries', '5')

['C:\\Users\\dinar\\Desktop\\gsoc_data\\timeseries\\5\\timeseries_all.mat']

In [36]:
os.listdir('C:\\Users\\dinar\\Desktop\\gsoc_data\\timeseries\\1')

['timeseries_all.mat']

In [8]:
os.path.exists('C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\distances_preop.txt')

False

In [8]:
import os
import zipfile
import incf.preprocess.subjects as subj
import incf.convert.convert as conv

def extract_zip(path):
    # get folder name
    basename = subj.get_filename(path)

    parent = path.replace(basename, '')

    # get the suffix (ses-preop or ses-postop)
    suffix = get_suffix(path)
    
    # 
    contents = os.listdir(parent)
    
    if [f'weights_{suffix}.txt', f'distances_{suffix}.txt', f'centres_{suffix}.txt'] in contents:
        return
    
    # rename existing files and skip zip file
    for file in contents:
        if file.endswith('.zip'): continue

        basename = os.path.basename(file)

        if suffix not in basename and not basename.endswith('txt'):
            new_file = os.path.join(parent, file.replace('.', f'_{suffix}.'))
            os.replace(os.path.join(parent, file), new_file)
    
    if f'weights_{suffix}.txt' not in contents and f'distances_{suffix}.txt' not in contents and f'centres_{suffix}.txt' not in contents:
        # open zip file
        archive = zipfile.ZipFile(path)

        # store all newly added files
        added = []

        # iterate over zip content and extract only the files that are included in TO_EXRACT
        for ext in conv.TO_EXTRACT:
            if ext in archive.namelist():
                # add the suffix to the newly extracted file
                new_filename = os.path.join(parent, add_suffix(ext, suffix))
                
                if not os.path.exists(os.path.join(parent, ext)) and not os.path.exists(new_filename):
                    print('old path:', os.path.join(parent, ext))
                    
                    # extract new file
                    archive.extract(ext, path=parent)

                    # rename tract_lengths.txt to distances.txt
                    if ext.startswith('tract_lengths'):
                        print('ext to be replaced:', ext)
                        new_filename = new_filename.replace('tract_lengths', 'distances')
                        print('new path for replaced:', new_filename)

                    # rename the new file
                    os.replace(os.path.join(parent, ext), new_filename)

                    # remove tract_lengths: get the path
                    tract_path = os.path.join(parent, f'tract_lengths_{suffix}.txt')

                    # remove tract_lengths
                    if os.path.exists(tract_path):
                        os.remove(tract_path)

                    # append newly added files
                    added.append(new_filename)

        return added

def verify_zip_rename(files):
    match = ['weights', 'distances', 'centres']

    for idx, file in enumerate(files):
        basename = os.path.basename(file)
        base_split = basename.split('.')[0]

        if ('preop' not in basename or 'postop' not in basename) and base_split in match:
            suffix = get_suffix(file)
            files[idx] = add_suffix(file, suffix)

    return list(set(files))


def get_suffix(path):
    return os.path.dirname(path).split('\\')[-1].split('-')[-1]


def add_suffix(filename, suffix):
    return filename.replace('.', f'_{suffix}.')

In [10]:
extract_zip('C:\\Users\\dinar\\Desktop\\test\\sub-CON01\\ses-preop\\SC.zip')

old path: C:\Users\dinar\Desktop\test\sub-CON01\ses-preop\tract_lengths.txt
ext to be replaced: tract_lengths.txt
new path for replaced: C:\Users\dinar\Desktop\test\sub-CON01\ses-preop\distances_preop.txt
old path: C:\Users\dinar\Desktop\test\sub-CON01\ses-preop\weights.txt
old path: C:\Users\dinar\Desktop\test\sub-CON01\ses-preop\centres.txt


['C:\\Users\\dinar\\Desktop\\test\\sub-CON01\\ses-preop\\distances_preop.txt',
 'C:\\Users\\dinar\\Desktop\\test\\sub-CON01\\ses-preop\\weights_preop.txt',
 'C:\\Users\\dinar\\Desktop\\test\\sub-CON01\\ses-preop\\centres_preop.txt']

In [15]:
'preop' in 'FC_preop.mat'

True

In [6]:
Files('C:\\Users\\dinar\\Desktop\\gsoc_data', ['New_Connectomes_MRTrix3'])

self.content: ['C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AA_20120815_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AA_20120815_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AA_20120815_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AC_20120917_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AC_20120917_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AC_20120917_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AD_20120813_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AD_20120813_SC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AD_20120813_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\New_Connectomes_MRTrix3\\AR_20120813_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\gso

In [39]:
import incf.preprocess.subjects as subj
import zipfile
import time


TO_EXTRACT = ['tract_lengths.txt', 'weights.txt', 'centres.txt']

"""
'tract_lengths_preop.txt', 'weights_preop.txt', 'centres_preop.txt',
              'tract_lengths_postop.txt', 'weights_postop.txt', 'centres_postop.txt',
              'distances.txt', 'distances_preop.txt', 'distances_postop.txt'
"""


def recursive_walk(path, basename=False):
    content = []

    for root, _, files in os.walk(path):
        for file in files:
            if file.endswith('.zip'):
                content += extract_zip(os.path.join(root, file))
                continue
            
            if basename:
                content.append(file)
            else:
                content.append(os.path.join(root, file))
                
    return content
                
def extract_zip(path):    
    # get folder name
    basename = subj.get_filename(path)
    
    parent = path.replace(basename, '')
    
    # get the suffix (ses-preop or ses-postop)
    suffix = get_suffix(path)

    # open zip file
    archive = zipfile.ZipFile(path)
    
    # store all newly added files
    added = []

    # iterate over zip content and extract only the files that are included in TO_EXRACT
    for ext in TO_EXTRACT:
        if ext in archive.namelist():
            # add the suffix to the newly extracted file
            new_filename = os.path.join(parent, add_suffix(ext, suffix))
            
            # rename tract_lengths.txt to distances.txt
            if 'tract_lengths' in new_filename:
                new_filename = new_filename.replace('tract_lengths', 'distances')
            
            if not os.path.exists(new_filename):
                # extract new file
                archive.extract(ext, path=parent)
                
                # rename the new file
                os.replace(os.path.join(parent, ext), new_filename)
                
                # remove tract_lengths: get the path
                tract_path = os.path.join(parent, f'tract_lengths_{suffix}.txt')
                
                # remove tract_lengths
                if os.path.exists(tract_path):
                    os.remove(tract_path)

                # append newly added files
                added.append(new_filename)
    
    return added


def verify_zip_rename(files):
    match = ['weights', 'distances', 'centres'] 
    
    for idx, file in enumerate(files):
        basename = os.path.basename(file)
        base_split = basename.split('.')[0]
        
        if ('preop' not in basename or 'postop' not in basename) and base_split in match:            
            suffix = get_suffix(file)
            files[idx] = add_suffix(file, suffix)
            
    return list(set(files))
            
            
def get_suffix(path):
    return os.path.dirname(path).split('\\')[-1].split('-')[-1]
            
    
def add_suffix(filename, suffix):
    return filename.replace('.', f'_{suffix}.')

In [37]:
recursive_walk('C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor')

['C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\centres_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\FC.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\HRF.csv',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\HRF.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\ROIts.dat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\distances_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\SCthrAn.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\tract_lengths_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\weights_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\centres_postop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop

In [42]:
contents = ['C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\centres_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\FC.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\HRF.csv',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\HRF.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\ROIts.dat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\distances_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\SCthrAn.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\tract_lengths_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\weights_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\centres_postop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\FC.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\HRF.csv',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\HRF.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\ROIts.dat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\distances_postop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\SCthrAn.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\tract_lengths_postop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\weights_postop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\centres_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\FC.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\HRF.csv',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\HRF.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\ROIts.dat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\distances_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\SCthrAn.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\tract_lengths_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\weights_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-postop\\centres_postop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-postop\\FC.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-postop\\HRF.csv',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-postop\\HRF.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-postop\\ROIts.dat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-postop\\distances_postop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-postop\\SCthrAn.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-postop\\tract_lengths_postop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-postop\\weights_postop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-preop\\centres_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-preop\\FC.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-preop\\HRF.csv',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-preop\\HRF.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-preop\\ROIts.dat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-preop\\distances_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-preop\\SCthrAn.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-preop\\tract_lengths_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-preop\\weights_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-postop\\centres_postop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-postop\\FC.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-postop\\HRF.csv',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-postop\\HRF.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-postop\\ROIts.dat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-postop\\distances_postop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-postop\\SCthrAn.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-postop\\tract_lengths_postop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-postop\\weights_postop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\areas.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\average_orientations.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\centres.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\cortical.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\FC.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\hemisphere.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\HRF.csv',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\HRF.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\ROIts.dat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\distances_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\weights_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\centres_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\SCthrAn.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\tract_lengths.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\weights.txt']

verify_zip_rename(contents)

['C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-preop\\tract_lengths_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-postop\\weights_postop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-postop\\FC.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-preop\\centres_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\weights_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\HRF.csv',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-postop\\FC.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\SCthrAn.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\centres_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\tract_lengths.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\se

In [43]:
contents

['C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\centres_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\FC.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\HRF.csv',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\HRF.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\ROIts.dat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\distances_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\SCthrAn.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\tract_lengths_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\weights_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\centres_postop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop

In [18]:
del contents[-1]

In [19]:
contents

['C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\centres_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\FC.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\HRF.csv',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\HRF.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\ROIts.dat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\distances_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\SCthrAn.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\tract_lengths_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\weights_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\centres_postop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop

In [71]:
recursive_walk('C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries')

['C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\1\\timeseries_all.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\2\\timeseries_all.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\3\\timeseries_all.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\4\\timeseries_all.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\5\\timeseries_all.mat']


In [26]:
print()

In [27]:
zipfile.namelist('')

AttributeError: module 'zipfile' has no attribute 'namelist'

#### 

In [56]:
import glob
import json
import os
import sys
from pathlib import Path

import pandas as pd
import panel as pn

import incf.preprocess.preprocess as prep
import incf.preprocess.simulations_h5 as h5
import incf.preprocess.simulations_matlab as mat
import incf.preprocess.structure as struct
import incf.preprocess.subjects as subj
import incf.preprocess.weights_distances as wdc
import incf.templates.templates as temp
import incf.utils as utils

sys.path.append('..')
SID = None
DURATION = 3000
TRAVERSE_FOLDERS = True
import zipfile

OUTPUT = '../output'
DESC = 'default'
CENTERS = False
MULTI_INPUT = False
ZIP_CONTENT = None


def check_compatibility(files):
    return len(set(files)) == len(files)


def traverse_files(path: str, files=None, basename: bool = False, recursive=False) -> list:
    """
    Recursively traverse a specified folder and sub-folders. If `basename` is enabled,
    save only the file names. Otherwise, save absolute paths.
    :param recursive:
    :param files:
    :param path: str
        Path to the folder location to traverse.
    :param basename: bool
        Whether to save values by their absolute paths (False) or basename (True).
    :return: list
        Returns a list of basename or absolute paths.
    """

    contents = []

    for file in files:
        if os.path.isdir(os.path.join(path, file)):
            contents += recursive_walk(path, basename)
        else:
            contents.append(file)

    return contents


def recursive_walk(path: str, basename: bool) -> list:
    global ZIP_CONTENT

    contents, zip_idx = [], None
    to_extract = ['tract_lengths.txt', 'weights.txt', 'centres.txt',
                  'tract_lengths_preop.txt', 'weights_preop.txt', 'centres_preop.txt',
                  'tract_lengths_postop.txt', 'weights_postop.txt', 'centres_postop.txt',
                  'distances.txt', 'distances_preop.txt', 'distances_postop.txt']

    # present =
    # zip_present = False

    for root, _, files in os.walk(path, topdown=True):
        exists = False
        for file in files:
            if file in to_extract:
                exists = True

            if not file.endswith('.zip'):
                if basename:
                    contents.append(subj.get_filename(file))
                else:
                    contents.append(os.path.join(root, file))
            else:
                if file.endswith('.zip'):
                    print(file)
                    if not exists:
                        if ZIP_CONTENT is None:
                            ZIP_CONTENT = get_zip_content(os.path.join(root, file))
                        extract_files(os.path.join(root, file))
                        contents += glob.glob(os.path.join(root, '*txt'))

    return contents


def get_zip_content(path):
    # specifying the zip file name
    file_name = path

    # opening the zip file in READ mode
    with zipfile.ZipFile(file_name, 'r') as file:
        return [x for x in file.namelist() if x != '.DS_Store']


def extract_files(path):
    basename = subj.get_filename(path)
    parent = path.replace(basename, '')
    suffix = os.path.dirname(path).split('\\')[-1].split('-')[-1]

    to_extract = ['tract_lengths.txt', 'weights.txt', 'centres.txt']
    archive = zipfile.ZipFile(path)

    for ext in to_extract:
        if ext in archive.namelist():
            archive.extract(ext, path=parent)
            os.rename(os.path.join(parent, ext),
                      os.path.join(parent, ext.replace('.', f'_{suffix}.')))


def verify_zip_files(paths):
    for path in paths:
        basename = subj.get_filename(path)
        suffix = os.path.dirname(path).split('\\')[-1].split('-')[-1]

        if basename.startswith('centres') and basename != f'centres_{suffix}.txt':
            os.rename(path, path.replace(basename, f'centres_{suffix}.txt'))
        elif basename.startswith('weights') and basename != f'weights_{suffix}.txt':
            os.rename(path, path.replace(basename, f'weights_{suffix}.txt'))
        elif basename.startswith('tract_lengths') and basename != f'tract_lengths_{suffix}.txt':
            os.rename(path, path.replace(basename, f'distances_{suffix}.txt'))
        elif basename.startswith('distances') and basename != f'distances_{suffix}.txt':
            os.rename(path, path.replace(basename, f'distances_{suffix}.txt'))


def check_file(path, files, subs=None, save=False):
    global ZIP_CONTENT

    if subs is None:
        subs = Files(path, files).subs

    if save:
        save_output(subs, OUTPUT)

        # remove zip folder contents
        if isinstance(ZIP_CONTENT, list):
            for content in get_content(path, files):
                file = content.split('\\')[-1].split('.')[0]
                file = file.replace('_preop', '') if 'preop' in file else file.replace('_postop', '') if 'postop' in file else file

                if file in ZIP_CONTENT:
                    os.remove(content)

                ZIP_CONTENT = None

    return subs, struct.create_layout(subs, OUTPUT)



def get_content(path, files, basename=False):
    print(files)
    
    # traversal specific to ses-preop and ses-postop
    if isinstance(files, str):
        return traverse_files(os.path.join(path, files), basename=basename, recursive=True)

    all_files = []


#     for file in files:
#         print(os.path.join(path, file))
#         if os.path.isdir(os.path.join(path, file)):
#             all_files += traverse_files(os.path.join(path, file), basename=basename, recursive=True)
#         else:
#             all_files.append(os.path.join(path, file))
#     return all_files


In [48]:
traverse_files(os.path.join(path, 'New_Connectomes_MRTrix3'), basename=True, recursive=True)

['AA_20120815_aparc_to_DWI_mrtrixlabels.csv',
 'AA_20120815_SC.mat',
 'AA_20120815_SCdist.csv',
 'AC_20120917_aparc_to_DWI_mrtrixlabels.csv',
 'AC_20120917_SC.mat',
 'AC_20120917_SCdist.csv',
 'AD_20120813_aparc_to_DWI_mrtrixlabels.csv',
 'AD_20120813_SC.mat',
 'AD_20120813_SCdist.csv',
 'AR_20120813_aparc_to_DWI_mrtrixlabels.csv',
 'AR_20120813_SC.mat',
 'AR_20120813_SCdist.csv',
 'AY_20111004_aparc_to_DWI_mrtrixlabels.csv',
 'AY_20111004_SC.mat',
 'AY_20111004_SCdist.csv',
 'AY_20111004_SCstrength_sift2.csv',
 'BQ_20120904_aparc_to_DWI_mrtrixlabels.csv',
 'BQ_20120904_SC.mat',
 'BQ_20120904_SCdist.csv',
 'CN_20120927_aparc_to_DWI_mrtrixlabels.csv',
 'CN_20120927_SC.mat',
 'CN_20120927_SCdist.csv',
 'convert_connectomes_2_SC.m',
 'DG_20120903_aparc_to_DWI_mrtrixlabels.csv',
 'DG_20120903_SC.mat',
 'DG_20120903_SCdist.csv',
 'DH_20120806_aparc_to_DWI_mrtrixlabels.csv',
 'DH_20120806_SC.mat',
 'DH_20120806_SCdist.csv',
 'DJ_20120810_aparc_to_DWI_mrtrixlabels.csv',
 'DJ_20120810_SC.mat',

In [ ]:
def check_files(path, files, subs=None, save=False):
    if subs is None:
        subs = Files(path, files).subs
        

In [40]:
check_file(path, 'New_Connectomes_MRTrix3')

New_Connectomes_MRTrix3
self.content: ['C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AA_20120815_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AA_20120815_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AA_20120815_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AC_20120917_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AC_20120917_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AC_20120917_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AD_20120813_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AD_20120813_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AD_20120813_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AR_20120813_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AR_20120813_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes

(OrderedDict(),
 '|___ output/<br>&emsp;&emsp;|___code/<br>&emsp;&emsp;|___coord/<br>&emsp;&emsp;|___eq/<br>&emsp;&emsp;|___param/<br>&emsp;&emsp;|___subjects/<br>|___CHANGES.txt<br>|___dataset_description.json<br>|___participants.tsv<br>|___README.txt<br>')

In [43]:
contents = ['C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AA_20120815_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AA_20120815_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AA_20120815_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AC_20120917_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AC_20120917_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AC_20120917_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AD_20120813_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AD_20120813_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AD_20120813_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AR_20120813_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AR_20120813_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AR_20120813_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AY_20111004_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AY_20111004_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AY_20111004_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AY_20111004_SCstrength_sift2.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\BQ_20120904_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\BQ_20120904_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\BQ_20120904_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\CN_20120927_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\CN_20120927_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\CN_20120927_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\convert_connectomes_2_SC.m', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\DG_20120903_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\DG_20120903_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\DG_20120903_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\DH_20120806_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\DH_20120806_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\DH_20120806_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\DJ_20120810_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\DJ_20120810_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\DJ_20120810_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\FE_20111010_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\FE_20111010_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\FE_20111010_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\FI_20120727_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\FI_20120727_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\FI_20120727_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\FJ_20120808_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\FJ_20120808_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\FJ_20120808_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\FR_20120903_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\FR_20120903_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\FR_20120903_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\GC_20120803_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\GC_20120803_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\GC_20120803_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\HA_20120813_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\HA_20120813_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\HA_20120813_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\IC_20120810_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\IC_20120810_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\IC_20120810_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\IQ_20120904_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\IQ_20120904_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\IQ_20120904_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\IS_20120809_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\IS_20120809_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\IS_20120809_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\JH_20120925_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\JH_20120925_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\JH_20120925_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\JH_20121009_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\JH_20121009_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\JH_20121009_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\JL_20120927_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\JL_20120927_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\JL_20120927_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\JS_20120910_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\JS_20120910_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\JS_20120910_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\JZ_20120824_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\JZ_20120824_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\JZ_20120824_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\KI_20121009_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\KI_20121009_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\KI_20121009_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\NI_20120831_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\NI_20120831_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\NI_20120831_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\NN_20120824_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\NN_20120824_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\NN_20120824_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\NN_20120831_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\NN_20120831_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\NN_20120831_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\OG_20120917_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\OG_20120917_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\OG_20120917_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\OK_20121011_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\OK_20121011_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\OK_20121011_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\OQ_20120925_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\OQ_20120925_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\OQ_20120925_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\QL_20110925_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\QL_20110925_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\QL_20110925_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\QL_20120814_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\QL_20120814_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\QL_20120814_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\QR_20111010_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\QR_20111010_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\QR_20111010_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RE_20120816_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RE_20120816_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RE_20120816_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RF_20120809_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RF_20120809_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RF_20120809_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RI_20110924_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RI_20110924_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RI_20110924_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RI_20120815_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RI_20120815_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RI_20120815_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RL_20110925_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RL_20110925_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RL_20110925_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RQ_20120903_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RQ_20120903_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RQ_20120903_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RQ_20120917_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RQ_20120917_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RQ_20120917_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RS_20120723_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RS_20120723_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RS_20120723_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RT_20110925_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RT_20110925_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RT_20110925_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\SE_20110924_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\SE_20110924_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\SE_20110924_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\UB_20120806_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\UB_20120806_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\UB_20120806_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\UE_20120803_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\UE_20120803_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\UE_20120803_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\UK_20110924_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\UK_20110924_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\UK_20110924_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\UK_20111004_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\UK_20111004_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\UK_20111004_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\XB_20120831_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\XB_20120831_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\XB_20120831_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\YE_20120910_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\YE_20120910_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\YE_20120910_SCdist.csv']

base_dir = os.path.basename(os.path.dirname(contents[0]))

for content in contents:
    if base_dir != os.path.basename(os.path.dirname(content)):
        print(content)

In [ ]:
def multi_folders(files):
    base_dir = os.path.basename(os.path.dirname(files[0]))

    for file in files:
        if base_dir != os.path.basename(os.path.dirname(file)):
            return False
    return True

In [60]:
import os
import csv
import re

import pandas as pd
import panel as pn

from incf.convert import convert
import os
from collections import OrderedDict
import incf.preprocess.preprocess as prep
import incf.convert.convert as conv


class Files:
    def __init__(self, path, files):
        self.path = path
        self.files = files
        self.subs = OrderedDict()

        # decide whether input files are for one or more patients
        self.content = get_content(path, files)
        print('self.content:', self.content)
        
#         print('self.content:', self.content, end='\n\n')
# #         self.basename = set(conv.traverse_files(path, files, basename=True))
#         self.basename = set(traverse_files(path, files, basename=True))
#         self.single = len(self.content) == len(self.basename)
#         print('self.single:', self.single)

#         # set multi-subject input to true
#         conv.MULTI_INPUT = False if self.single else True
#         self.traverse_files()

    def traverse_files(self):
        # folder structure inputs
        if conv.MULTI_INPUT:
            if multi_folders(self.files):
                print(prepare_subs(self.content, sid))
            
#             for sel in self.files:
#                 sid = prep.create_uuid() if re.findall('[0-9]+', sel) == 0 else sel.replace('sub-', '')

#                 if sel not in self.subs.keys():
#                     self.subs[sel] = {}

#                 if 'ses-preop' in os.listdir(os.path.join(self.path, sel)):
# #                     content = conv.get_content(os.path.join(self.path), sel)
#                     content = get_content(os.path.join(self.path), sel)
#                     conv.verify_zip_files(content)
#                     self.subs[sel].update(
#                         prepare_subs(conv.get_content(os.path.join(self.path), sel), sid, suffix='preop'))
#                 elif 'ses-postop' in os.listdir(os.path.join(self.path, sel)):
#                     content = conv.get_content(os.path.join(self.path), sel)
#                     conv.verify_zip_files(content)

#                     self.subs[sel].update(
#                         prepare_subs(conv.get_content(os.path.join(self.path), sel), sid, suffix='postop'))
#                 else:
#                     print(sel)
#                     self.subs[sel] = prepare_subs(self.content, sel)

#         else:
#             sid = prep.create_uuid()
#             self.subs[sid] = prepare_subs(self.content, sid)


def multi_folders(files):
    base_dir = os.path.basename(os.path.dirname(files[0]))

    for file in files:
        if base_dir != os.path.basename(os.path.dirname(file)):
            return False
    return True


def find_str(path, word, before=True):
    split = path.find(word) + len(word) + 1

    if before:
        return path[:split]
    return path[split:]


def prepare_subs(file_paths, sid, suffix=None):
    subs = {}
    suffix = '' if suffix is None else '_' + suffix
    accepted = ['tract_lengths.txt', 'weights.txt', 'centres.txt',
                'tract_lengths_preop.txt', 'weights_preop.txt', 'centres_preop.txt',
                'tract_lengths_postop.txt', 'weights_postop.txt', 'centres_postop.txt',
                'distances.txt', 'distances_preop.txt', 'distances_postop.txt']

    for file_path in file_paths:

        if file_path.endswith('txt') and get_filename(file_path) not in accepted:
            continue
        name = get_filename(file_path)
        desc = convert.DESC + 'h5' if file_path.endswith('h5') else convert.DESC

        if 'preop' in name or 'postop' in name:
            nsuffix = name.split('.')[0]
        else:
            nsuffix = name.split('.')[0] + suffix

        n = nsuffix + '.' + name.split('.')[1]

        subs[n] = {
            'name': nsuffix,
            'fname': name,
            'sid': sid,
            'desc': desc,
            'sep': find_separator(file_path),
            'path': file_path,
            'ext': get_file_ext(file_path),
        }

        if subs[n]['name'] in ['tract_lengths', 'tract_length',
                               'tract_lengths_preop', 'tract_length_preop',
                               'tract_lengths_postop', 'tract_length_postop']:
            subs[n]['name'] = f'distances_preop' if 'preop' in name else 'distances_postop' \
                if 'postop' in name else 'distances'
        if subs[n]['name'] in ['centres', 'centers', 'centres_preop', 'centres_postop']:
            conv.CENTERS = True
    return subs


def get_filename(path):
    return os.path.basename(path)


def get_file_ext(path):
    return path.split('.')[-1]


def find_separator(path):
    """
    Find the separator/delimiter used in the file to ensure no exception
    is raised while reading files.
    :param path:
    :return:
    """
    if path.split('.')[-1] not in ['txt', 'csv']:
        return

    try:
        f = pd.read_csv(path)
    except pd.errors.EmptyDataError:
        pn.state.notifications.error(f'File {os.path.basename(path)} is empty! Creating an empty file...')
        return ''

    sniffer = csv.Sniffer()

    with open(path) as fp:
        try:
            delimiter = sniffer.sniff(fp.read(5000)).delimiter
        except Exception:
            delimiter = sniffer.sniff(fp.read(100)).delimiter
        except Exception:
            delimiter = sniffer.sniff(fp.read(500)).delimiter

    delimiter = '\s' if delimiter == ' ' else delimiter
    return delimiter


In [61]:
path = 'C:\\Users\\dinar\\Desktop'

f = Files(path, ['New_Connectomes_MRTrix3'])

['New_Connectomes_MRTrix3']
['New_Connectomes_MRTrix3']
self.content: None


In [44]:
import os
import csv
import re

import pandas as pd
import panel as pn

from incf.convert import convert
import os
from collections import OrderedDict
import incf.preprocess.preprocess as prep
import incf.convert.convert as conv


class Files:
    def __init__(self, path, files):
        self.path = path
        self.files = files
        self.subs = OrderedDict()

        # decide whether input files are for one or more patients
        self.content = conv.get_content(path, files)
        print('self.content:', self.content, end='\n\n')
        self.basename = set(conv.traverse_files(path, files, basename=True))
        self.single = len(self.content) == len(self.basename)
        print('self.single:', self.single)

        # set multi-subject input to true
        conv.MULTI_INPUT = False if self.single else True
#         self.traverse_files()

#     def traverse_files(self):
#         # folder structure inputs
#         if conv.MULTI_INPUT:
#             for sel in self.files:
#                 sid = prep.create_uuid() if re.findall('[0-9]+', sel) == 0 else sel.replace('sub-', '')

#                 if sel not in self.subs.keys():
#                     self.subs[sel] = {}

#                 if 'ses-preop' in os.listdir(os.path.join(self.path, sel)):
#                     self.subs[sel].update(
#                         prepare_subs(conv.get_content(os.path.join(self.path), sel), sid, suffix='preop'))
#                 elif 'ses-postop' in os.listdir(os.path.join(self.path, sel)):
#                     self.subs[sel].update(
#                         prepare_subs(conv.get_content(os.path.join(self.path), sel), sid, suffix='postop'))
#                 else:
#                     self.subs[sel] = prepare_subs(conv.get_content(self.path, sel), sel)

#         else:
#             sid = prep.create_uuid()
#             self.subs[sid] = prepare_subs(self.content, sid)


# def find_str(path, word, before=True):
#     split = path.find(word) + len(word) + 1

#     if before:
#         return path[:split]
#     return path[split:]


# def prepare_subs(file_paths, sid, suffix=None):
#     subs = {}
#     suffix = '' if suffix is None else '_' + suffix
#     accepted = ['tract_lengths.txt', 'weights.txt', 'centres.txt',
#                 'tract_lengths_preop.txt', 'weights_preop.txt', 'centres_preop.txt',
#                 'tract_lengths_postop.txt', 'weights_postop.txt', 'centres_postop.txt',
#                 'distances.txt', 'distances_preop.txt', 'distances_postop.txt']

#     for file_path in file_paths:

#         if file_path.endswith('txt') and get_filename(file_path) not in accepted:
#             continue
#         name = get_filename(file_path)
#         desc = convert.DESC + 'h5' if file_path.endswith('h5') else convert.DESC

#         if 'preop' in name or 'postop' in name:
#             nsuffix = name.split('.')[0]
#         else:
#             nsuffix = name.split('.')[0] + suffix

#         n = nsuffix + '.' + name.split('.')[1]

#         subs[n] = {
#             'name': nsuffix,
#             'fname': name,
#             'sid': sid,
#             'desc': desc,
#             'sep': find_separator(file_path),
#             'path': file_path,
#             'ext': get_file_ext(file_path),
#         }

#         if subs[n]['name'] in ['tract_lengths', 'tract_length',
#                                'tract_lengths_preop', 'tract_length_preop',
#                                'tract_lengths_postop', 'tract_length_postop']:
#             subs[n]['name'] = f'distances_preop' if 'preop' in name else 'distances_postop' \
#                 if 'postop' in name else 'distances'
#         if subs[n]['name'] in ['centres', 'centers', 'centres_preop', 'centres_postop']:
#             conv.CENTERS = True
#     return subs


# def get_filename(path):
#     return os.path.basename(path)


# def get_file_ext(path):
#     return path.split('.')[-1]


# def find_separator(path):
#     """
#     Find the separator/delimiter used in the file to ensure no exception
#     is raised while reading files.
#     :param path:
#     :return:
#     """
#     if path.split('.')[-1] not in ['txt', 'csv']:
#         return

#     try:
#         f = pd.read_csv(path)
#     except pd.errors.EmptyDataError:
#         pn.state.notifications.error(f'File {os.path.basename(path)} is empty! Creating an empty file...')
#         return ''

#     sniffer = csv.Sniffer()

#     with open(path) as fp:
#         try:
#             delimiter = sniffer.sniff(fp.read(5000)).delimiter
#         except Exception:
#             delimiter = sniffer.sniff(fp.read(100)).delimiter
#         except Exception:
#             delimiter = sniffer.sniff(fp.read(500)).delimiter

#     delimiter = '\s' if delimiter == ' ' else delimiter
#     return delimiter


files = Files('C:\\Users\\dinar\\Desktop\\gsoc_data', ['brain_tumor'])

In [45]:
# get_content('C:\\Users\\dinar\\Desktop\\gsoc_data', ['brain_tumor']
files = Files('C:\\Users\\dinar\\Desktop\\gsoc_data', ['brain_tumor'])

SC.zip
SC.zip
SC.zip
SC.zip
SC.zip
SC.zip
SC.zip
self.content: ['C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\centres_preop.txt', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\distances_preop.txt', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\FC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\HRF.csv', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\HRF.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\ROIts.dat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\SCthrAn.mat', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON01\\ses-preop\\weights_preop.txt', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON02\\ses-postop\\centres_postop.txt', 'C:\\Users\\dinar\\Desktop\\gsoc_data\\brain_tumor\\sub-CON02\\ses-postop\\distances_postop.txt', 'C:\\Users\\dinar\\Desktop\\gsoc_

In [6]:
paths = ['C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AA_20120815_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AA_20120815_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AA_20120815_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AC_20120917_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AC_20120917_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AC_20120917_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AD_20120813_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AD_20120813_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AD_20120813_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AR_20120813_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AR_20120813_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AR_20120813_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AY_20111004_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AY_20111004_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AY_20111004_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AY_20111004_SCstrength_sift2.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\BQ_20120904_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\BQ_20120904_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\BQ_20120904_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\CN_20120927_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\CN_20120927_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\CN_20120927_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\convert_connectomes_2_SC.m', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\DG_20120903_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\DG_20120903_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\DG_20120903_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\DH_20120806_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\DH_20120806_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\DH_20120806_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\DJ_20120810_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\DJ_20120810_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\DJ_20120810_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\FE_20111010_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\FE_20111010_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\FE_20111010_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\FI_20120727_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\FI_20120727_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\FI_20120727_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\FJ_20120808_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\FJ_20120808_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\FJ_20120808_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\FR_20120903_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\FR_20120903_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\FR_20120903_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\GC_20120803_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\GC_20120803_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\GC_20120803_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\HA_20120813_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\HA_20120813_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\HA_20120813_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\IC_20120810_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\IC_20120810_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\IC_20120810_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\IQ_20120904_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\IQ_20120904_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\IQ_20120904_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\IS_20120809_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\IS_20120809_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\IS_20120809_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\JH_20120925_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\JH_20120925_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\JH_20120925_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\JH_20121009_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\JH_20121009_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\JH_20121009_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\JL_20120927_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\JL_20120927_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\JL_20120927_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\JS_20120910_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\JS_20120910_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\JS_20120910_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\JZ_20120824_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\JZ_20120824_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\JZ_20120824_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\KI_20121009_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\KI_20121009_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\KI_20121009_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\NI_20120831_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\NI_20120831_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\NI_20120831_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\NN_20120824_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\NN_20120824_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\NN_20120824_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\NN_20120831_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\NN_20120831_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\NN_20120831_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\OG_20120917_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\OG_20120917_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\OG_20120917_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\OK_20121011_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\OK_20121011_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\OK_20121011_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\OQ_20120925_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\OQ_20120925_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\OQ_20120925_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\QL_20110925_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\QL_20110925_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\QL_20110925_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\QL_20120814_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\QL_20120814_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\QL_20120814_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\QR_20111010_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\QR_20111010_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\QR_20111010_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RE_20120816_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RE_20120816_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RE_20120816_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RF_20120809_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RF_20120809_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RF_20120809_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RI_20110924_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RI_20110924_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RI_20110924_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RI_20120815_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RI_20120815_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RI_20120815_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RL_20110925_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RL_20110925_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RL_20110925_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RQ_20120903_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RQ_20120903_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RQ_20120903_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RQ_20120917_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RQ_20120917_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RQ_20120917_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RS_20120723_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RS_20120723_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RS_20120723_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RT_20110925_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RT_20110925_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\RT_20110925_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\SE_20110924_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\SE_20110924_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\SE_20110924_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\UB_20120806_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\UB_20120806_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\UB_20120806_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\UE_20120803_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\UE_20120803_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\UE_20120803_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\UK_20110924_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\UK_20110924_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\UK_20110924_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\UK_20111004_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\UK_20111004_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\UK_20111004_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\XB_20120831_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\XB_20120831_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\XB_20120831_SCdist.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\YE_20120910_aparc_to_DWI_mrtrixlabels.csv', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\YE_20120910_SC.mat', 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\YE_20120910_SCdist.csv']

for path in paths:
    if path.endswith('.zip'):
        print(path)

In [15]:
import os
import csv
import re

import pandas as pd
import panel as pn

from incf.convert import convert
import os
from collections import OrderedDict
import incf.preprocess.preprocess as prep
import incf.convert.convert as conv


def prepare_subs(file_paths, sid, suffix=None):
    subs = {}
    suffix = '' if suffix is None else '_' + suffix
    accepted = ['tract_lengths.txt', 'weights.txt', 'centres.txt',
                'tract_lengths_preop.txt', 'weights_preop.txt', 'centres_preop.txt',
                'tract_lengths_postop.txt', 'weights_postop.txt', 'centres_postop.txt',
                'distances.txt', 'distances_preop.txt', 'distances_postop.txt']

    for file_path in file_paths:

        if file_path.endswith('txt') and get_filename(file_path) not in accepted:
            continue
        name = get_filename(file_path)
        desc = convert.DESC + 'h5' if file_path.endswith('h5') else convert.DESC

        if 'preop' in name or 'postop' in name:
            nsuffix = name.split('.')[0]
        else:
            nsuffix = name.split('.')[0] + suffix

        n = nsuffix + '.' + name.split('.')[1]
        print(file_path, end='\n\n')
        subs[n] = {
            'name': nsuffix,
            'fname': name,
            'sid': sid,
            'desc': desc,
            'sep': find_separator(file_path),
            'path': file_path,
            'ext': get_file_ext(file_path),
        }

        if subs[n]['name'] in ['tract_lengths', 'tract_length',
                               'tract_lengths_preop', 'tract_length_preop',
                               'tract_lengths_postop', 'tract_length_postop']:
            subs[n]['name'] = f'distances_preop' if 'preop' in name else 'distances_postop' \
                if 'postop' in name else 'distances'
        if subs[n]['name'] in ['centres', 'centers', 'centres_preop', 'centres_postop']:
            conv.CENTERS = True
    return subs



def get_filename(path):
    return os.path.basename(path)


def get_file_ext(path):
    return path.split('.')[-1]


def find_separator(path):
    """
    Find the separator/delimiter used in the file to ensure no exception
    is raised while reading files.
    :param path:
    :return:
    """
    if path.split('.')[-1] in ['mat', 'zip', 'h5', 'm']:
        return

    try:
        f = pd.read_csv(path)
    except pd.errors.EmptyDataError:
        pn.state.notifications.error(f'File {os.path.basename(path)} is empty! Creating an empty file...')
        return ''

    sniffer = csv.Sniffer()

    with open(path) as fp:
        try:
            delimiter = sniffer.sniff(fp.read(5000)).delimiter
        except Exception:
            delimiter = sniffer.sniff(fp.read(100)).delimiter
        except Exception:
            delimiter = sniffer.sniff(fp.read(500)).delimiter

    delimiter = '\s' if delimiter == ' ' else delimiter
    return delimiter

In [16]:
prepare_subs(paths, '01')

C:\Users\dinar\Desktop\New_Connectomes_MRTrix3\AA_20120815_aparc_to_DWI_mrtrixlabels.csv

C:\Users\dinar\Desktop\New_Connectomes_MRTrix3\AA_20120815_SC.mat

C:\Users\dinar\Desktop\New_Connectomes_MRTrix3\AA_20120815_SCdist.csv

C:\Users\dinar\Desktop\New_Connectomes_MRTrix3\AC_20120917_aparc_to_DWI_mrtrixlabels.csv

C:\Users\dinar\Desktop\New_Connectomes_MRTrix3\AC_20120917_SC.mat

C:\Users\dinar\Desktop\New_Connectomes_MRTrix3\AC_20120917_SCdist.csv

C:\Users\dinar\Desktop\New_Connectomes_MRTrix3\AD_20120813_aparc_to_DWI_mrtrixlabels.csv

C:\Users\dinar\Desktop\New_Connectomes_MRTrix3\AD_20120813_SC.mat

C:\Users\dinar\Desktop\New_Connectomes_MRTrix3\AD_20120813_SCdist.csv

C:\Users\dinar\Desktop\New_Connectomes_MRTrix3\AR_20120813_aparc_to_DWI_mrtrixlabels.csv

C:\Users\dinar\Desktop\New_Connectomes_MRTrix3\AR_20120813_SC.mat

C:\Users\dinar\Desktop\New_Connectomes_MRTrix3\AR_20120813_SCdist.csv

C:\Users\dinar\Desktop\New_Connectomes_MRTrix3\AY_20111004_aparc_to_DWI_mrtrixlabels.csv

C:\Users\dinar\Desktop\New_Connectomes_MRTrix3\RT_20110925_SC.mat

C:\Users\dinar\Desktop\New_Connectomes_MRTrix3\RT_20110925_SCdist.csv

C:\Users\dinar\Desktop\New_Connectomes_MRTrix3\SE_20110924_aparc_to_DWI_mrtrixlabels.csv

C:\Users\dinar\Desktop\New_Connectomes_MRTrix3\SE_20110924_SC.mat

C:\Users\dinar\Desktop\New_Connectomes_MRTrix3\SE_20110924_SCdist.csv

C:\Users\dinar\Desktop\New_Connectomes_MRTrix3\UB_20120806_aparc_to_DWI_mrtrixlabels.csv

C:\Users\dinar\Desktop\New_Connectomes_MRTrix3\UB_20120806_SC.mat

C:\Users\dinar\Desktop\New_Connectomes_MRTrix3\UB_20120806_SCdist.csv

C:\Users\dinar\Desktop\New_Connectomes_MRTrix3\UE_20120803_aparc_to_DWI_mrtrixlabels.csv

C:\Users\dinar\Desktop\New_Connectomes_MRTrix3\UE_20120803_SC.mat

C:\Users\dinar\Desktop\New_Connectomes_MRTrix3\UE_20120803_SCdist.csv

C:\Users\dinar\Desktop\New_Connectomes_MRTrix3\UK_20110924_aparc_to_DWI_mrtrixlabels.csv

C:\Users\dinar\Desktop\New_Connectomes_MRTrix3\UK_20110924_SC.mat

C:\Users\dinar\Deskto

{'AA_20120815_aparc_to_DWI_mrtrixlabels.csv': {'name': 'AA_20120815_aparc_to_DWI_mrtrixlabels',
  'fname': 'AA_20120815_aparc_to_DWI_mrtrixlabels.csv',
  'sid': '01',
  'desc': 'default',
  'sep': '\\s',
  'path': 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AA_20120815_aparc_to_DWI_mrtrixlabels.csv',
  'ext': 'csv'},
 'AA_20120815_SC.mat': {'name': 'AA_20120815_SC',
  'fname': 'AA_20120815_SC.mat',
  'sid': '01',
  'desc': 'default',
  'sep': None,
  'path': 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AA_20120815_SC.mat',
  'ext': 'mat'},
 'AA_20120815_SCdist.csv': {'name': 'AA_20120815_SCdist',
  'fname': 'AA_20120815_SCdist.csv',
  'sid': '01',
  'desc': 'default',
  'sep': '\\s',
  'path': 'C:\\Users\\dinar\\Desktop\\New_Connectomes_MRTrix3\\AA_20120815_SCdist.csv',
  'ext': 'csv'},
 'AC_20120917_aparc_to_DWI_mrtrixlabels.csv': {'name': 'AC_20120917_aparc_to_DWI_mrtrixlabels',
  'fname': 'AC_20120917_aparc_to_DWI_mrtrixlabels.csv',
  'sid': '01',
  'desc': 'default

In [6]:
# importing required modules
from zipfile import ZipFile

def get_zip_content(path):
    # specifying the zip file name
    file_name = path

    # opening the zip file in READ mode
    with ZipFile(file_name, 'r') as zip:
        return [x for x in zip.namelist() if x != '.DS_Store']

In [7]:
contents = get_zip_content("C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\SC.zip")

In [6]:
from incf.convert import convert 

convert.get_content('C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor', ['sub-CON01'])

['C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\centres_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\FC.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\HRF.csv',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\HRF.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\ROIts.dat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\SCthrAn.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\tract_lengths_preop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\weights_preop.txt']

In [8]:
contents = [x.split('.')[0] for x in contents]

for content in ['C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\centres_preop.txt', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\FC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\HRF.csv', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\HRF.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\ROIts.dat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\SCthrAn.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\tract_lengths_preop.txt', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\weights_preop.txt', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\centres_postop.txt', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\FC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\HRF.csv', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\HRF.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\ROIts.dat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\SCthrAn.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\tract_lengths_postop.txt', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\weights_postop.txt', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\centres_preop.txt', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\FC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\HRF.csv', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\HRF.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\ROIts.dat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\SCthrAn.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\tract_lengths_preop.txt', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\weights_preop.txt']:
    file = content.split('\\')[-1].split('.')[0]
    file = file.replace('_preop', '') if 'preop' in file else file.replace('_postop', '') if 'postop' in file else file 
    
    if file in contents:
        os.remove(content)

In [11]:
# iterate over the folder and verify it includes `preop` and `postop` such folders exist
import incf.preprocess.subjects as subj


def verify_zip_files(paths):
    for path in paths:
        basename = subj.get_filename(path)
        parent = path.replace(basename, '')
        suffix = os.path.dirname(path).split('\\')[-1].split('-')[-1]

        if basename.startswith('centres') and basename != f'centres_{suffix}.txt':
            os.rename(path, path.replace(basename, f'centres_{suffix}.txt'))
        elif basename.startswith('weights') and basename != f'weights_{suffix}.txt':
            os.rename(path, path.replace(basename, f'weights_{suffix}.txt'))
        elif basename.startswith('tract_lengths') and basename != f'tract_lengths_{suffix}.txt':
            os.rename(path, path.replace(basename, f'distances_{suffix}.txt'))
        elif basename.startswith('distances') and basename != f'distances_{suffix}.txt':
            os.rename(path, path.replace(basename, f'distances_{suffix}.txt'))

# os.rename(os.path.join(parent, ext),
#                   os.path.join(parent, ext.replace('.', f'_{suffix}.')))

In [14]:
paths = ['C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-postop\\centres_postop.txt', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-postop\\FC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-postop\\HRF.csv', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-postop\\HRF.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-postop\\ROIts.dat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-postop\\SCthrAn.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-postop\\tract_lengths_postop.txt', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-postop\\weights_postop.txt', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\areas.txt', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\average_orientations.txt', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\centres.txt', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\cortical.txt', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\FC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\hemisphere.txt', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\HRF.csv', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\HRF.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\ROIts.dat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\SCthrAn.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\tract_lengths.txt', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\weights.txt']

verify_zip_files(paths)

In [6]:
import glob

glob.glob('C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\*txt')

['C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\centres_postop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\tract_lengths_postop.txt',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\weights_postop.txt']

In [8]:
import zipfile

to_extract = ['tract_lengths.txt', 'weights.txt', 'centres.txt']
archive = zipfile.ZipFile('C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\SC.zip')

for ext in to_extract:
    archive.extract(ext, path='C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop')

In [7]:
os.path.dirname('C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\SC.zip').split('\\')[-1].split('-')[-1]

'postop'

In [8]:
files = os.listdir('C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop')
files

['FC.mat', 'HRF.csv', 'HRF.mat', 'ROIts.dat', 'SC.zip', 'SCthrAn.mat']

ERROR:bokeh.server.protocol_handler:error handling message
 message: Message 'PATCH-DOC' content: {'events': [{'kind': 'MessageSent', 'msg_type': 'bokeh_event', 'msg_data': {'event_name': 'button_click', 'event_values': {}}}], 'references': []} 
 error: FileNotFoundError(2, 'No such file or directory')
Traceback (most recent call last):
  File "C:\Users\dinar\miniconda3\envs\incf\lib\site-packages\bokeh\server\protocol_handler.py", line 97, in handle
    work = await handler(message, connection)
  File "C:\Users\dinar\miniconda3\envs\incf\lib\site-packages\bokeh\server\session.py", line 95, in _needs_document_lock_wrapper
    result = func(self, *args, **kwargs)
  File "C:\Users\dinar\miniconda3\envs\incf\lib\site-packages\bokeh\server\session.py", line 289, in _handle_patch
    message.apply_to_document(self.document, self)
  File "C:\Users\dinar\miniconda3\envs\incf\lib\site-packages\bokeh\protocol\messages\patch_doc.py", line 115, in apply_to_document
    invoke_with_curdoc(doc, lam

['C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\FC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\HRF.csv', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\HRF.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\ROIts.dat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\weights.txt', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\tract_lengths.txt', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\areas.txt', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\average_orientations.txt', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\centres.txt', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\cortical.txt', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\hemisphere.txt', 'C:\\Users\\di

In [7]:
suffix = 'preop'

for file in files:
    if suffix not in file and not file.endswith('zip'):
        os.remove(os.path.join(path, file))

.DS_Store
areas.txt
average_orientations.txt
centres.txt
cortical.txt
hemisphere.txt
tract_lengths.txt
weights.txt


In [10]:
suffix = 'preop'
# files = ['FC.mat', 'HRF.csv', 'HRF.mat', 'ROIts.dat', 'SC.zip', 'SCthrAn.mat', 'weights.txt', 'distances.txt', 'centres.txt']
path = 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop'
files = os.listdir(path)
files

['areas.txt',
 'average_orientations.txt',
 'centres.txt',
 'cortical.txt',
 'FC.mat',
 'hemisphere.txt',
 'HRF.csv',
 'HRF.mat',
 'ROIts.dat',
 'SC.zip',
 'SCthrAn.mat',
 'tract_lengths.txt',
 'weights.txt']

In [14]:
for file in os.listdir(path):
    if file in ['centres.txt', 'tract_lengths.txt', 'weights.txt']:
        os.rename(os.path.join(path, file), os.path.join(path, file.replace('.', f'_{suffix}.')))
    elif file in [f'centres_{suffix}.txt', f'tract_lengths_{suffix}.txt', f'weights_{suffix}.txt']:
        continue 
    elif file.endswith('.txt'):
        os.remove(os.path.join(path, file))

In [6]:
os.rename('C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\tract_lengths.txt', 
          'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\tract_lengths_preop.txt')

In [6]:
pd.read_csv('C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\tract_lengths.txt')

EmptyDataError: No columns to parse from file

In [5]:
s = {'FC_preop.mat': {'name': 'FC_preop', 'fname': 'FC.mat', 'sid': 'CON01', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\FC.mat', 'ext': 'mat'}, 'HRF_preop.csv': {'name': 'HRF_preop', 'fname': 'HRF.csv', 'sid': 'CON01', 'desc': 'default', 'sep': ',', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\HRF.csv', 'ext': 'csv'}, 'HRF_preop.mat': {'name': 'HRF_preop', 'fname': 'HRF.mat', 'sid': 'CON01', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\HRF.mat', 'ext': 'mat'}, 'ROIts_preop.dat': {'name': 'ROIts_preop', 'fname': 'ROIts.dat', 'sid': 'CON01', 'desc': 'default', 'sep': '\\s', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\ROIts.dat', 'ext': 'dat'}, 'SC_preop.zip': {'name': 'SC_preop', 'fname': 'SC.zip', 'sid': 'CON01', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\SC.zip', 'ext': 'zip'}, 'SCthrAn_preop.mat': {'name': 'SCthrAn_preop', 'fname': 'SCthrAn.mat', 'sid': 'CON01', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\SCthrAn.mat', 'ext': 'mat'}
    }
for k, v in s.items():
    print(k, v, end='\n\n\n\n')

FC_preop.mat {'name': 'FC_preop', 'fname': 'FC.mat', 'sid': 'CON01', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\FC.mat', 'ext': 'mat'}



HRF_preop.csv {'name': 'HRF_preop', 'fname': 'HRF.csv', 'sid': 'CON01', 'desc': 'default', 'sep': ',', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\HRF.csv', 'ext': 'csv'}



HRF_preop.mat {'name': 'HRF_preop', 'fname': 'HRF.mat', 'sid': 'CON01', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\HRF.mat', 'ext': 'mat'}



ROIts_preop.dat {'name': 'ROIts_preop', 'fname': 'ROIts.dat', 'sid': 'CON01', 'desc': 'default', 'sep': '\\s', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\ROIts.dat', 'ext': 'dat'}



SC_preop.zip {'name': 'SC_preop', 'fname': 'SC.zip', 'sid': 'CON01', 'desc': 'default', 'sep': None, 'path': 'C:\\Users\\dinar\\Desktop\\gso

In [7]:
f = ['C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\FC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\HRF.csv', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\HRF.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\ROIts.dat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\SC.zip', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\SCthrAn.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\FC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\HRF.csv', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\HRF.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\ROIts.dat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\SC.zip', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\SCthrAn.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\FC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\HRF.csv', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\HRF.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\ROIts.dat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\SC.zip', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\SCthrAn.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-postop\\FC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-postop\\HRF.csv', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-postop\\HRF.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-postop\\ROIts.dat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-postop\\SC.zip', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-postop\\SCthrAn.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-preop\\FC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-preop\\HRF.csv', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-preop\\HRF.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-preop\\ROIts.dat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-preop\\SC.zip', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON03\\ses-preop\\SCthrAn.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-postop\\FC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-postop\\HRF.csv', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-postop\\HRF.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-postop\\ROIts.dat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-postop\\SC.zip', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-postop\\SCthrAn.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\FC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\HRF.csv', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\HRF.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\ROIts.dat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\SC.zip', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON04\\ses-preop\\SCthrAn.mat']

for e in f:
    print(e, end='\n\n\n')

C:\Users\dinar\Desktop\gsoc\data\brain_tumor\sub-CON01\ses-preop\FC.mat


C:\Users\dinar\Desktop\gsoc\data\brain_tumor\sub-CON01\ses-preop\HRF.csv


C:\Users\dinar\Desktop\gsoc\data\brain_tumor\sub-CON01\ses-preop\HRF.mat


C:\Users\dinar\Desktop\gsoc\data\brain_tumor\sub-CON01\ses-preop\ROIts.dat


C:\Users\dinar\Desktop\gsoc\data\brain_tumor\sub-CON01\ses-preop\SC.zip


C:\Users\dinar\Desktop\gsoc\data\brain_tumor\sub-CON01\ses-preop\SCthrAn.mat


C:\Users\dinar\Desktop\gsoc\data\brain_tumor\sub-CON02\ses-postop\FC.mat


C:\Users\dinar\Desktop\gsoc\data\brain_tumor\sub-CON02\ses-postop\HRF.csv


C:\Users\dinar\Desktop\gsoc\data\brain_tumor\sub-CON02\ses-postop\HRF.mat


C:\Users\dinar\Desktop\gsoc\data\brain_tumor\sub-CON02\ses-postop\ROIts.dat


C:\Users\dinar\Desktop\gsoc\data\brain_tumor\sub-CON02\ses-postop\SC.zip


C:\Users\dinar\Desktop\gsoc\data\brain_tumor\sub-CON02\ses-postop\SCthrAn.mat


C:\Users\dinar\Desktop\gsoc\data\brain_tumor\sub-CON02\ses-preop\FC.mat


C:\Users\din

In [6]:
# unzipping folder

import zipfile
 
zip_file = 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\SC.zip'
basename = os.path.basename(zip_file)
 
try:
    with zipfile.ZipFile(zip_file) as z:
        z.extractall(zip_file.replace(basename, ''))
        print('Extracted all')
except:
    print('Invalid file')

Extracted all


In [7]:
import zipfile


def extract_files(path):
    basename = os.path.basename(path)
    folder_path = path.replace(basename, '')
    old_structure = os.listdir(folder_path)
    
    try:
        with zipfile.ZipFile(path) as z:
            z.extractall(path.replace(basename, ''))
    except:
        print('Invalid file')
    else:
        new = os.listdir(folder_path)
        return old_structure, [x for x in new if x != '.DS_Store']
                    
                    
extract_files('C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\SC.zip')

(['.DS_Store',
  'areas.txt',
  'average_orientations.txt',
  'centres.txt',
  'cortical.txt',
  'FC.mat',
  'hemisphere.txt',
  'HRF.csv',
  'HRF.mat',
  'ROIts.dat',
  'SC.zip',
  'SCthrAn.mat',
  'tract_lengths.txt',
  'weights.txt'],
 ['areas.txt',
  'average_orientations.txt',
  'centres.txt',
  'cortical.txt',
  'FC.mat',
  'hemisphere.txt',
  'HRF.csv',
  'HRF.mat',
  'ROIts.dat',
  'SC.zip',
  'SCthrAn.mat',
  'tract_lengths.txt',
  'weights.txt'])

In [5]:
path = 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor'
files = os.listdir(path)

In [6]:
import re

re.findall('[0-9]+', 'sub1231')

['1231']

In [9]:
import os
import csv
from incf.convert import convert
import os
from collections import OrderedDict
import incf.preprocess.preprocess as prep
import incf.convert.convert as conv


class Files:
    def __init__(self, path, files):
        self.path = path
        self.files = files
        self.subs = OrderedDict()

        # decide whether input files are for one or more patients
        self.content = conv.get_content(path, files)
        self.basename = set(conv.traverse_files(path, files, basename=True))
        self.single = len(self.content) == len(self.basename)

        # set multi-subject input to true
        conv.MULTI_INPUT = False if self.single else True
        self.traverse_files()

    def traverse_files(self):
        # folder structure inputs
        if conv.MULTI_INPUT:  
            main_folder = os.path.basename(self.path)
            
            for sel in sels:
                if sel not in self.subs.keys():
                    self.subs[sel] = {}

                if 'ses-preop' in os.listdir(os.path.join(self.path, sel)):
                    self.subs[sel].update(prepare_subs(conv.get_content(self.path, sel), sel, suffix='preop'))
                elif 'ses-postop' in os.listdir(os.path.join(self.path, sel)):
                    self.subs[sel].update(prepare_subs(conv.get_content(self.path, sel), sel, suffix='postop'))
                else:
                    self.subs[sel] = prepare_subs(conv.get_content(self.path, sel), sel)

        else:
            sid = prep.create_uuid()
            self.subs[sid] = prepare_subs(conv.get_content(self.path, self.files), sid)
            
            
def find_str(path, word, before=True):
    split = path.find(word) + len(word) + 1 
    
    if before:
        return path[:split]
    return path[split:]            


def prepare_subs(file_paths, sid, suffix=None):
    subs = {}
    suffix = '' if suffix is None else suffix
    
    for file_path in file_paths:
        name = get_filename(file_path)
        desc = convert.DESC + 'h5' if file_path.endswith('h5') else convert.DESC
        nsuffix = name.split('.')[0] + '_' + suffix + '.'
        
        n = nsuffix + name.split('.')[1]
        
        subs[n] = {
            'name': nsuffix,
            'fname': name + name.split('.')[1],
            'sid': sid,
            'desc': desc,
            'path': file_path,
            'ext': get_file_ext(file_path),
        }
        
        if subs[n]['name'].endswith('.txt') or name.endswith('.csv'):
            subs[n]['sep'] = find_separator(file_path)
        
        if subs[n]['name'] in ['tract_lengths', 'tract_length']:
            subs[n]['name'] = 'distances'
        if subs[n]['name'] == 'centres':
            conv.CENTERS = True

    return subs


def get_filename(path):
    return os.path.basename(path)


def get_file_ext(path):
    return path.split('.')[-1]


def find_separator(path):
    """
    Find the separator/delimiter used in the file to ensure no exception
    is raised while reading files.

    :param path:
    :return:
    """
    sniffer = csv.Sniffer()

    with open(path) as fp:
        try:
            delimiter = sniffer.sniff(fp.read(5000)).delimiter
        except Exception:
            delimiter = sniffer.sniff(fp.read(100)).delimiter

    delimiter = '\s' if delimiter == ' ' else delimiter
    return delimiter


In [10]:
path = 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries'
sels = os.listdir(path)

Files(path, sels).subs

OrderedDict([('1',
              {'timeseries_all_.mat': {'name': 'timeseries_all_.',
                'fname': 'timeseries_all.matmat',
                'sid': '1',
                'desc': 'default',
                'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\1\\timeseries_all.mat',
                'ext': 'mat'}}),
             ('2',
              {'timeseries_all_.mat': {'name': 'timeseries_all_.',
                'fname': 'timeseries_all.matmat',
                'sid': '2',
                'desc': 'default',
                'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\2\\timeseries_all.mat',
                'ext': 'mat'}}),
             ('3',
              {'timeseries_all_.mat': {'name': 'timeseries_all_.',
                'fname': 'timeseries_all.matmat',
                'sid': '3',
                'desc': 'default',
                'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\3\\timeseries_all.mat',
                'ext': 'mat'}}),
    

In [9]:
path = 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\txt_files\\2'
sels = os.listdir(path)

Files(path, sels).subs

OrderedDict([('02',
              {'centres_.txt': {'name': 'centres_.',
                'fname': 'centres.txttxt',
                'sid': '02',
                'desc': 'default',
                'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\txt_files\\2\\centres.txt',
                'ext': 'txt'},
               'tract_lengths_.txt': {'name': 'tract_lengths_.',
                'fname': 'tract_lengths.txttxt',
                'sid': '02',
                'desc': 'default',
                'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\txt_files\\2\\tract_lengths.txt',
                'ext': 'txt'},
               'weights_.txt': {'name': 'weights_.',
                'fname': 'weights.txttxt',
                'sid': '02',
                'desc': 'default',
                'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\txt_files\\2\\weights.txt',
                'ext': 'txt'}})])

In [45]:
subjects = OrderedDict()

for sel in sels:
    if sel not in subjects.keys():
        subjects[sel] = {}
    
    if 'ses-preop' in os.listdir(os.path.join(path, sel)):
        subjects[sel].update(prepare_subs(conv.get_content(os.path.join(path), sel), sel, suffix='preop'))
    if 'ses-postop' in os.listdir(os.path.join(path, sel)):
        subjects[sel].update(prepare_subs(conv.get_content(os.path.join(path), sel), sel, suffix='postop'))

In [46]:
subjects

OrderedDict([('sub-CON01',
              {'FC_preop.mat': {'name': 'FC_preop.',
                'fname': 'FC.matmat',
                'sid': 'sub-CON01',
                'desc': 'default',
                'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\FC.mat',
                'ext': 'mat'},
               'HRF_preop.csv': {'name': 'HRF_preop.',
                'fname': 'HRF.csvcsv',
                'sid': 'sub-CON01',
                'desc': 'default',
                'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\HRF.csv',
                'ext': 'csv',
                'sep': ','},
               'HRF_preop.mat': {'name': 'HRF_preop.',
                'fname': 'HRF.matmat',
                'sid': 'sub-CON01',
                'desc': 'default',
                'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\HRF.mat',
                'ext': 'mat'},
               'ROIts_preop.dat': 

In [15]:
p = 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor'
f = Files(p, os.listdir(p))
f.subs

['sub-CON01', 'sub-CON02', 'sub-CON03', 'sub-CON04']
sub-CON01 ses-preop
C:\Users\dinar\Desktop\gsoc\data\brain_tumor\sub-CON01\ses-preop\FC.mat
sub-CON01 ses-preop
C:\Users\dinar\Desktop\gsoc\data\brain_tumor\sub-CON01\ses-preop\HRF.csv
sub-CON01 ses-preop
C:\Users\dinar\Desktop\gsoc\data\brain_tumor\sub-CON01\ses-preop\HRF.mat
sub-CON01 ses-preop
C:\Users\dinar\Desktop\gsoc\data\brain_tumor\sub-CON01\ses-preop\ROIts.dat
sub-CON01 ses-preop
C:\Users\dinar\Desktop\gsoc\data\brain_tumor\sub-CON01\ses-preop\SC.zip
sub-CON01 ses-preop
C:\Users\dinar\Desktop\gsoc\data\brain_tumor\sub-CON01\ses-preop\SCthrAn.mat
sub-CON02 ses-postop
C:\Users\dinar\Desktop\gsoc\data\brain_tumor\sub-CON02\ses-postop\FC.mat
sub-CON02 ses-postop
C:\Users\dinar\Desktop\gsoc\data\brain_tumor\sub-CON02\ses-postop\HRF.csv
sub-CON02 ses-postop
C:\Users\dinar\Desktop\gsoc\data\brain_tumor\sub-CON02\ses-postop\HRF.mat
sub-CON02 ses-postop
C:\Users\dinar\Desktop\gsoc\data\brain_tumor\sub-CON02\ses-postop\ROIts.dat
sub-

OrderedDict([('sub-CON01', {}),
             ('sub-CON02', {}),
             ('sub-CON03', {}),
             ('sub-CON04', {})])

In [ ]:
p = 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\txt_files'
f = Files(p, os.listdir(p))
f.subs

In [7]:
f = Files('C:\\Users\\dinar\\Desktop\\gsoc\\data\\txt_files\\2', ['centres.txt', 'tract_lengths.txt', 'weights.txt'])
f.subs

self.content ['C:\\Users\\dinar\\Desktop\\gsoc\\data\\txt_files\\2\\centres.txt', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\txt_files\\2\\tract_lengths.txt', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\txt_files\\2\\weights.txt']
self.basename {'centres.txt', 'weights.txt', 'tract_lengths.txt'}
self.single True


OrderedDict([('02',
              {'centres.txt': {'fname': 'centres.txt',
                'sid': '02',
                'desc': 'default',
                'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\txt_files\\2\\centres.txt',
                'ext': 'txt',
                'name': 'centres'},
               'tract_lengths.txt': {'fname': 'tract_lengths.txt',
                'sid': '02',
                'desc': 'default',
                'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\txt_files\\2\\tract_lengths.txt',
                'ext': 'txt',
                'name': 'distances'},
               'weights.txt': {'fname': 'weights.txt',
                'sid': '02',
                'desc': 'default',
                'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\txt_files\\2\\weights.txt',
                'ext': 'txt',
                'name': 'weights'}})])

In [8]:
p = 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor'

f = Files(p, os.listdir(p))
f.subs

self.content ['C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\FC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\HRF.csv', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\HRF.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\ROIts.dat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\SC.zip', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON01\\ses-preop\\SCthrAn.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\FC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\HRF.csv', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\HRF.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\ROIts.dat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\SC.zip', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\br

IndexError: list index out of range

In [18]:
f = Files(path, filesssss)


NameError: name 'filesssss' is not defined

In [21]:
filesssss = os.listdir(path)
files = Files(path, filesssss)

for k, v in files.subs.items():
    print(k, v, end='\n\n\n')

sub-CON01 {'ses-preop': ['FC.mat', 'HRF.csv', 'HRF.mat', 'ROIts.dat', 'SC.zip', 'SCthrAn.mat']}


sub-CON02 {'ses-postop': ['FC.mat', 'HRF.csv', 'HRF.mat', 'ROIts.dat', 'SC.zip', 'SCthrAn.mat'], 'ses-preop': ['FC.mat', 'HRF.csv', 'HRF.mat', 'ROIts.dat', 'SC.zip', 'SCthrAn.mat']}


sub-CON03 {'ses-postop': ['FC.mat', 'HRF.csv', 'HRF.mat', 'ROIts.dat', 'SC.zip', 'SCthrAn.mat'], 'ses-preop': ['FC.mat', 'HRF.csv', 'HRF.mat', 'ROIts.dat', 'SC.zip', 'SCthrAn.mat']}


sub-CON04 {'ses-postop': ['FC.mat', 'HRF.csv', 'HRF.mat', 'ROIts.dat', 'SC.zip', 'SCthrAn.mat'], 'ses-preop': ['FC.mat', 'HRF.csv', 'HRF.mat', 'ROIts.dat', 'SC.zip', 'SCthrAn.mat']}




In [ ]:
def combine_folders(path, files):
    

In [35]:
def find_str(path, word, before=True):
    split = path.find(word) + len(word) + 1 
    
    if before:
        return path[:split]
    return path[split:]


filess = ['C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\FC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\HRF.csv', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\HRF.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\ROIts.dat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\SC.zip', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\SCthrAn.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\FC.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\HRF.csv', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\HRF.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\ROIts.dat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\SC.zip', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\SCthrAn.mat']
folders = OrderedDict()
main_folder = os.path.basename(path) 
folders[main_folder] = OrderedDict()
current_folder = None

for file in filess:
    p = find_str(file, main_folder, before=False)
    split = p.split('\\')
    
    if split[0] not in folders[main_folder]:
        folders[main_folder][split[0]] = {}
        
    if os.path.isdir(find_str(file, split[1])) and split[1] not in folders[main_folder][split[0]]:
        folders[main_folder][split[0]][split[1]] = []
    
    folders[main_folder][split[0]][split[1]].append(os.path.basename(file))
    
    



sub-CON02\ses-postop\FC.mat
sub-CON02\ses-postop\HRF.csv
sub-CON02\ses-postop\HRF.mat
sub-CON02\ses-postop\ROIts.dat
sub-CON02\ses-postop\SC.zip
sub-CON02\ses-postop\SCthrAn.mat
sub-CON02\ses-preop\FC.mat
sub-CON02\ses-preop\HRF.csv
sub-CON02\ses-preop\HRF.mat
sub-CON02\ses-preop\ROIts.dat
sub-CON02\ses-preop\SC.zip
sub-CON02\ses-preop\SCthrAn.mat


In [36]:
folders

OrderedDict([('brain_tumor',
              OrderedDict([('sub-CON02',
                            {'ses-postop': ['FC.mat',
                              'HRF.csv',
                              'HRF.mat',
                              'ROIts.dat',
                              'SC.zip',
                              'SCthrAn.mat'],
                             'ses-preop': ['FC.mat',
                              'HRF.csv',
                              'HRF.mat',
                              'ROIts.dat',
                              'SC.zip',
                              'SCthrAn.mat']})]))])

In [18]:
def get_content(path, files):
    all_files = []
    
    if isinstance(files, str):
        return conv.traverse_files(os.path.join(path, files))

    for file in files:
        if os.path.isdir(os.path.join(path, file)):
            print(True)
            all_files += conv.traverse_files(os.path.join(path, file))
        else:
            all_files.append(os.path.join(path, file))
    return all_files


files = os.listdir(path)
get_content(path, files[1])

['C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\FC.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\HRF.csv',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\HRF.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\ROIts.dat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\SC.zip',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-postop\\SCthrAn.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\FC.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\HRF.csv',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\HRF.mat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\ROIts.dat',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\brain_tumor\\sub-CON02\\ses-preop\\SC.zip',
 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\bra

In [6]:
from scipy.io import loadmat


loadmat('C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\1\\timeseries_all.mat')

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Thu Feb  6 12:00:52 2020',
 '__version__': '1.0',
 '__globals__': [],
 'data': array([[12974.08918, 11915.09521, 12078.56191, ...,  9686.10222,
         10282.49023, 10792.33884],
        [12959.45346, 11819.84904, 12055.11605, ...,  9665.7039 ,
         10121.32577, 10689.26117],
        [12905.20787, 11837.78934, 11962.39905, ...,  9593.44135,
         10135.87439, 10713.51698],
        ...,
        [13157.87715, 11893.20308, 12102.12388, ...,  9584.80664,
         10276.75591, 10826.52015],
        [13083.40412, 11890.30498, 12074.14658, ...,  9550.86363,
         10231.36131, 10768.50695],
        [12970.37017, 11870.84991, 12098.38231, ...,  9557.05501,
         10139.23922, 10715.65225]])}

In [7]:
from collections import OrderedDict
f = OrderedDict([('04', {'timeseries_all.mat': {'fname': 'timeseries_all.mat', 'sid': '04', 'sep': None, 'desc': 'default', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\1\\timeseries_all.mat', 'ext': 'mat', 'name': 'timeseries_all'}}), ('05', {'timeseries_all.mat': {'fname': 'timeseries_all.mat', 'sid': '05', 'sep': None, 'desc': 'default', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\2\\timeseries_all.mat', 'ext': 'mat', 'name': 'timeseries_all'}}), ('06', {'timeseries_all.mat': {'fname': 'timeseries_all.mat', 'sid': '06', 'sep': None, 'desc': 'default', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\3\\timeseries_all.mat', 'ext': 'mat', 'name': 'timeseries_all'}}), ('07', {'timeseries_all.mat': {'fname': 'timeseries_all.mat', 'sid': '07', 'sep': None, 'desc': 'default', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\4\\timeseries_all.mat', 'ext': 'mat', 'name': 'timeseries_all'}}), ('08', {'timeseries_all.mat': {'fname': 'timeseries_all.mat', 'sid': '08', 'sep': None, 'desc': 'default', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\5\\timeseries_all.mat', 'ext': 'mat', 'name': 'timeseries_all'}})])


if isinstance(f, OrderedDict):
    print(True)




True


In [5]:
class FolderStructure:
    def __init__(self, path, subs):
        self.path = path
        self.subs = subs

        self.components = OrderedDict({
            'code': [],
            'coord': [],
            'eq': [],
            'param': [],
            'subjects': {},
            'files': ['CHANGES.txt', 'dataset_description.json', 'participants.tsv', 'README.txt']
        })

        self.layout = []
        self.default_format = 'sub-{}_desc-{}_{}.{}'
        self.coord_format = 'desc-{}_{}.{}'
        self.populate()

    def common_structure(self, v, name=None):
        name = v['name'] if name is None else name
        return [self.default_format.format(v['sid'], v['desc'], name, 'tsv'),
                self.default_format.format(v['sid'], v['desc'], name, 'json')]

    def coord_structure(self, v):
        return [self.coord_format.format(v['desc'], 'nodes', 'tsv'),
                self.coord_format.format(v['desc'], 'nodes', 'json'),
                self.coord_format.format(v['desc'], 'labels', 'tsv'),
                self.coord_format.format(v['desc'], 'labels', 'json')]
    
    def iterate(self, k, v):
        sid = v['sid']
        if sid not in self.components['subjects']:
            self.components['subjects'][sid] = {'net': [], 'ts': [], 'spatial': []}

        if k in ['weights.txt', 'tract_lengths.txt', 'distances.txt']:
            self.components['subjects'][sid]['net'] += self.common_structure(v)
        elif k in ['centres.txt', 'centers.txt']:
            self.components['coord'] += self.coord_structure(v)
        elif k.endswith('.mat'):
            self.components['subjects'][sid]['ts'] += self.common_structure(v)
            self.components['coord'] += [self.coord_format.format(v['desc'], 'times', 'tsv'),
                                         self.coord_format.format(v['desc'], 'times', 'json')]
        elif k.endswith('.h5'):
            file = h.File(v['path'])
            keys = file.keys()
            name = v['fname'].split('_')[0].lower()

            sid = v['sid']
            if sid not in self.components['subjects']:
                self.components['subjects'][sid] = {'net': [], 'ts': [], 'spatial': []}

            if sim.check_params(file):
                self.components['subjects'][sid]['net'] += self.common_structure(v, 'weights')
                self.components['subjects'][sid]['net'] += self.common_structure(v, 'distances')
                self.components['coord'] += self.coord_structure(v)
            else:
                if len(list(keys)) > 0:
                    self.components['param'] += [self.coord_format.format(v['desc'], name, 'xml'),
                                                 self.coord_format.format(v['desc'], name, 'json')]

    def populate(self):
        for k, v in self.subs.items():
            if 'sid' in v:
                # traverse single-instance files
                self.iterate(k, v)
            else:
                for k2, v2 in v.items():
                    self.iterate(k2, v2)
                    
        self.create_layout()

    def join(self, files, form='files'):
        subfile = '&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___{}<br>'
        file = '&emsp;&emsp;&emsp;&emsp;|___{}<br>'
        joiner = lambda x: ''.join(x)

        if form == 'files':
            return joiner([file.format(f) for f in files])
        return joiner([subfile.format(f) for f in files])

    def create_layout(self):
        fold = '&emsp;&emsp;|___{}/<br>'
        subfold = '&emsp;&emsp;&emsp;&emsp;|___{}/<br>'
        main_files = '|___{}<br>'

        self.layout.append('|___ output/<br>')

        for k, v in self.components.items():
            if len(v) == 0:
                self.layout.append(fold.format(k))
            else:
                if isinstance(v, list) and k != 'files':
                    self.layout += [fold.format(k), self.join(v)]
                elif isinstance(v, dict):
                    for k2, v2 in v.items():
                        self.layout += [fold.format(f'sub-{k2}'), subfold.format('net'),
                                        self.join(v2['net'], 'subfile'), subfold.format('ts'),
                                        self.join(v2['ts'], 'subfile'), subfold.format('spatial'),
                                        self.join(v2['spatial'], 'subfile')]

        self.layout += [main_files.format(x) for x in self.components['files']]
        self.layout = ''.join(self.layout)

In [6]:
from collections import OrderedDict

subs = OrderedDict([('01', {'timeseries_all.mat': {'fname': 'timeseries_all.mat', 'sid': '01', 'sep': None, 'desc': 'default', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\1\\timeseries_all.mat', 'ext': 'mat', 'name': 'timeseries_all'}}), ('02', {'timeseries_all.mat': {'fname': 'timeseries_all.mat', 'sid': '02', 'sep': None, 'desc': 'default', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\2\\timeseries_all.mat', 'ext': 'mat', 'name': 'timeseries_all'}}), ('03', {'timeseries_all.mat': {'fname': 'timeseries_all.mat', 'sid': '03', 'sep': None, 'desc': 'default', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\3\\timeseries_all.mat', 'ext': 'mat', 'name': 'timeseries_all'}}), ('04', {'timeseries_all.mat': {'fname': 'timeseries_all.mat', 'sid': '04', 'sep': None, 'desc': 'default', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\4\\timeseries_all.mat', 'ext': 'mat', 'name': 'timeseries_all'}}), ('05', {'timeseries_all.mat': {'fname': 'timeseries_all.mat', 'sid': '05', 'sep': None, 'desc': 'default', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\5\\timeseries_all.mat', 'ext': 'mat', 'name': 'timeseries_all'}})])

structure = FolderStructure('C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries', subs)





In [7]:
structure.layout

'|___ output/<br>&emsp;&emsp;|___code/<br>&emsp;&emsp;|___coord/<br>&emsp;&emsp;&emsp;&emsp;|___desc-default_times.tsv<br>&emsp;&emsp;&emsp;&emsp;|___desc-default_times.json<br>&emsp;&emsp;&emsp;&emsp;|___desc-default_times.tsv<br>&emsp;&emsp;&emsp;&emsp;|___desc-default_times.json<br>&emsp;&emsp;&emsp;&emsp;|___desc-default_times.tsv<br>&emsp;&emsp;&emsp;&emsp;|___desc-default_times.json<br>&emsp;&emsp;&emsp;&emsp;|___desc-default_times.tsv<br>&emsp;&emsp;&emsp;&emsp;|___desc-default_times.json<br>&emsp;&emsp;&emsp;&emsp;|___desc-default_times.tsv<br>&emsp;&emsp;&emsp;&emsp;|___desc-default_times.json<br>&emsp;&emsp;|___eq/<br>&emsp;&emsp;|___param/<br>&emsp;&emsp;|___sub-01/<br>&emsp;&emsp;&emsp;&emsp;|___net/<br>&emsp;&emsp;&emsp;&emsp;|___ts/<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-01_desc-default_timeseries_all.tsv<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|___sub-01_desc-default_timeseries_all.json<br>&emsp;&emsp;&emsp;&emsp;|___spatial/<br>&emsp;&emsp;|___sub-02/<br>&emsp;&e

In [7]:
import incf.convert.convert as conv

# conv.check_compatibility(['C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\1\\timeseries_all.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\2\\timeseries_all.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\3\\timeseries_all.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\4\\timeseries_all.mat', 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\5\\timeseries_all.mat'])
files = ['1', '2', '3', '4', '5']
path = 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries'
content = conv.get_content(path, files)
basenames = set(conv.traverse_files(path, basename=True))

len(basenames) == len(content)

False

In [10]:
files = ['sub-CON01', 'sub-CON02', 'sub-CON03', 'sub-CON04', 'sub-CON05']
path = 'C:\\Users\\dinar\\Desktop\\TVB_brain_tumor'
content = conv.get_content(path, files)
content
basenames = set(conv.traverse_files(path, basename=True))

len(basenames) == len(content)

False

In [6]:
from collections import OrderedDict
import incf.preprocess.preprocess as prep
import incf.convert.convert as conv 


class Files:
    def __init__(self, path, files):
        self.path = path
        self.files = files 
        
        self.subs = None
        
        # decide whether input files are for one or more patients 
        self.content = conv.get_content(path, files)
        self.basename = set(conv.traverse_files(path, basename=True))
        self.multi = len(self.content) == len(self.basename)
        self.traverse_files()
    
    def traverse_files(self):
        if not self.multi:
            self.subs = OrderedDict()
            
            for file in self.files:
                if os.path.isdir(os.path.join(self.path, file)):
                    sid = prep.create_uuid()
                    self.subs[sid] = conv.prepare_subs(conv.get_content(self.path, file), sid)
            
        else:
            self.subs = conv.prepare_subs(self.content, sid=prep.create_uuid()) 
        

In [8]:
files = Files('C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries', ['1'])
files.subs

{1: '01', 2: '02', 3: '03', 4: '04', 5: '05', 6: '06'}
['C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\1\\timeseries_all.mat']
{'timeseries_all.mat': {'fname': 'timeseries_all.mat', 'sid': '06', 'sep': None, 'desc': 'default', 'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\1\\timeseries_all.mat', 'ext': 'mat', 'name': 'timeseries_all'}}


{'timeseries_all.mat': {'fname': 'timeseries_all.mat',
  'sid': '06',
  'sep': None,
  'desc': 'default',
  'path': 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\1\\timeseries_all.mat',
  'ext': 'mat',
  'name': 'timeseries_all'}}

In [6]:
for file in ['1', '2', '3', '4', '5']:
    p = os.path.join('C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries', file)
    
    if os.path.isdir(p):
        print(True)

True
True
True
True
True


In [7]:
os.pardir('C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\1')

TypeError: 'str' object is not callable

In [9]:
os.path.abspath(os.path.join('C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\1', os.pardir))

'C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries'

In [1]:
pd.read_csv('C:\\Users\\dinar\\Desktop\\gsoc\\data\\txt_files\\1\\weights.txt', sep=' ', header=None, index_col=False)[:]

NameError: name 'pd' is not defined

In [10]:
import csv

def find_separator(path):
    """
    Find the separator/delimiter used in the file to ensure no exception
    is raised while reading files.

    :param path:
    :return:
    """
    if path.endswith('.mat') or path.endswith('.h5'):
        return

    sniffer = csv.Sniffer()

    with open(path) as fp:
        try:
            delimiter = sniffer.sniff(fp.read(5000)).delimiter
        except Exception:
            delimiter = sniffer.sniff(fp.read(100)).delimiter

    delimiter = '\s' if delimiter == ' ' else delimiter
    return delimiter


find_separator('C:\\Users\\dinar\\Desktop\\gsoc\\data\\txt_files\\1\\weights.txt')

'\\s'

In [ ]:
def reorder_files(current, future):
    order = []
    
    

In [ ]:
import re

re.findall('[a-zA-Z.]*')

In [6]:
a = [1, 2, 3, 4]
a[2:4]

[3, 4]

In [5]:
def get_current_output(path, indentation=2):
    tree = []

    if os.path.exists(path) and len(os.listdir(path)) > 0:
        for root, dirs, files in os.walk(path):
            level = root.replace(path, '').count(os.sep)
            indent = '&emsp;' * indentation * level
            tree.append('{}{}/'.format(indent, os.path.basename(root)))
        tree = '<br>'.join(tree)
    return tree


get_current_output('../output')

'output/<br>&emsp;&emsp;code/<br>&emsp;&emsp;coord/<br>&emsp;&emsp;eq/<br>&emsp;&emsp;param/<br>&emsp;&emsp;sub-10qA0/<br>&emsp;&emsp;&emsp;&emsp;net/<br>&emsp;&emsp;&emsp;&emsp;spatial/<br>&emsp;&emsp;&emsp;&emsp;ts/<br>&emsp;&emsp;sub-kh0iA/<br>&emsp;&emsp;&emsp;&emsp;net/<br>&emsp;&emsp;&emsp;&emsp;spatial/<br>&emsp;&emsp;&emsp;&emsp;ts/<br>&emsp;&emsp;sub-RuBcx/<br>&emsp;&emsp;&emsp;&emsp;net/<br>&emsp;&emsp;&emsp;&emsp;spatial/<br>&emsp;&emsp;&emsp;&emsp;ts/'

In [12]:
from pathlib import Path
from itertools import islice

space =  '    '
branch = '│   '
tee =    '├── '
last =   '└── '



def tree(dir_path: Path, level: int=-1, limit_to_directories: bool=False,
         length_limit: int=1000):
    """Given a directory Path object print a visual tree structure"""
    struct = []
    
    dir_path = Path(dir_path) # accept string coerceable to Path
    files = 0
    directories = 0
    def inner(dir_path: Path, prefix: str='', level=-1):
        nonlocal files, directories
        if not level: 
            return # 0, stop iterating
        if limit_to_directories:
            contents = [d for d in dir_path.iterdir() if d.is_dir()]
        else: 
            contents = list(dir_path.iterdir())
        pointers = [tee] * (len(contents) - 1) + [last]
        for pointer, path in zip(pointers, contents):
            if path.is_dir():
                yield prefix + pointer + path.name
                directories += 1
                extension = branch if pointer == tee else space 
                yield from inner(path, prefix=prefix+extension, level=level-1)
            elif not limit_to_directories:
                yield prefix + pointer + path.name
                files += 1
    struct.append(dir_path.name)
    
    
    iterator = inner(dir_path, level=level)
    for line in islice(iterator, length_limit):
        struct.append(line)
    return '<br>'.join(struct)
    

tree('../output')

'output<br>├── CHANGES.txt<br>├── code<br>├── coord<br>│   ├── desc-default_connectivity_labels.json<br>│   ├── desc-default_connectivity_labels.tsv<br>│   ├── desc-default_connectivity_nodes.json<br>│   └── desc-default_connectivity_nodes.tsv<br>├── dataset_description.json<br>├── eq<br>├── param<br>│   └── desc-default-generic2doscillator.xml<br>├── participants.tsv<br>├── README.txt<br>├── sub-10qA0<br>│   ├── net<br>│   ├── spatial<br>│   └── ts<br>├── sub-kh0iA<br>│   ├── net<br>│   │   ├── sub-kh0iA_desc-default_connectivity-distances.json<br>│   │   ├── sub-kh0iA_desc-default_connectivity-distances.tsv<br>│   │   ├── sub-kh0iA_desc-default_connectivity-weights.json<br>│   │   └── sub-kh0iA_desc-default_connectivity-weights.tsv<br>│   ├── spatial<br>│   └── ts<br>└── sub-RuBcx<br>    ├── net<br>    │   ├── sub-RuBcx_desc-default_connectivity-distances.json<br>    │   ├── sub-RuBcx_desc-default_connectivity-distances.tsv<br>    │   ├── sub-RuBcx_desc-default_connectivity-weights.j

In [46]:
import pprint

pp = pprint.PrettyPrinter()

def path_to_dict(path, d={'dirs':{},'files':[]}):
    
    name = os.path.basename(path)

    if os.path.isdir(path):
        if name not in d['dirs']:
            d['dirs'][name] = {'dirs':{},'files':[]}
        for x in os.listdir(path):
            path_to_dict(os.path.join(path,x), d['dirs'][name])
    else:
        d['files'].append(name)
        
    return d['dirs']


mydict = path_to_dict('../output')
mydict

{'output': {'dirs': {'code': {'dirs': {}, 'files': []},
   'coord': {'dirs': {},
    'files': ['desc-default_connectivity_labels.json',
     'desc-default_connectivity_labels.tsv',
     'desc-default_connectivity_nodes.json',
     'desc-default_connectivity_nodes.tsv']},
   'eq': {'dirs': {}, 'files': []},
   'param': {'dirs': {}, 'files': ['desc-default-generic2doscillator.xml']},
   'sub-10qA0': {'dirs': {'net': {'dirs': {}, 'files': []},
     'spatial': {'dirs': {}, 'files': []},
     'ts': {'dirs': {}, 'files': []}},
    'files': []},
   'sub-kh0iA': {'dirs': {'net': {'dirs': {},
      'files': ['sub-kh0iA_desc-default_connectivity-distances.json',
       'sub-kh0iA_desc-default_connectivity-distances.tsv',
       'sub-kh0iA_desc-default_connectivity-weights.json',
       'sub-kh0iA_desc-default_connectivity-weights.tsv']},
     'spatial': {'dirs': {}, 'files': []},
     'ts': {'dirs': {}, 'files': []}},
    'files': []},
   'sub-RuBcx': {'dirs': {'net': {'dirs': {},
      'files':

In [45]:
output = list(mydict.keys())[0]
mydict[output]
# output

{'dirs': {'code': {'dirs': {}, 'files': []},
  'coord': {'dirs': {},
   'files': ['desc-default_connectivity_labels.json',
    'desc-default_connectivity_labels.tsv',
    'desc-default_connectivity_nodes.json',
    'desc-default_connectivity_nodes.tsv']},
  'eq': {'dirs': {}, 'files': []},
  'param': {'dirs': {}, 'files': ['desc-default-generic2doscillator.xml']},
  'sub-10qA0': {'dirs': {'net': {'dirs': {}, 'files': []},
    'spatial': {'dirs': {}, 'files': []},
    'ts': {'dirs': {}, 'files': []}},
   'files': []},
  'sub-kh0iA': {'dirs': {'net': {'dirs': {},
     'files': ['sub-kh0iA_desc-default_connectivity-distances.json',
      'sub-kh0iA_desc-default_connectivity-distances.tsv',
      'sub-kh0iA_desc-default_connectivity-weights.json',
      'sub-kh0iA_desc-default_connectivity-weights.tsv']},
    'spatial': {'dirs': {}, 'files': []},
    'ts': {'dirs': {}, 'files': []}},
   'files': []},
  'sub-RuBcx': {'dirs': {'net': {'dirs': {},
     'files': ['sub-RuBcx_desc-default_connec

In [ ]:
def get_structure(path, files):
    struct = path_to_dict('../output')
    output = list(struct.keys())[0]
    struct = struct[output]
    
    add_structure(struct, files)
    

def add_structure(struct, files):
    for file in files:
        
    
    

In [ ]:
def create_layout(subs=None, output='../output'):
    """
    Create folder structure according to BEP034 and passed `subs` parameter.

    :param output:
    :param subs:
    :return:
    """

    output = output.replace('.', '').replace('/', '')
    layout = create_sub(subs)
    layout = '&emsp;'.join(layout) if len(layout) > 1 else ''.join(layout)

    return f"""
    {output}/ <br>
        &emsp;|___ code <br>
        &emsp;|___ eq <br>
        &emsp;|___ param <br>
        &emsp;{layout}
        &emsp;|___ README <br>
        &emsp;|___ CHANGES <br>
        &emsp;|___ dataset_description.json <br>
        &emsp;|___ participants.tsv
    """

In [25]:
def showFolderTree(path, indentation=2):
    """
    Shows the content of a folder in a tree structure.
    path -(string)- path of the root folder we want to show.
    show_files -(boolean)-  Whether or not we want to see files listed.
                            Defaults to False.
    indentation -(int)- Indentation we want to use, defaults to 2.   
    file_output -(string)-  Path (including the name) of the file where we want
                            to save the tree.
    """


    tree = []

    for root, dirs, files in os.walk(path):
        level = root.replace(path, '').count(os.sep)
        indent = '&emsp;'*indentation*(level)
        tree.append('{}{}/'.format(indent, os.path.basename(root)))
    return tree

            
showFolderTree('../output')

['output/',
 '&emsp;&emsp;code/',
 '&emsp;&emsp;coord/',
 '&emsp;&emsp;eq/',
 '&emsp;&emsp;param/',
 '&emsp;&emsp;sub-10qA0/',
 '&emsp;&emsp;&emsp;&emsp;net/',
 '&emsp;&emsp;&emsp;&emsp;spatial/',
 '&emsp;&emsp;&emsp;&emsp;ts/',
 '&emsp;&emsp;sub-kh0iA/',
 '&emsp;&emsp;&emsp;&emsp;net/',
 '&emsp;&emsp;&emsp;&emsp;spatial/',
 '&emsp;&emsp;&emsp;&emsp;ts/',
 '&emsp;&emsp;sub-RuBcx/',
 '&emsp;&emsp;&emsp;&emsp;net/',
 '&emsp;&emsp;&emsp;&emsp;spatial/',
 '&emsp;&emsp;&emsp;&emsp;ts/']

In [30]:
def get_folder_contents(path, subs):
    subs[path] = os.listdir()
    
    for file in os.listdir(path):
        if os.path.isdir(os.path.join(path, file)):
             get_folder_contents(os.path.join(path, file), subs)
            
        


def get_contents(path):
    subs = OrderedDict({'main': []})
    
    for file in os.listdir(path):
        dir_path = os.path.join(path, file)
        if os.path.isdir(dir_path):
            print(get_folder_contents(dir_path, subs))
            
        else:
            subs['main'].append(file)
            
    print(subs)
        
        
get_contents('../output')

None
None
None
None
None
None
None
OrderedDict([('main', ['CHANGES.txt', 'dataset_description.json', 'participants.tsv', 'README.txt']), ('../output\\coord', 'desc-default_connectivity_nodes.tsv'), ('../output\\param', 'desc-default-generic2doscillator.xml')])


In [14]:
from collections import OrderedDict

def get_current_files(path):
    file_count, dir_count = 0, 0
    struct = OrderedDict()
    
    for root, dirs, files in os.walk(path):
        level = root.replace(path, '').count(os.sep)
        indent = ' ' * 4 * level
        subindent = ' ' * 4 * (level + 1)
        
        name = os.path.basename(root)
        struct[name] = []
        struct[name].append('{}{}\n'.format(indent, name))
        
        for file in files:
            struct[name].append('{}{}'.format(subindent, file))
            file_count += 1
        dir_count += len(dirs)
        
    return struct
        
get_current_files('../output')

OrderedDict([('output',
              ['output\n',
               '    CHANGES.txt',
               '    dataset_description.json',
               '    participants.tsv',
               '    README.txt']),
             ('code', ['    code\n']),
             ('coord',
              ['    coord\n',
               '        desc-default_connectivity_labels.json',
               '        desc-default_connectivity_labels.tsv',
               '        desc-default_connectivity_nodes.json',
               '        desc-default_connectivity_nodes.tsv']),
             ('eq', ['    eq\n']),
             ('param',
              ['    param\n', '        desc-default-generic2doscillator.xml']),
             ('sub-10qA0', ['    sub-10qA0\n']),
             ('net',
              ['        net\n',
               '            sub-RuBcx_desc-default_connectivity-distances.json',
               '            sub-RuBcx_desc-default_connectivity-distances.tsv',
               '            sub-RuBcx_desc-defa

In [58]:
def list_files(startpath, n_indent=2):
    file_count, dir_count = 0, 0
    struct = []
    
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ('&emsp;' * n_indent * level) + '|___'
        subindent = ('&emsp;' * n_indent * (level + 1)) + '|___'
        
        # append values
        struct.append('{}{}/'.format(indent, os.path.basename(root)))
        
        # iterate over content and append values
        for f in files:
            struct.append('{}{}'.format(subindent, f))
            file_count += 1
        dir_count += len(dirs)
        
    txt = f'<br>{dir_count} directories, {file_count} files'
    return struct, txt


In [6]:
def append_files(path, include=None):
    """
    Append all files in a specified file using recursive walk. It implements a
    top-down approach of traversing the directory.

    :param path:
    :param include:
    :return:
    """

    all_files = []
    for root, dirs, files in os.walk(path, topdown=True):
        for file in files:
            if include is None:
                all_files.append(os.path.basename(file))
            else:
                if file in include:
                    all_files.append(os.path.join(root, file))
    return all_files


append_files('C:\\Users\\dinar\\Desktop\\gsoc\\data\\txt_files\\1')

['weights.txt']

In [12]:
append_files('C:\\Users\\dinar\\Desktop\\gsoc\\data\\txt_files\\2')

['centres.txt', 'tract_lengths.txt', 'weights.txt']

In [14]:
import functools

def get_directory_structure(rootdir):
    """
    Creates a nested dictionary that represents the folder structure of rootdir
    """
    dir = {}
    rootdir = rootdir.rstrip(os.sep)
    start = rootdir.rfind(os.sep) + 1
    for path, dirs, files in os.walk(rootdir):
        folders = path[start:].split(os.sep)
        print(f'folders: {folders}')
        subdir = dict.fromkeys(files)
        print(f'subdir: {subdir}')
        parent = functools.reduce(dict.get, folders[:-1], dir)
        print(f'parent: {parent}')
        parent[folders[-1]] = subdir
    return dir

In [15]:
get_directory_structure('C:\\Users\\dinar\\Desktop\\gsoc\\data\\txt_files')

folders: ['txt_files']
subdir: {}
parent: {}
folders: ['txt_files', '1']
subdir: {'weights.txt': None}
parent: {}
folders: ['txt_files', '2']
subdir: {'centres.txt': None, 'tract_lengths.txt': None, 'weights.txt': None}
parent: {'1': {'weights.txt': None}}


{'txt_files': {'1': {'weights.txt': None},
  '2': {'centres.txt': None, 'tract_lengths.txt': None, 'weights.txt': None}}}

In [20]:
import functools

def get_directory_structure(rootdir):
    """
    Creates a nested dictionary that represents the folder structure of rootdir
    """
    dir = {}
    rootdir = rootdir.rstrip(os.sep)
    start = rootdir.rfind(os.sep) + 1
    
    for path, dirs, files in os.walk(rootdir):
        folders = path[start:].split(os.sep)
        subdir = {}
        
        for file in files:
            print(os.path.join(rootdir, file))
            if os.path.isdir(os.path.join(rootdir, file)):
                print(True, file)
        
        
        
        print(f'subdir: {subdir}')
#         parent = functools.reduce(dict.get, folders[:-1], dir)
#         print(f'parent: {parent}')
#         parent[folders[-1]] = subdir
    return dir

In [21]:
get_directory_structure('C:\\Users\\dinar\\Desktop\\gsoc\\data\\txt_files')

subdir: {}
C:\Users\dinar\Desktop\gsoc\data\txt_files\weights.txt
subdir: {}
C:\Users\dinar\Desktop\gsoc\data\txt_files\centres.txt
C:\Users\dinar\Desktop\gsoc\data\txt_files\tract_lengths.txt
C:\Users\dinar\Desktop\gsoc\data\txt_files\weights.txt
subdir: {}


{}

In [13]:
# initializing list
lis = [1, 3, 5, 6, 2, ]
 
# using reduce to compute sum of list
print("The sum of the list elements is : ", end="")
print(functools.reduce(lambda a, b: a+b, lis))

The sum of the list elements is : 17
<function <lambda> at 0x00000255B5C692D0> [1, 3, 5, 6, 2]


In [ ]:
def dir_walk(path):
    all_files = {}
    r, d = None, None
    cur_dir = None

    for root, dirs, files in os.walk(path, topdown=True):
        if r is None:
            r = root


#         rname = os.path.basename(root)

#         all_files[rname] = {'files': [], 'dirs': []}

#         for d in dirs:
#             all_files[rname]['dirs'].append(os.path.join(root, d))
#         for f in files:
#             all_files[rname]['files'].append(os.path.join(root, f))
#         for file in files:
#             print()

#         for file in files:
#             print(os.path.join(root, file))
    return all_files



In [1]:
def dir_walk(path):
    all_files = {}
    r, d = None, None
    cur_dir = None
    
    for root, dirs, files in os.walk(path, topdown=True):
        if r is None:
            r = root
            
        
#         rname = os.path.basename(root)
        
#         all_files[rname] = {'files': [], 'dirs': []}
        
#         for d in dirs:
#             all_files[rname]['dirs'].append(os.path.join(root, d))
#         for f in files:
#             all_files[rname]['files'].append(os.path.join(root, f))
#         for file in files:
#             print()
            
#         for file in files:
#             print(os.path.join(root, file))
    return all_files
            
            

In [2]:
dir_walk('C:\\Users\\dinar\\Desktop\\gsoc\\data')

NameError: name 'os' is not defined

In [64]:
dir_walk('C:\\Users\\dinar\\Desktop\\gsoc\\data\\simulations\\1')

C:\Users\dinar\Desktop\gsoc\data\simulations\1\datatypes\Connectivity_6b28b6c61836452bb52cbd18538a7775.h5
C:\Users\dinar\Desktop\gsoc\data\simulations\1\BurstConfiguration_a29116b38d05429588535667ce4d1eee.h5
C:\Users\dinar\Desktop\gsoc\data\simulations\1\Generic2dOscillator_4e4d2edc01e2448192baf025336e0d73.h5
C:\Users\dinar\Desktop\gsoc\data\simulations\1\HeunDeterministicViewModel_4007260a59a443cfbc076eb4c231c6f5.h5
C:\Users\dinar\Desktop\gsoc\data\simulations\1\Linear_f639e3b95c48436dae5ffaa29072963e.h5
C:\Users\dinar\Desktop\gsoc\data\simulations\1\SimulatorAdapterModel_994a4afbef39461abf2a4feaf9761ca0.h5
C:\Users\dinar\Desktop\gsoc\data\simulations\1\SubSampleViewModel_cd2fe7f44797434b9ae2cc3418a9f091.h5


{}

In [65]:
class Settings(param.Parameterized):
    sub_options = ['Single subject', 'Multiple subjects']
    checkbox_options = ['Traverse sub-folders', ]

    sub_select = pn.widgets.RadioButtonGroup(options=sub_options, button_type='default',
                                             value=[], margin=(-20, 0, 0, 0))
    checkbox = pn.widgets.CheckBoxGroup(value=[], options=checkbox_options)

    @pn.depends('sub_select.value', watch=True)
    def _change_selection(self):
        if self.sub_select.value == self.sub_options[0]:
            # TODO: add all the files to one folder
            print('Selected single value')
#             pass
        else:
            # TODO: infer which relates to which
            pass
        pass

    def view(self):
        return pn.Column(
            '## Settings',
            '#### Select subject count',
            self.sub_select
        )


pn.serve(Settings().view())

C:\Users\dinar\Desktop\gsoc\data\timeseries\1\timeseries_all.mat
C:\Users\dinar\Desktop\gsoc\data\timeseries\2\timeseries_all.mat
C:\Users\dinar\Desktop\gsoc\data\timeseries\3\timeseries_all.mat
C:\Users\dinar\Desktop\gsoc\data\timeseries\4\timeseries_all.mat
C:\Users\dinar\Desktop\gsoc\data\timeseries\5\timeseries_all.mat


{}

In [8]:
class Settings(param.Parameterized):
    sub_options = ['Single subject', 'Multiple subjects']
    checkbox_options = ['Traverse sub-folders', ]
    
    sub_select = pn.widgets.RadioButtonGroup(options=sub_options, button_type='default',
                                             value=[], margin=(-20, 0, 0, 0))
    checkbox = pn.widgets.CheckBoxGroup(value=[], options=checkbox_options)

    @pn.depends('sub_select.value', watch=True)
    def _change_selection(self):
        if self.sub_select.value == self.sub_options[0]:
            # TODO: add all the files to one folder
            print('Selected single value')
#             pass
        else:
            # TODO: infer which relates to which
            pass 
        pass

    def view(self):
        return pn.Column(
            '## Settings',
            '#### Select subject count',
            self.sub_select
        )

    
pn.serve(Settings().view())

Launching server at http://localhost:58960


Selected single value


In [ ]:
checkbox_group = pn.widgets.CheckBoxGroup(
    name='Checkbox Group', value=['Apple', 'Pear'], options=['Apple', 'Banana', 'Pear', 'Strawberry'],
    inline=True)

In [ ]:
def get_files(path='../output', ftype='.json'):
    f = []

    for root, dirs, files in os.walk(path, topdown=False):
        for name in files:
            if ftype in name:
                f.append(os.path.join(root, name))
    return f


class ViewResults(param.Parameterized):
    options = ['JSON files', 'TSV files']
    file_selection = pn.widgets.RadioButtonGroup(options=options, button_type='primary', value=[])
#     widget = pn.WidgetBox('### Select File', select_options)

    def __init__(self, path):
        super().__init__()
        self.path = path
        self.layout = None
        self.select_options = pn.widgets.Select()
        self.widget = pn.WidgetBox('### Select File', self.select_options)

    @pn.depends('file_selection.value', watch=True)
    def _change_filetype(self):
        print(self.file_selection.value)
        if self.file_selection.value == 'JSON files':
            self.select_options.options = get_files(path=self.path)
            self.change_file()
        elif self.file_selection.value == 'TSV files':
            self.select_options.options = get_files(path=self.path, ftype='.tsv')
            self.change_file()

#     @pn.depends('select_options.value', watch=True)
    def change_file(self):
        if len(self.widget) > 2:
            self.widget.pop(-1)

        if self.file_selection.value == 'JSON files':
            try:
                file = json.load(open(self.select_options.value))
            except Exception:
                print(f'File `{self.select_options.value}` is empty!')
            else:
                self.widget.append(pn.widgets.JSONEditor(value=file, height=350))

        elif self.file_selection.value == 'TSV files':
            try:
                file = pd.read_csv(self.select_options.value, sep='\t', header=None, index_col=None)
            except Exception:
                print(f'File `{self.select_options.value}` is empty!')
            else:
                self.widget.append(pn.widgets.Tabulator(file))

    def view(self):
        return pn.Column(self.file_selection, self.widget)


pn.serve(ViewResults('../output').view())

In [4]:
pn.state.notifications.error('This is an error notification.', duration=1000)
pn.state.notifications.info('This is a info notification.', duration=2000)
pn.state.notifications.success('This is a success notification.')
pn.state.notifications.warning('This is a warning notification.', duration=4000);

In [15]:
def get_files(path='../output', ftype='.json'):
    f = []

    for root, dirs, files in os.walk(path, topdown=False):
        for name in files:
            if ftype in name:
                f.append(os.path.join(root, name))
    return f


class ViewResults(param.Parameterized):
    options = ['JSON files', 'TSV files']
    file_selection = pn.widgets.RadioButtonGroup(options=options, button_type='primary', value=[])
#     widget = pn.WidgetBox('### Select File', select_options)
    
    def __init__(self, path):
        super().__init__()
        self.path = path
        self.layout = None 
        self.select_options = pn.widgets.Select()
        self.widget = pn.WidgetBox('### Select File', self.select_options)

    @pn.depends('file_selection.value', watch=True)
    def _change_filetype(self):
        print(self.file_selection.value)
        if self.file_selection.value == 'JSON files':
            self.select_options.options = get_files(path=self.path)
            self.change_file()
        elif self.file_selection.value == 'TSV files':
            self.select_options.options = get_files(path=self.path, ftype='.tsv')
            self.change_file()

#     @pn.depends('select_options.value', watch=True)
    def change_file(self):
        if len(self.widget) > 2:
            self.widget.pop(-1)

        if self.file_selection.value == 'JSON files':
            try:
                file = json.load(open(self.select_options.value))
            except Exception:
                print(f'File `{self.select_options.value}` is empty!')
            else:
                self.widget.append(pn.widgets.JSONEditor(value=file, height=350))

        elif self.file_selection.value == 'TSV files':
            try:
                file = pd.read_csv(self.select_options.value, sep='\t', header=None, index_col=None)
            except Exception:
                print(f'File `{self.select_options.value}` is empty!')
            else:
                self.widget.append(pn.widgets.Tabulator(file))

    def view(self):
        return pn.Column(self.file_selection, self.widget)


pn.serve(ViewResults('../output').view())

Launching server at http://localhost:53762


None
JSON files
File `../output\coord\desc-default_labels.json` is empty!


In [7]:
cs = pn.widgets.CrossSelector(width=700)
pn.serve(cs)

Launching server at http://localhost:50123


In [12]:
string = param.String('hehe')
dir(string)

['__class__',
 '__classdoc',
 '__delattr__',
 '__delete__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__set__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_internal_name',
 '_label',
 '_on_set',
 '_post_setter',
 '_serializers',
 '_set_instantiate',
 '_set_names',
 '_validate',
 '_validate_regex',
 '_validate_value',
 'allow_None',
 'constant',
 'default',
 'deserialize',
 'doc',
 'instantiate',
 'label',
 'name',
 'owner',
 'per_instance',
 'pickle_default_value',
 'precedence',
 'readonly',
 'regex',
 'schema',
 'serialize',
 'watchers']

In [18]:
string.default

'hehe'

In [19]:
string = 'm'

In [21]:
string

AttributeError: 'str' object has no attribute 'default'

### Check if there's a difference between empirical FC & simulated data

In [4]:
from scipy.io import loadmat
import mat73

In [8]:
sim = loadmat("C:\\Users\\dinar\\Desktop\\simulated_data\\1\\AY_20111004_SC_new.mat\\cspeed20\\csf 0.05\\bold.mat", squeeze_me=True)
sim

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Tue Feb  3 22:52:09 2015',
 '__version__': '1.0',
 '__globals__': [],
 'arr': array([[-110.52488034, -445.28196601, -130.27069805, ...,  488.25445135,
          343.85266138,  400.69137436],
        [-110.88021998, -445.46276722, -130.90696322, ...,  486.99161812,
          342.56528117,  399.46318728],
        [-113.08478947, -446.57163538, -134.26416291, ...,  478.1312449 ,
          334.87214655,  391.93338033],
        ...,
        [-621.6660361 , -627.18351817, -763.04972688, ..., -526.41360245,
         -742.10168801, -753.38319788],
        [-611.04317113, -614.24401962, -776.81508901, ..., -541.60214243,
         -747.9391402 , -755.31800616],
        [-600.79134046, -599.38352235, -785.59528833, ..., -552.76523283,
         -742.87992851, -759.7311778 ]])}

In [16]:
np.percentile(sim['arr'], [0, 50, 100])

array([-1177.56475519,  -617.04150771,   588.59473112])

File `None` is empty!
Creating folder `output`...
Creating folder `eq`...
Creating folder `code`...
Creating folder `coord`...
Creating folder `param`...
Creating file `README.txt`...
Creating file `participants.tsv`...
Creating file `dataset_description.json`...
Creating file `CHANGES.txt`...
Creating folder `../output\sub-dD8vA`
Creating folder `../output\sub-dD8vA\net`
Creating folder `../output\sub-dD8vA\spatial`
Creating folder `../output\sub-dD8vA\ts`


In [8]:
data = loadmat('C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\1\\timeseries_all.mat')

np.percentile(data['data'], [0, 50, 100])

array([ 5657.60373 , 11453.467125, 18926.23528 ])

In [15]:
emp_fc = loadmat("C:\\Users\\dinar\\Desktop\\simulated_data\\1\\all_empFC.mat")
np.percentile(emp_fc['all_empFC'], [0, 50, 100])

array([-0.64908182,  0.44813303,  1.        ])

In [11]:
loadmat('C:\\Users\\dinar\\Desktop\\gsoc\\data\\ses-preop\\FC.mat')

{'__header__': b'MATLAB 5.0 MAT-file, written by Octave 4.0.2, 2017-02-21 10:35:45 UTC',
 '__version__': '1.0',
 '__globals__': [],
 'CON01T1_ROIts': array([[  923.70188, 10967.75413, 10075.33814, ...,  9686.10222,
         10282.49023, 10792.33884],
        [  916.27902, 10939.30918,  9958.87689, ...,  9665.7039 ,
         10121.32577, 10689.26117],
        [  906.29391, 10909.99712,  9918.38669, ...,  9593.44135,
         10135.87439, 10713.51698],
        ...,
        [  911.26295, 10948.92151,  9881.90784, ...,  9584.80664,
         10276.75591, 10826.52015],
        [  911.89234, 10926.43679,  9956.72355, ...,  9550.86363,
         10231.36131, 10768.50695],
        [  914.11457, 10917.22143,  9884.80337, ...,  9557.05501,
         10139.23922, 10715.65225]]),
 'CON01T1_ROIts_DK68': array([[12974.08918, 11915.09521, 12078.56191, ...,  9686.10222,
         10282.49023, 10792.33884],
        [12959.45346, 11819.84904, 12055.11605, ...,  9665.7039 ,
         10121.32577, 10689.26117]

### speed20, etc

In [19]:
data = loadmat('C:\\Users\\dinar\\Desktop\\simulated_data\\2\\first_exploration\\AA_20120815_log_SC_csf_0.0001_speed_50.mat',
               squeeze_me=True)
data

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Fri Sep 29 11:46:43 2017',
 '__version__': '1.0',
 '__globals__': [],
 'subs_data': array([[[ 0.51877537,  0.32200462,  0.62658588, ...,  0.60863629,
           0.568951  ,  0.78985981],
         [ 0.58101576,  0.61059093,  0.40993171, ...,  0.55985378,
           0.43702166,  0.40615905]],
 
        [[-1.53511119, -0.49331051, -0.53308641, ..., -0.51905395,
          -0.55381113,  0.04665336],
         [-0.50735542, -0.4094834 , -0.41427879, ...,  0.08833556,
          -0.46056716, -0.46410892]],
 
        [[-1.04546626, -0.35251508, -0.41500527, ..., -0.20582285,
          -1.25996799, -0.7764147 ],
         [-0.34540655, -0.32373849, -0.34252525, ..., -0.39158608,
          -0.34193913, -0.36278282]],
 
        ...,
 
        [[ 0.77479767, -0.98001667, -0.84482694, ..., -1.28450549,
          -1.14173223,  1.56361209],
         [ 1.223174  , -0.36430809, -0.34337304, ...,  0.10573174,
          -0.64746735,  0.495819

In [20]:
data['subs_time'].shape, data['subs_data'].shape

((1500,), (1500, 2, 68))

In [23]:
data['subs_data'][0]

array([[0.51877537, 0.32200462, 0.62658588, 0.41850138, 0.26502229,
        0.43742028, 0.48043986, 0.15356728, 0.54771136, 0.59302886,
        0.48912306, 0.74123135, 0.48709837, 0.37643532, 0.58197464,
        0.96119918, 0.72236114, 0.30243342, 0.21796086, 0.42015013,
        0.31026511, 0.56846824, 0.58036834, 0.56682696, 0.86364423,
        0.02419162, 0.70922303, 0.33941018, 0.40719929, 0.24735968,
        0.65620849, 0.35796918, 0.58750035, 0.47030968, 0.44316665,
        0.53620101, 0.6096645 , 0.14700385, 0.51637336, 0.6265656 ,
        0.50779293, 0.64715845, 0.53941423, 0.3632348 , 0.41904449,
        0.03918741, 0.32387129, 0.75251248, 0.42285177, 0.37035287,
        0.6414281 , 0.56651524, 0.53169924, 0.58234798, 0.73980684,
        0.47107773, 0.77864327, 0.54553439, 0.62377154, 0.1393394 ,
        0.33361236, 0.66322912, 0.436849  , 0.3422435 , 0.48019297,
        0.60863629, 0.568951  , 0.78985981],
       [0.58101576, 0.61059093, 0.40993171, 0.48334802, 0.71842177,
   

In [65]:
os.listdir('C:\\Users\\dinar\\Desktop\\gsoc\\data\\simulations\\1')

['BurstConfiguration_a29116b38d05429588535667ce4d1eee.h5',
 'datatypes',
 'Generic2dOscillator_4e4d2edc01e2448192baf025336e0d73.h5',
 'HeunDeterministicViewModel_4007260a59a443cfbc076eb4c231c6f5.h5',
 'Linear_f639e3b95c48436dae5ffaa29072963e.h5',
 'SimulatorAdapterModel_994a4afbef39461abf2a4feaf9761ca0.h5',
 'SubSampleViewModel_cd2fe7f44797434b9ae2cc3418a9f091.h5']

In [13]:
path = 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\simulations\\1'

for h5 in os.listdir(path):
    if h5 == 'datatypes': continue

    print('\n', h5, '\n\t', h5py.File(os.path.join(path, h5)).keys())


 BurstConfiguration_a29116b38d05429588535667ce4d1eee.h5 
	 <KeysViewHDF5 []>

 Generic2dOscillator_4e4d2edc01e2448192baf025336e0d73.h5 
	 <KeysViewHDF5 ['I', 'a', 'alpha', 'b', 'beta', 'c', 'd', 'e', 'f', 'g', 'gamma', 'tau']>

 HeunDeterministicViewModel_4007260a59a443cfbc076eb4c231c6f5.h5 
	 <KeysViewHDF5 []>

 Linear_f639e3b95c48436dae5ffaa29072963e.h5 
	 <KeysViewHDF5 ['a', 'b']>

 SimulatorAdapterModel_994a4afbef39461abf2a4feaf9761ca0.h5 
	 <KeysViewHDF5 []>

 SubSampleViewModel_cd2fe7f44797434b9ae2cc3418a9f091.h5 
	 <KeysViewHDF5 ['variables_of_interest']>


### Trial to save XML params

```
Generic2dOscillator_4e4d2edc01e2448192baf025336e0d73.h5
	 <KeysViewHDF5 ['I', 'a', 'alpha', 'b', 'beta', 'c', 'd', 'e', 'f', 'g', 'gamma', 'tau']>
```

In [16]:
import h5py
TXT_COLS = ['areas', 'centres', 'cortical', 'hemispheres', 'orientations', 'region_labels', 'tract_lengths', 'weights']


class XML:
    def __init__(self, path, output):
        self.path = path
        self.params = []
        self.output = output
        self.template = """<?xml version="1.0" ?>
<Lems xmlns="http://www.neuroml.org/lems/0.7.6" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.neuroml.org/lems/0.7.6 https://raw.githubusercontent.com/LEMS/LEMS/development/Schemas/LEMS/LEMS_v0.7.6.xsd">
    <Component {}/>
</Lems>"""
        self.parse_file()

    def parse_file(self):
        if self.path.endswith('.h5'):
            self.file = h5py.File(self.path)
            self.keys = list(self.file.keys())
            self.eq = list(self.check_keys())

            for k in self.eq:
                self.params.append('{}="{}"'.format(k, self.file[k][:][0]))

            if len(self.params) > 0:
                print(f'Found following parameters:\n{self.params}')
                self.populate_template()

    def check_keys(self):
        return set(self.keys).difference(set(TXT_COLS))

    def populate_template(self):
        if len(self.params) > 0:
            self.template = self.template.format(' '.join(self.params))
            self.create_xml()

    def create_xml(self):
        with open(self.output, 'w') as f:
            f.write(self.template)


xml = XML('C:\\Users\\dinar\\Desktop\\gsoc\\data\\simulations\\1\\Generic2dOscillator_4e4d2edc01e2448192baf025336e0d73.h5',
          'generic2doscillator.xml')

Found following parameters:
['d="0.02"', 'alpha="1.0"', 'b="-10.0"', 'e="3.0"', 'g="0.0"', 'gamma="1.0"', 'I="0.0"', 'f="1.0"', 'tau="1.0"', 'c="0.0"', 'beta="1.0"', 'a="-2.0"']


In [53]:
path = 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\simulations\\1'

for idx, h5 in enumerate(os.listdir(path)):
    if h5 == 'datatypes': continue

    xml = XML(os.path.join(path, h5), f'test-{idx}')
    xml.parse_file()
    print(xml.template, end='\n\n\n')

Found following parameters:
[]
<?xml version="1.0" ?>
<Lems xmlns="http://www.neuroml.org/lems/0.7.6" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.neuroml.org/lems/0.7.6 https://raw.githubusercontent.com/LEMS/LEMS/development/Schemas/LEMS/LEMS_v0.7.6.xsd">
    <Component {}/>
</Lems>


Found following parameters:
['I', 'a', 'alpha', 'b', 'beta', 'c', 'd', 'e', 'f', 'g', 'gamma', 'tau']
<?xml version="1.0" ?>
<Lems xmlns="http://www.neuroml.org/lems/0.7.6" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.neuroml.org/lems/0.7.6 https://raw.githubusercontent.com/LEMS/LEMS/development/Schemas/LEMS/LEMS_v0.7.6.xsd">
    <Component I="0.0" a="-2.0" alpha="1.0" b="-10.0" beta="1.0" c="0.0" d="0.02" e="3.0" f="1.0" g="0.0" gamma="1.0" tau="1.0"/>
</Lems>


Found following parameters:
[]
<?xml version="1.0" ?>
<Lems xmlns="http://www.neuroml.org/lems/0.7.6" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schem

In [6]:
import h5py

f = h5py.File('C:\\Users\\dinar\\Desktop\\gsoc\\data\\simulations\\1\\datatypes\\Connectivity_6b28b6c61836452bb52cbd18538a7775.h5')
f.keys()

<KeysViewHDF5 ['areas', 'centres', 'cortical', 'hemispheres', 'orientations', 'region_labels', 'tract_lengths', 'weights']>

In [6]:
np.array([str(x).strip("b'") for x in f])

array(['areas', 'centres', 'cortical', 'hemispheres', 'orientations',
       'region_labels', 'tract_lengths', 'weights'], dtype='<U13')

In [8]:
f['region_labels'][:]

array([b'rA1', b'rA2', b'rAMYG', b'rCCA', b'rCCP', b'rCCR', b'rCCS',
       b'rFEF', b'rG', b'rHC', b'rIA', b'rIP', b'rM1', b'rPCI', b'rPCIP',
       b'rPCM', b'rPCS', b'rPFCCL', b'rPFCDL', b'rPFCDM', b'rPFCM',
       b'rPFCORB', b'rPFCPOL', b'rPFCVL', b'rPHC', b'rPMCDL', b'rPMCM',
       b'rPMCVL', b'rS1', b'rS2', b'rTCC', b'rTCI', b'rTCPOL', b'rTCS',
       b'rTCV', b'rV1', b'rV2', b'rCC', b'lA1', b'lA2', b'lAMYG', b'lCCA',
       b'lCCP', b'lCCR', b'lCCS', b'lFEF', b'lG', b'lHC', b'lIA', b'lIP',
       b'lM1', b'lPCI', b'lPCIP', b'lPCM', b'lPCS', b'lPFCCL', b'lPFCDL',
       b'lPFCDM', b'lPFCM', b'lPFCORB', b'lPFCPOL', b'lPFCVL', b'lPHC',
       b'lPMCDL', b'lPMCM', b'lPMCVL', b'lS1', b'lS2', b'lTCC', b'lTCI',
       b'lTCPOL', b'lTCS', b'lTCV', b'lV1', b'lV2', b'lCC'], dtype=object)

In [6]:
import h5py


data = h5py.File('C:\\Users\\dinar\\Desktop\\gsoc\\data\\simulations\\1\\Linear_f639e3b95c48436dae5ffaa29072963e.h5')
dir(data)

['_MutableMapping__marker',
 '__abstractmethods__',
 '__bool__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_d',
 '_e',
 '_gcpl_crt_order',
 '_id',
 '_ipython_key_completions_',
 '_lapl',
 '_lcpl',
 '_libver',
 'attrs',
 'build_virtual_dataset',
 'clear',
 'close',
 'copy',
 'create_dataset',
 'create_dataset_like',
 'create_group',
 'create_virtual_dataset',
 'driver',
 'file',
 'filename',
 'flush',
 'get',
 'id',
 'items',
 'key

In [6]:
data.keys()

<KeysViewHDF5 ['a', 'b']>

In [77]:
data['a'][:]

array([0.05])

In [14]:
data = h5py.File('C:\\Users\\dinar\\Desktop\\gsoc\\data\\simulations\\1\\datatypes\\Connectivity_6b28b6c61836452bb52cbd18538a7775.h5')
data.keys()

<KeysViewHDF5 ['areas', 'centres', 'cortical', 'hemispheres', 'orientations', 'region_labels', 'tract_lengths', 'weights']>

In [82]:
list(set(['centres', 'weights', 'tract_lengths']).intersection(set(data.keys())))

['weights', 'tract_lengths', 'centres']

In [95]:
data['region_labels'][:]

array([b'rA1', b'rA2', b'rAMYG', b'rCCA', b'rCCP', b'rCCR', b'rCCS',
       b'rFEF', b'rG', b'rHC', b'rIA', b'rIP', b'rM1', b'rPCI', b'rPCIP',
       b'rPCM', b'rPCS', b'rPFCCL', b'rPFCDL', b'rPFCDM', b'rPFCM',
       b'rPFCORB', b'rPFCPOL', b'rPFCVL', b'rPHC', b'rPMCDL', b'rPMCM',
       b'rPMCVL', b'rS1', b'rS2', b'rTCC', b'rTCI', b'rTCPOL', b'rTCS',
       b'rTCV', b'rV1', b'rV2', b'rCC', b'lA1', b'lA2', b'lAMYG', b'lCCA',
       b'lCCP', b'lCCR', b'lCCS', b'lFEF', b'lG', b'lHC', b'lIA', b'lIP',
       b'lM1', b'lPCI', b'lPCIP', b'lPCM', b'lPCS', b'lPFCCL', b'lPFCDL',
       b'lPFCDM', b'lPFCM', b'lPFCORB', b'lPFCPOL', b'lPFCVL', b'lPHC',
       b'lPMCDL', b'lPMCM', b'lPMCVL', b'lS1', b'lS2', b'lTCC', b'lTCI',
       b'lTCPOL', b'lTCS', b'lTCV', b'lV1', b'lV2', b'lCC'], dtype=object)

Creating folder `../output\sub-P0kvO`
Creating folder `../output\sub-P0kvO\net`
Creating folder `../output\sub-P0kvO\spatial`
Creating folder `../output\sub-P0kvO\ts`


In [97]:
data['weights'][:]

array([[2., 2., 0., ..., 0., 0., 0.],
       [3., 2., 0., ..., 0., 0., 0.],
       [0., 0., 2., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 2., 2., 0.],
       [0., 0., 0., ..., 3., 2., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [15]:
data['centres'][:]

array([[ -9.885591, -47.084818,  -3.13936 ],
       [ -2.605247, -55.324507,  -7.065423],
       [  6.489431, -11.670519, -30.851544],
       [ 48.734149,  -2.004701,   8.019123],
       [-12.1714  ,  -6.494719,  25.361539],
       [-38.1525  , -13.054752,  -5.607666],
       [ 30.150795,  -4.286333, -31.136279],
       [ 30.400644, -33.753689,  34.832572],
       [ 36.159029, -35.084312, -18.938829],
       [-12.502286, -24.875165, -20.872351],
       [ 23.242996, -42.031295, -22.110624],
       [  9.822772, -44.114976, -18.760442],
       [  8.003023, -32.833685,  38.712349],
       [-22.152851, -56.734891,  16.852029],
       [-51.151034, -35.899675,  25.679908],
       [-43.927847,  -8.226723,  27.584594],
       [-30.092058, -33.534852,  43.097159],
       [ 54.048553, -37.427609,  12.535183],
       [ 59.995574, -14.366518,  26.025739],
       [ 59.612735,  -2.407101,  -0.844581],
       [ 59.286775,  -2.547348, -21.924281],
       [ 47.987255, -16.887697, -39.699154],
       [ 7

In [60]:
loadmat("C:\\Users\\dinar\\Desktop\\simulated_data\\1\\MSE\\P2_log_SC_long_bold_csf_0.1_K11_0.1_K12_0.1.mat")

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Thu Apr 30 16:48:01 2020',
 '__version__': '1.0',
 '__globals__': [],
 'subs_data': array([[[[  -3.84944715,   -1.96095335,   -2.69455228],
          [  -4.04889455,   -1.98311898,   -2.33277659],
          [  -3.94176721,   -2.05415186,   -2.87601393],
          ...,
          [  -4.29774122,   -1.85233491,   -2.90531064],
          [  -4.49811713,   -1.65955181,   -2.68645867],
          [  -3.88675838,   -2.01378071,   -2.872956  ]],
 
         [[  -3.69193033,   -1.68575999,   -2.23651396],
          [  -3.94768865,   -1.94424875,   -2.54331918],
          [  -3.82984156,   -2.33304909,   -2.40755582],
          ...,
          [  -4.06293296,   -1.47831599,   -2.22740778],
          [  -3.79118515,   -1.82996363,   -2.39362346],
          [  -4.25364555,   -1.90993204,   -2.78710827]]],
 
 
        [[[ -65.38614263,  -30.49315838,  -45.18039752],
          [ -64.89151942,  -30.94269337,  -42.84714715],
          [ -6

In [16]:
import mat73
from scipy.io import loadmat

In [10]:
data = loadmat('C:\\Users\\dinar\\Desktop\\simulated_data\\1\\DBS\\newest_timeseries\\GPi_stim\\tvb_timeseries.mat',
               squeeze_me=True)
data['dims']

array(['Time          ', 'State Variable', 'Region        ',
       'Neurons       '], dtype='<U14')

In [53]:
def find_mat_array(mat):
    data = []

    for k, v in mat.items():
        if type(v) not in [bytes, str, list]:
            data.append(k)

    return data


find_mat_array(data)

['dims', 'attrs', 'data', 'coords', 'transient']

In [55]:
data

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Thu Mar 25 16:23:45 2021',
 '__version__': '1.0',
 '__globals__': [],
 'dims': array(['Time          ', 'State Variable', 'Region        ',
        'Neurons       '], dtype='<U14'),
 'attrs': None,
 'data': array([[[9.24158645e-02, 8.96056728e-02, 4.89930394e-02, ...,
          1.01463449e-01, 1.55864629e-02, 5.87542230e-02],
         [1.71684241e+00, 1.73930070e+00, 1.80429717e+00, ...,
          0.00000000e+00, 2.69357687e-04, 1.10159536e-02],
         [1.31146622e+00, 1.31892330e+00, 1.34228813e+00, ...,
          8.75345250e-01, 4.67516590e-01, 6.83007184e-01],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [3.39237277e-01, 3.34011871e-01, 5.75926660e-01, ...,
          3.22753203e-01, 3.02678035e-01, 3.14682057e-01]],
 
        [[9.15398269e-02, 9.09408421e-02, 4.88707572e-02, ...,
          1.00317053e-01, 1.22318579e-02, 5.47590293

['ALS_binary', 'ALS_weighted', 'HC_binary', 'HC_weighted', 'coord', 'regionDescriptions']


In [54]:
type(data['coords'])

numpy.ndarray

In [11]:
data['attrs']

In [12]:
data['data']

array([[[9.24158645e-02, 8.96056728e-02, 4.89930394e-02, ...,
         1.01463449e-01, 1.55864629e-02, 5.87542230e-02],
        [1.71684241e+00, 1.73930070e+00, 1.80429717e+00, ...,
         0.00000000e+00, 2.69357687e-04, 1.10159536e-02],
        [1.31146622e+00, 1.31892330e+00, 1.34228813e+00, ...,
         8.75345250e-01, 4.67516590e-01, 6.83007184e-01],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [3.39237277e-01, 3.34011871e-01, 5.75926660e-01, ...,
         3.22753203e-01, 3.02678035e-01, 3.14682057e-01]],

       [[9.15398269e-02, 9.09408421e-02, 4.88707572e-02, ...,
         1.00317053e-01, 1.22318579e-02, 5.47590293e-02],
        [1.71147984e+00, 1.73721764e+00, 1.80401869e+00, ...,
         4.86993512e-03, 5.56826672e-04, 6.19613245e-03],
        [1.31028333e+00, 1.31882550e+00, 1.34324129e+00, ...,
         9.04258234e-01, 4.82442158e-01, 6.64427306e-01],
        [0.00000000e+00, 0.00000000e+0

In [18]:
dir(data['coords'])

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_finalize__',
 '__array_function__',
 '__array_interface__',
 '__array_prepare__',
 '__array_priority__',
 '__array_struct__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__class_getitem__',
 '__complex__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__divmod__',
 '__dlpack__',
 '__dlpack_device__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__ilshift__',
 '__imatmul__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',

In [36]:
dir(data['coords'])

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_finalize__',
 '__array_function__',
 '__array_interface__',
 '__array_prepare__',
 '__array_priority__',
 '__array_struct__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__class_getitem__',
 '__complex__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__divmod__',
 '__dlpack__',
 '__dlpack_device__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__ilshift__',
 '__imatmul__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',

In [37]:
data['transient']

500.0

In [38]:
data

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Thu Mar 25 16:23:45 2021',
 '__version__': '1.0',
 '__globals__': [],
 'dims': array(['Time          ', 'State Variable', 'Region        ',
        'Neurons       '], dtype='<U14'),
 'attrs': None,
 'data': array([[[9.24158645e-02, 8.96056728e-02, 4.89930394e-02, ...,
          1.01463449e-01, 1.55864629e-02, 5.87542230e-02],
         [1.71684241e+00, 1.73930070e+00, 1.80429717e+00, ...,
          0.00000000e+00, 2.69357687e-04, 1.10159536e-02],
         [1.31146622e+00, 1.31892330e+00, 1.34228813e+00, ...,
          8.75345250e-01, 4.67516590e-01, 6.83007184e-01],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [3.39237277e-01, 3.34011871e-01, 5.75926660e-01, ...,
          3.22753203e-01, 3.02678035e-01, 3.14682057e-01]],
 
        [[9.15398269e-02, 9.09408421e-02, 4.88707572e-02, ...,
          1.00317053e-01, 1.22318579e-02, 5.47590293

In [46]:
dir(data['coords'])

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_finalize__',
 '__array_function__',
 '__array_interface__',
 '__array_prepare__',
 '__array_priority__',
 '__array_struct__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__class_getitem__',
 '__complex__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__divmod__',
 '__dlpack__',
 '__dlpack_device__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__ilshift__',
 '__imatmul__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',

In [52]:
data['coords'].base[0][0]

array(('State Variable', None, array(['S   ', 'Rint', 'R   ', 'Rin ', 'I   '], dtype='<U4')),
      dtype=[('dims', 'O'), ('attrs', 'O'), ('data', 'O')])

In [20]:
mat73.loadmat("C:\\Users\\dinar\\Desktop\\simulated_data\\1\\berlin-unimanual\\AA_20120815_analysis_.mat").keys()

dict_keys(['MSE', 'PSP_baseline', 'dominant_rhythm'])